# Importing libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
import os
import math as mt
from astropy.modeling import models
from astropy import units as u
from scipy.optimize import curve_fit
from astropy.timeseries import LombScargle
from scipy.special import voigt_profile
import Analysis_functions as AF
from importlib import reload
import scipy.stats as ss
import matplotlib.colors as mc
import csv
import pandas as pd
from scipy.interpolate import interp1d
import bindensity as bind
import itertools
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
import spectrum_model as spec_mod

In [ ]:
day = '2023-01-26'
new = True
location = '/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/new_Solar-data/'+day
os.chdir(location)
if not os.path.exists('Plot_Results'):
    os.mkdir('Plot_Results')
    
if not os.path.exists('Final_Plot_Results'):
    os.mkdir('Final_Plot_Results')

save_location = location + '/Final_Plot_Results/'

#Defining directories
blaze_dir = 'Blaze-data'
telluric_dir = 'Telluric-data'
dir_name = 'Corr-data'
CCF_dir_name = 'CCF_RV-data'
rassine_dir_name = 'Rassine-data'
Rassine = True

#Continuum Ranges used for normalization and EW calculation
poly_order = 0
wav_ranges = [(10804.8, 10805.9), (10807.4, 10808.5), (10825.5, 10826.5), (10839, 10840), (10842.4, 10843.1), (10850.2, 10851.2), (10856.5, 10857.9)]

# Importing data

## Test 1

In [ ]:
test = pf.open(dir_name+'/'+sorted(os.listdir(dir_name))[10])

test[0].header['HIERARCH ESO OBS TARG NAME']

In [ ]:
test[4].data[14]

## Test 2

In [ ]:
test_RV = pf.open(telluric_dir+'/'+sorted(os.listdir(telluric_dir))[10])

test_RV.info()

In [ ]:
test_RV[2].data[14]

In [ ]:
plt.plot(test_RV[2].data[14], test_RV[6].data[14])
plt.xlim(10830, 10840)

# Reducing the data

## Reduction diagnostic plots - Plotting RV and chopping limits

In [ ]:
reload(AF)
#Defining a variable to save or not the results.
save = False

#Plotting the RV time series to see the cut and the possible outliers.
    
IQR = np.percentile(tot_RV, 75)-np.percentile(tot_RV, 25)

plt.figure(figsize=[10, 6])
plt.errorbar(tot_date, tot_RV, yerr = tot_RV_err, fmt='.')
plt.axhline((1+20*IQR)*np.median(tot_RV))
plt.axhline((1-20*IQR)*np.median(tot_RV))
plt.xlabel('Time (in MJD)')
plt.ylabel('RV (km/s)')
plt.title('RV Time Series before Correction')
plt.show()

## Reduction diagnostic plots - Before and after reduction

In [ ]:
save=False 

#Plotting the SNR time series to make sure the outliers have been removed
if np.sum(tot_mode=='A') != len(tot_mode) and np.sum(tot_mode=='E')!= len(tot_mode):
    plt.figure(figsize=[10, 6])
    plt.plot(tot_date, tot_SNR, 'b.', linestyle='-', label='Before clipping')
    plt.plot(date_HA, total_SNR_HA, 'r.', linestyle='-', label='After clipping')
    plt.plot(date_HE, total_SNR_HE, 'r.', linestyle='-')
    plt.xlabel('Time (MJD)')
    plt.ylabel('SNR')
    plt.title('SNR time series before and after clipping')
    plt.legend()
else:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[15, 5])
    ax1.plot(tot_date-59970, tot_SNR, 'b.', label='Before clipping')
    ax1.plot(date-59970, total_SNR, 'r.', label='After clipping')
    ax2.plot(tot_date-59970, tot_RV, 'b.', label='Before clipping')
    ax2.plot(date-59970, total_RV, 'r.', label='After clipping')   
    ax1.set_xlabel('Time (MJD-59970)', fontsize=12)
    ax1.set_ylabel('SNR', fontsize=12)
    ax2.set_xlabel('Time (MJD-59970)', fontsize=12)
    ax2.set_ylabel('CCF RV (km/s)', fontsize=12)
    ax1.tick_params(axis='both', labelsize=12)
    ax2.tick_params(axis='both', labelsize=12)
    #ax1.set_title('SNR time series before and after clipping')
    ax1.legend(fontsize=12)
    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Clipping.pdf')

# Main analysis

## Fitting a Gaussian to the He triplet line

### Test - Full line

In [ ]:
reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

lower_lim = 10827.0
#lower_lim = 10830.5
upper_lim = 10834.1

lower_lim_ew = 10831.5
upper_lim_ew = 10834.1
#guess_params = [-0.05, -10, 10833.3, 0.3, 1, 1, 1, 1]
#bounds_param = ((-1, -np.inf, 10833.2, 0, -np.inf, -np.inf, -np.inf, -np.inf),(0, np.inf, 10833.4, np.inf, np.inf, np.inf, np.inf, np.inf)) 

#guess_params = [10, 10833.3, 0.3, -10]
#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
guess_params = [np.log10(17000), 3, 15, 17, 2500, -500, -500, 10, -0.002]
#bounds_param = ((np.log10(10000), 13, 0.99, 0),(np.log10(20000), 14.998990975, 1.01, 1)) 
#Bounds used mainly by curve_fit - Worst fit
#bounds_param = ((np.log10(10000), np.log10(10000), 13, 15, 0, 0, 0, 0.99, 0),
                #(np.log10(20000), np.log10(30000), 16, 20, 1, np.inf, np.inf, 1.01, 1)) 
#Bounds for lmfit - Too loose priors
bounds_param = ((0, 0, 0, 0, 0, -np.inf, -np.inf, -np.inf, -np.inf), (6, 6, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf))
#Better priors - 1st round                                                      #100
#bounds_param_better = ((3, 3, 10, 10, 0, -5000, -5000, -50, -1), (7, 7, 20, 20, 10000, 5000, 5000, 50, 1))
#Better priors - 2nd round - Si temperature was really going to the bottom -> wants to go lower, made lower bound quite low, let's see what happens                                                 
bounds_param_better = ((3.9, 2, 14.3, 15.6, 2000, -1500, -1200, 0, -0.005), (4.5, 3.5, 15.5, 17.1, 3470, 200, 170, 50, 0))

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    def planetary_model_HA(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HA)
        return full_model
    
    def planetary_model_HE(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HE)
        return full_model

    theta_HA, err_theta_HA = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                               rassine_total_norm_err_HA, 'r', wav_ranges)

    theta_HE, err_theta_HE = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                               rassine_total_norm_err_HE, 'k', wav_ranges)
    theta_rassine_HA[:, 0] = np.abs(theta_rassine_HA[:, 0])
    theta_rassine_HE[:, 0] = np.abs(theta_rassine_HE[:, 0])

#If there is one mode of observation
else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he_and_Si(x, poly, 10**log_temp_He, 10**log_temp_Si, 10**log_density_He, 
                                                 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, RV_offset_Si, R_pow = R_power)
        return full_model

    param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
    theta, err_theta, lmfit_thetas, err_lmfit_thetas = AF.fit_spctr_line(planetary_model, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param, total_lamda, total_norm_spctr_Antaress, 
                                         total_norm_err_Antaress, 'r', wav_ranges, param_name, R_power)
    
    better_theta, better_err_theta, better_lmfit_thetas, better_err_lmfit_thetas = AF.fit_spctr_line(planetary_model, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, total_lamda, total_norm_spctr_Antaress, 
                                         total_norm_err_Antaress, 'r', wav_ranges, param_name, R_power)

In [ ]:
fig, ax = plt.subplots(5, 2, sharex = True, figsize = [10, 12])

param = theta
param_err = err_theta
param_lmfit = lmfit_thetas
param_lmfit_err = err_lmfit_thetas
time = date
axs = ax.ravel()
for i in range(len(axs)):
    if i==1:
        axs[i].plot(date, param[:, i], 'b.', label='Curve fit')
        axs[i].plot(date, param_lmfit[:, i], 'r.', label='Lmfit')
    else:
        axs[i].plot(date, param[:, i], 'b.')
        axs[i].plot(date, param_lmfit[:, i], 'r.')
    if i<9:
        axs[i].set_ylabel(param_name[i])
    else:
        axs[i].set_ylabel('Eq. width')

axs[8].set_xlabel('Time (MJD)')
axs[9].set_xlabel('Time (MJD)')

axs[1].legend()
plt.subplots_adjust(hspace=0)
plt.show()

In [ ]:
fig, ax = plt.subplots(5, 2, sharex = True, figsize = [7, 8])

param = better_theta
param_err = better_err_theta
param_lmfit = better_lmfit_thetas
param_lmfit_err = better_err_lmfit_thetas
time = date
axs = ax.ravel()
for i in range(len(axs)):
    if i==1:
        axs[i].plot(date, param[:, i], 'b.', label='Curve fit')
        axs[i].plot(date, param_lmfit[:, i], 'r.', label='Lmfit')
    else:
        axs[i].plot(date, param[:, i], 'b.')
        axs[i].plot(date, param_lmfit[:, i], 'r.')
    if i<9:
        axs[i].set_ylabel(param_name[i])
    else:
        axs[i].set_ylabel('Eq. width')

axs[8].set_xlabel('Time (MJD)')
axs[9].set_xlabel('Time (MJD)')

axs[1].legend()
plt.subplots_adjust(hspace=0)
plt.show()

### Test - Left wing

In [ ]:
reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

lower_lim = 10827
upper_lim = 10834.1

include_ranges = ((10827.0, 10829.5), (10831.5, 10834.1))

lower_lim_ew = 10831.5
upper_lim_ew = 10834.1
#guess_params = [-0.05, -10, 10833.3, 0.3, 1, 1, 1, 1]
#bounds_param = ((-1, -np.inf, 10833.2, 0, -np.inf, -np.inf, -np.inf, -np.inf),(0, np.inf, 10833.4, np.inf, np.inf, np.inf, np.inf, np.inf)) 

#guess_params = [10, 10833.3, 0.3, -10]
#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
guess_params = [np.log10(17000), np.log10(25000), 15, 17, 0.001, 0, 0, 1, 0.00001]
#bounds_param = ((np.log10(10000), 13, 0.99, 0),(np.log10(20000), 14.998990975, 1.01, 1)) 
#Bounds used mainly by curve_fit - Worst fit
#bounds_param = ((np.log10(10000), np.log10(10000), 13, 15, 0, 0, 0, 0.99, 0),
                #(np.log10(20000), np.log10(30000), 16, 20, 1, np.inf, np.inf, 1.01, 1)) 
#Bounds for lmfit
bounds_param = ((0, 0, 0, 0, 0, -np.inf, -np.inf, -np.inf, -np.inf), (6, 6, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf))
#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    def planetary_model_HA(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HA)
        return full_model
    
    def planetary_model_HE(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HE)
        return full_model

    theta_HA, err_theta_HA = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                               rassine_total_norm_err_HA, 'r', wav_ranges)

    theta_HE, err_theta_HE = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                               rassine_total_norm_err_HE, 'k', wav_ranges)
    theta_rassine_HA[:, 0] = np.abs(theta_rassine_HA[:, 0])
    theta_rassine_HE[:, 0] = np.abs(theta_rassine_HE[:, 0])

#If there is one mode of observation
else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he_and_Si(x, poly, 10**log_temp_He, 10**log_temp_Si, 10**log_density_He, 
                                                 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, RV_offset_Si, R_pow = R_power)
        return full_model

    param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
    
    theta, err_theta, lmfit_thetas, err_lmfit_thetas = AF.fit_spctr_line_special(planetary_model, include_ranges, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param, total_lamda, total_norm_spctr_Antaress, 
                                         total_norm_err_Antaress, 'r', wav_ranges, param_name, R_power)

### Test - Right wing

In [ ]:
reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

lower_lim = 10827
upper_lim = 10834.1

include_ranges = ((10830.6, 10831.5), (10831.5, 10834.1))

lower_lim_ew = 10831.5
upper_lim_ew = 10834.1
#guess_params = [-0.05, -10, 10833.3, 0.3, 1, 1, 1, 1]
#bounds_param = ((-1, -np.inf, 10833.2, 0, -np.inf, -np.inf, -np.inf, -np.inf),(0, np.inf, 10833.4, np.inf, np.inf, np.inf, np.inf, np.inf)) 

#guess_params = [10, 10833.3, 0.3, -10]
#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
guess_params = [np.log10(17000), np.log10(25000), 15, 17, 0.001, 0, 0, 1, 0.00001]
#bounds_param = ((np.log10(10000), 13, 0.99, 0),(np.log10(20000), 14.998990975, 1.01, 1)) 
#Bounds used mainly by curve_fit - Worst fit
#bounds_param = ((np.log10(10000), np.log10(10000), 13, 15, 0, 0, 0, 0.99, 0),
                #(np.log10(20000), np.log10(30000), 16, 20, 1, np.inf, np.inf, 1.01, 1)) 
#Bounds for lmfit
bounds_param = ((0, 0, 0, 0, 0, -np.inf, -np.inf, -np.inf, -np.inf), (6, 6, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf))
#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation

#Temperature - 1000 - 10,000,000
#0 - 100 damping parameter (see Wikipedia)
#Density 10^8 - 10^20
#Look into the priors yourself 

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    def planetary_model_HA(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HA)
        return full_model
    
    def planetary_model_HE(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HE)
        return full_model

    theta_HA, err_theta_HA = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                               rassine_total_norm_err_HA, 'r', wav_ranges)

    theta_HE, err_theta_HE = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                               rassine_total_norm_err_HE, 'k', wav_ranges)
    theta_rassine_HA[:, 0] = np.abs(theta_rassine_HA[:, 0])
    theta_rassine_HE[:, 0] = np.abs(theta_rassine_HE[:, 0])

#If there is one mode of observation
else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he_and_Si(x, poly, 10**log_temp_He, 10**log_temp_Si, 10**log_density_He, 
                                                 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, RV_offset_Si, R_pow = R_power)
        return full_model

    param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
    
    theta, err_theta, lmfit_thetas, err_lmfit_thetas = AF.fit_spctr_line_special(planetary_model, include_ranges, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param, total_lamda, total_norm_spctr_Antaress, 
                                         total_norm_err_Antaress, 'r', wav_ranges, param_name, R_power)

### Test - Both wings

In [ ]:
reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

lower_lim = 10827
upper_lim = 10834.1

include_ranges = ((10827.0, 10829.5), (10830.6, 10834.1))

lower_lim_ew = 10831.5
upper_lim_ew = 10834.1
#guess_params = [-0.05, -10, 10833.3, 0.3, 1, 1, 1, 1]
#bounds_param = ((-1, -np.inf, 10833.2, 0, -np.inf, -np.inf, -np.inf, -np.inf),(0, np.inf, 10833.4, np.inf, np.inf, np.inf, np.inf, np.inf)) 

#guess_params = [10, 10833.3, 0.3, -10]
#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
guess_params = [np.log10(17000), np.log10(25000), 15, 17, 0.001, 0, 0, 1, 0.00001]
#bounds_param = ((np.log10(10000), 13, 0.99, 0),(np.log10(20000), 14.998990975, 1.01, 1)) 
#Bounds used mainly by curve_fit - Worst fit
#bounds_param = ((np.log10(10000), np.log10(10000), 13, 15, 0, 0, 0, 0.99, 0),
                #(np.log10(20000), np.log10(30000), 16, 20, 1, np.inf, np.inf, 1.01, 1)) 
#Bounds for lmfit
bounds_param = ((0, 0, 0, 0, 0, -np.inf, -np.inf, -np.inf, -np.inf), (np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf))
#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    def planetary_model_HA(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HA)
        return full_model
    
    def planetary_model_HE(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HE)
        return full_model

    theta_HA, err_theta_HA = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                               rassine_total_norm_err_HA, 'r', wav_ranges)

    theta_HE, err_theta_HE = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                               rassine_total_norm_err_HE, 'k', wav_ranges)
    theta_rassine_HA[:, 0] = np.abs(theta_rassine_HA[:, 0])
    theta_rassine_HE[:, 0] = np.abs(theta_rassine_HE[:, 0])

#If there is one mode of observation
else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he_and_Si(x, poly, 10**log_temp_He, 10**log_temp_Si, 10**log_density_He, 
                                                 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, RV_offset_Si, R_pow = R_power)
        return full_model

    param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
    
    theta, err_theta, lmfit_thetas, err_lmfit_thetas = AF.fit_spctr_line_special(planetary_model, include_ranges, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param, total_lamda, total_norm_spctr_Antaress, 
                                         total_norm_err_Antaress, 'r', wav_ranges, param_name, R_power)

### Test - Voigt core

In [ ]:
reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

lower_lim = 10827
upper_lim = 10834.1

include_ranges = ((10829.6, 10830.5), (10831.5, 10834.1))

lower_lim_ew = 10831.5
upper_lim_ew = 10834.1
#guess_params = [-0.05, -10, 10833.3, 0.3, 1, 1, 1, 1]
#bounds_param = ((-1, -np.inf, 10833.2, 0, -np.inf, -np.inf, -np.inf, -np.inf),(0, np.inf, 10833.4, np.inf, np.inf, np.inf, np.inf, np.inf)) 

#guess_params = [10, 10833.3, 0.3, -10]
#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
guess_params = [np.log10(17000), np.log10(25000), 15, 17, 0.001, 0, 0, 1, 0.00001]
#bounds_param = ((np.log10(10000), 13, 0.99, 0),(np.log10(20000), 14.998990975, 1.01, 1)) 
#Bounds used mainly by curve_fit - Worst fit
#bounds_param = ((np.log10(10000), np.log10(10000), 13, 15, 0, 0, 0, 0.99, 0),
                #(np.log10(20000), np.log10(30000), 16, 20, 1, np.inf, np.inf, 1.01, 1)) 
#Bounds for lmfit
bounds_param = ((0, 0, 0, 0, 0, -np.inf, -np.inf, 0, -np.inf), (6, 6, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf))
#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    def planetary_model_HA(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HA)
        return full_model
    
    def planetary_model_HE(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HE)
        return full_model

    theta_HA, err_theta_HA = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                               rassine_total_norm_err_HA, 'r', wav_ranges)

    theta_HE, err_theta_HE = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                               rassine_total_norm_err_HE, 'k', wav_ranges)
    theta_rassine_HA[:, 0] = np.abs(theta_rassine_HA[:, 0])
    theta_rassine_HE[:, 0] = np.abs(theta_rassine_HE[:, 0])

#If there is one mode of observation
else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he_and_Si(x, poly, 10**log_temp_He, 10**log_temp_Si, 10**log_density_He, 
                                                 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, RV_offset_Si, R_pow = R_power)
        return full_model

    param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
    
    theta, err_theta, lmfit_thetas, err_lmfit_thetas = AF.fit_spctr_line_special(planetary_model, include_ranges, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param, total_lamda, total_norm_spctr_Antaress, 
                                         total_norm_err_Antaress, 'r', wav_ranges, param_name, R_power)

### Test - Gauss core

In [ ]:
reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

lower_lim = 10827
upper_lim = 10834.1

include_ranges = ((10829.6, 10830.5), (10831.5, 10834.1))

lower_lim_ew = 10831.5
upper_lim_ew = 10834.1
#guess_params = [-0.05, -10, 10833.3, 0.3, 1, 1, 1, 1]
#bounds_param = ((-1, -np.inf, 10833.2, 0, -np.inf, -np.inf, -np.inf, -np.inf),(0, np.inf, 10833.4, np.inf, np.inf, np.inf, np.inf, np.inf)) 

#guess_params = [10, 10833.3, 0.3, -10]
#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
guess_params = [np.log10(17000), np.log10(25000), 15, 17, 0, 0, 1, 0.00001]
#bounds_param = ((np.log10(10000), 13, 0.99, 0),(np.log10(20000), 14.998990975, 1.01, 1)) 
#Bounds used mainly by curve_fit - Worst fit
#bounds_param = ((np.log10(10000), np.log10(10000), 13, 15, 0, 0, 0, 0.99, 0),
                #(np.log10(20000), np.log10(30000), 16, 20, 1, np.inf, np.inf, 1.01, 1)) 
#Bounds for lmfit
bounds_param = ((0, 0, 0, 0, -np.inf, -np.inf, 0, -np.inf), (6, 6, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf))
#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    def planetary_model_HA(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HA)
        return full_model
    
    def planetary_model_HE(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HE)
        return full_model

    theta_HA, err_theta_HA = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                               rassine_total_norm_err_HA, 'r', wav_ranges)

    theta_HE, err_theta_HE = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                               rassine_total_norm_err_HE, 'k', wav_ranges)
    theta_rassine_HA[:, 0] = np.abs(theta_rassine_HA[:, 0])
    theta_rassine_HE[:, 0] = np.abs(theta_rassine_HE[:, 0])

#If there is one mode of observation
else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, RV_offset_He, RV_offset_Si, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he_and_Si_Gauss(x, poly, 10**log_temp_He, 10**log_temp_Si, 10**log_density_He, 
                                                 10**log_density_Si, None, 'VACUUM', RV_offset_He, RV_offset_Si, R_pow = R_power)
        return full_model

    param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
    
    theta, err_theta, lmfit_thetas, err_lmfit_thetas = AF.fit_spctr_line_special(planetary_model, include_ranges, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param, total_lamda, total_norm_spctr_Antaress, 
                                         total_norm_err_Antaress, 'r', wav_ranges, param_name, R_power)

### More testing

In [ ]:
%matplotlib inline

h=50
total_range = [10827, 10834.1]
core_range = [10829.78, 10830.38]
left_wing_range = [10827, 10829.7]
right_wing_range = [10830.45, 10834.1]
bound_lamda = AF.bound(total_range[0], total_range[1], total_lamda[h], total_lamda[h])
bound_flux = AF.bound(total_range[0], total_range[1], total_lamda[h], total_norm_spctr_Antaress[h])
#For plotting purposes#
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=[12, 4])
core_lamda = AF.bound(core_range[0], core_range[1], total_lamda[h], total_lamda[h]) 
core_flux = AF.bound(core_range[0], core_range[1], total_lamda[h], total_norm_spctr_Antaress[h]) 
left_wing_lamda = AF.bound(left_wing_range[0], left_wing_range[1], total_lamda[h], total_lamda[h])
left_wing_flux = AF.bound(left_wing_range[0], left_wing_range[1], total_lamda[h], total_norm_spctr_Antaress[h]) 
right_wing_lamda = AF.bound(right_wing_range[0], right_wing_range[1], total_lamda[h], total_lamda[h]) 
right_wing_flux = AF.bound(right_wing_range[0], right_wing_range[1], total_lamda[h], total_norm_spctr_Antaress[h])

ax1.plot(left_wing_lamda, left_wing_flux, 'b')
ax1.plot(core_lamda, core_flux, 'r')
ax1.plot(right_wing_lamda, right_wing_flux, 'b')

##

wing_f = np.zeros(bound_lamda.shape)
core_f = np.zeros(bound_lamda.shape)

wing_lambda = np.append(left_wing_lamda, right_wing_lamda)
core_lambda = np.copy(core_lamda)
print(len(bound_lamda))
for i, x in enumerate(bound_lamda):
    if left_wing_range[0] < x < left_wing_range[1] or right_wing_range[0] < x < right_wing_range[1]:
        wing_f[i] = 1.
    elif core_range[0] < x < core_range[1]:
        core_f[i] = 1.
##

ax1.plot(bound_lamda, wing_f, 'b.')
ax1.plot(bound_lamda, core_f, 'r.')

'''new_wing_lambda = np.copy(bound_lamda)
new_core_lambda = np.copy(bound_lamda)
for i, x in enumerate(len(bound_param)):
    if new_wing_lambda[i] not in wing_lambda:
        new_wing_lambda = 0
    if new_core_lambda[i] not in core_lambda:
        new_core_lambda = 0'''
    
    
model_wings = spec_mod.spectrum_he_and_Si(bound_lamda, wing_f,  10**4.4, 10**5.5, 
                           10**15, 10**16.15, vsini=None, medium='VACUUM', Si_delta_damping=2500, RV_offset_He=-500, 
                            RV_offset_Si=-500, He_delta_damping = 0, R_pow = None, choice = [True,True,True,True], 
                                    micro_t = None)

model_core = spec_mod.spectrum_he_and_Si_Gauss(bound_lamda, core_f,  10**4, 10**5, 
                           10**14, 10**16.1, vsini=None, medium='VACUUM', RV_offset_He=-500, 
                            RV_offset_Si=-500, He_delta_damping = 0, Si_delta_damping=0, R_pow = None, choice = [True,True,True,True], 
                                    micro_t = None)

model_Voigt = spec_mod.spectrum_he_and_Si(bound_lamda, np.ones(bound_lamda.shape),  10**4.5, 10**4.7, 
                           10**15.1, 10**16.27, vsini=None, medium='VACUUM', Si_delta_damping=2500, RV_offset_He=-500, 
                            RV_offset_Si=-500, He_delta_damping = 0, R_pow = None, choice = [True,True,True,True], 
                                    micro_t = None)


ax2.plot(bound_lamda, bound_flux)
ax2.plot(bound_lamda, model_wings, 'k.-',label='Complicated model', alpha=0.3)
ax2.plot(bound_lamda, model_core, 'k--', alpha=0.3)
ax2.plot(bound_lamda, model_core+model_wings, 'k.')

ax3.plot(bound_lamda, bound_flux)
ax3.plot(bound_lamda, model_Voigt, 'g.-')
plt.subplots_adjust(wspace=0)
ax1.set_ylabel('Normalized Flux')
ax1.set_xlabel('Wavlength ($\AA$)')
ax2.set_xlabel('Wavlength ($\AA$)')
ax3.set_xlabel('Wavlength ($\AA$)')
ax1.set_title('Study range')
ax2.set_title('Cut up fit')
ax3.set_title('Together fit')


ax1.set_ylim(0.35, 1.1)

In [ ]:
reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

lower_lim = 10827
upper_lim = 10834.1

lower_lim_ew = [10831.5, 10827]
upper_lim_ew = [10834.1, 10831.5]

ref_Si = AF.air2vac(10827.091)

#guess_params = [-0.05, -10, 10833.3, 0.3, 1, 1, 1, 1]
#bounds_param = ((-1, -np.inf, 10833.2, 0, -np.inf, -np.inf, -np.inf, -np.inf),(0, np.inf, 10833.4, np.inf, np.inf, np.inf, np.inf, np.inf)) 

#guess_params = [10, 10833.3, 0.3, -10]

#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
#guess_params = [np.log10(17000), 4.8, 14.7, 17, 2500, -700, -500, 10, -0.002]

#guess_params with variable range
#guess_params = [np.log10(17000), 4.75, 14.7, 16, 2500, -700, -500, 10, -0.002, 10829.78, 10830.38]
#guess_params = [np.log10(17000), 4.75, 14.7, 16, 2500, -700, -500, 10, -0.002, 10829.5, 10830.5]

#Best one so far - to use if testing of newton doesn't work
#guess_params = [np.log10(17000), 4.75, 14.7, 16, 2500, -700, -500, 10, -0.002, 0.5]
#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
#guess_params = [np.log10(17000), 4.75, 14.7, 16, 2500, -700, -500, -70, 0.01, -5.7e-7]
#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
#guess_params = [np.log10(17000), 4.75, 14.7, 16, 2500, -700, -500, 10, -0.002]
#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
#guess_params = [np.log10(17000), 4.75, 14.7, 16, 2500, -700, -500, 1]
#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']
guess_params = [np.log10(17000), 4.75, 14.7, 16, 2500, -700, -500]

#bounds_param = ((np.log10(10000), 13, 0.99, 0),(np.log10(20000), 14.998990975, 1.01, 1)) 
#Bounds used mainly by curve_fit - Worst fit

#bounds_param = ((np.log10(10000), np.log10(10000), 13, 15, 0, 0, 0, 0.99, 0),
                #(np.log10(20000), np.log10(30000), 16, 20, 1, np.inf, np.inf, 1.01, 1)) 

#Bounds for lmfit - Too loose priors
#bounds_param = ((0, 0, 0, 0, 0, -np.inf, -np.inf, -np.inf, -np.inf, -np.inf, -np.inf), (6, 6, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf))

#Better priors - 1st round                                                      #100
#bounds_param_better = ((3.9, 3, 10, 10, 0, -5000, -5000, 0, -1), (7, 7, 14.8, 20, 10000, 5000, 5000, 50, 1))

#Better priors - 2nd round
#bounds_param_better = ((3.9, 4, 14.3, 15.6, 2000, -1500, -1200, 0, -0.005, 10829, 10830.1), (4.5, 5, 15.5, 17.1, 5000, 200, 170, 50, 0, 10829.85, 10831))

#Best priors - to use if newton doesn't work
#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']
bounds_param_better = ((4.1, 4, 14.3, 15.6, 2000, -1500, -1200), (4.8, 5, 15.5, 17.1, 7000, 200, 170))
#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
#bounds_param_better = ((4.1, 4, 14.3, 15.6, 2000, -1500, -1200, -100, -0.01, -1e-3), (4.5, 5, 15, 17.1, 6000, 200, 170, 100, 0.01, 0))
#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
#bounds_param_better = ((4.1, 4, 14.3, 15.6, 2000, -1500, -1200, 0, -0.005), (4.5, 5, 15, 17.1, 6000, 200, 170, 50, 0))
#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
#bounds_param_better = ((4.1, 3.5, 14.3, 15.4, 2000, -1500, -1200, 0.95), (4.6, 5, 15.5, 17.1, 6000, 200, 170, 1.05))


#Better priors - 2nd round - with variable range

if mode[0] == 'A':
    R_power = 86000
else:
    R_power = 74000

param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']

test_theta, test_err_theta = AF.test_fit_spctr_line(lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                     guess_params, bounds_param_better, total_lamda, total_norm_spctr_Antaress, 
                                     total_norm_err_Antaress, wav_ranges, param_name, '', R_power, poly_order)


In [ ]:
ref_time = 59970
save=True 


theta_in_question = test_theta
err_in_question = test_err_theta
date_in_question = date

num_bins = 20

binned_date = ss.binned_statistic(date_in_question, date_in_question, statistic=np.ma.median, bins=num_bins)[0]

new_param_name = param_name + ['He EW', 'Si EW']

fig, axs = plt.subplots (mt.ceil(theta_in_question.shape[1]/2), 2, sharex=True, figsize=[12, 14])
ax = axs.ravel()

for i in range(theta_in_question.shape[1]):
    avg = np.average(theta_in_question[:, i])
    std = np.sqrt(np.average((theta_in_question[:, i] - avg)**2))
    
    binned_data = ss.binned_statistic(date_in_question, theta_in_question[:, i], statistic=np.ma.median, bins=num_bins)[0]
    
    ax[i].plot(date_in_question - ref_time, theta_in_question[:, i], '.', alpha=0.2)
    ax[i].plot(binned_date - ref_time, binned_data, 'r.', markersize=10)
    
    textstr = '$\sigma$ = %.4f'%std
    ax[i].text(0.04, 0.90, textstr, transform=ax[i].transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    
    ax[i].set_ylabel(new_param_name[i], fontsize=14)
    ax[i].tick_params(axis='y', labelsize=14)
    if i == theta_in_question.shape[1]-1 or i == theta_in_question.shape[1]-2:
        ax[i].tick_params(axis='x', labelsize=14)
        ax[i].set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
        
if not ax[-1].lines: 
    ax[-1].set_visible(False)
        

fig.tight_layout()
plt.subplots_adjust(hspace=0)
if save:
    plt.savefig('/Users/samsonmercier/Downloads/Test_Timeseries_'+str(poly_order)+'th_order.pdf')

In [ ]:
reload(AF)

AF.Correlation_Plot('', test_theta[:, 0], test_theta[:, 1], test_err_theta[:, 0], test_err_theta[:, 1], 'T He', 'T Si', 'Test Antaress Temperature', '2023-01-26')

AF.Correlation_Plot('', test_theta[:, 2], test_theta[:, 3], test_err_theta[:, 2], test_err_theta[:, 3], 'n He', 'n Si', 'Test Antaress Density', '2023-01-26')

AF.Correlation_Plot('', test_theta[:, 5], test_theta[:, 6], test_err_theta[:, 5], test_err_theta[:, 6], 'He RV', 'Si RV', 'Test Antaress RV', '2023-01-26')

AF.Correlation_Plot('', test_theta[:, -2], test_theta[:, -1], test_err_theta[:, -2], test_err_theta[:, -1], 'He EW', 'Si EW', 'Test Antaress Eq. Width', '2023-01-26')


### Fitting wings with Voigt and core with Gauss - differential evolution

In [ ]:
reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)
reload(line_mod)

lower_lim = 10827
upper_lim = 10834.1

include_ranges = ((10829.6, 10830.5), (10831.5, 10834.1))

core_range = [10829.78, 10830.38]
left_wing_range = [10827, 10829.78]
right_wing_range = [10830.38, 10834.1]

lower_lim_ew = [10831.5, 10827]
upper_lim_ew = [10834.1, 10831.5]

ref_Si = AF.air2vac(10827.091)
c_light_m=299792458.

#guess_params = [-0.05, -10, 10833.3, 0.3, 1, 1, 1, 1]
#bounds_param = ((-1, -np.inf, 10833.2, 0, -np.inf, -np.inf, -np.inf, -np.inf),(0, np.inf, 10833.4, np.inf, np.inf, np.inf, np.inf, np.inf)) 

#guess_params = [10, 10833.3, 0.3, -10]

#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
#guess_params = [np.log10(17000), 4.8, 14.7, 17, 2500, -700, -500, 10, -0.002]

#guess_params with variable range
#guess_params = [np.log10(17000), 4.75, 14.7, 16, 2500, -700, -500, 10, -0.002, 10829.78, 10830.38]
#guess_params = [np.log10(17000), 4.75, 14.7, 16, 2500, -700, -500, 10, -0.002, 10829.5, 10830.5]

#Best one so far - to use if testing of newton doesn't work
guess_params = [np.log10(17000), 4.75, 14.7, 16, 2500, -700, -500, 10, -0.002, 0.5]

#bounds_param = ((np.log10(10000), 13, 0.99, 0),(np.log10(20000), 14.998990975, 1.01, 1)) 
#Bounds used mainly by curve_fit - Worst fit

#bounds_param = ((np.log10(10000), np.log10(10000), 13, 15, 0, 0, 0, 0.99, 0),
                #(np.log10(20000), np.log10(30000), 16, 20, 1, np.inf, np.inf, 1.01, 1)) 

#Bounds for lmfit - Too loose priors
#bounds_param = ((0, 0, 0, 0, 0, -np.inf, -np.inf, -np.inf, -np.inf, -np.inf, -np.inf), (6, 6, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf))

#Better priors - 1st round                                                      #100
#bounds_param_better = ((3.9, 3, 10, 10, 0, -5000, -5000, 0, -1), (7, 7, 14.8, 20, 10000, 5000, 5000, 50, 1))

#Better priors - 2nd round
#bounds_param_better = ((3.9, 4, 14.3, 15.6, 2000, -1500, -1200, 0, -0.005, 10829, 10830.1), (4.5, 5, 15.5, 17.1, 5000, 200, 170, 50, 0, 10829.85, 10831))

#Best priors - to use if newton doesn't work
bounds_param_better = ((4.1, 4, 14.3, 15.6, 2000, -1500, -1200, 0, -0.005, 0.1), (4.5, 5, 15, 17.1, 5000, 200, 170, 50, 0, 0.5))


#Better priors - 2nd round - with variable range

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    def planetary_model_HA(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HA)
        return full_model
    
    def planetary_model_HE(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HE)
        return full_model

    theta_HA, err_theta_HA = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                               rassine_total_norm_err_HA, 'r', wav_ranges)

    theta_HE, err_theta_HE = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                               rassine_total_norm_err_HE, 'k', wav_ranges)
    theta_rassine_HA[:, 0] = np.abs(theta_rassine_HA[:, 0])
    theta_rassine_HE[:, 0] = np.abs(theta_rassine_HE[:, 0])

#If there is one mode of observation
else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A, B, ranges):
        #Describing the continuum with a polynomial
        poly = A+B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        lamda_offset_Si = (ref_Si * RV_offset_Si)/c_light_m
        for i, j in enumerate(x):
            if lower_lim <= j <= (ref_Si + lamda_offset_Si - ranges) or (ref_Si + lamda_offset_Si + ranges) <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif (ref_Si + lamda_offset_Si - ranges) <= j <= (ref_Si + lamda_offset_Si + ranges):
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        full_model = model_wings + model_core     
        return full_model


    param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'ranges']
    
    theta, err_theta, lmfit_thetas, err_lmfit_thetas = AF.fit_spctr_line(planetary_model, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, total_lamda, total_norm_spctr_Antaress, 
                                         total_norm_err_Antaress, wav_ranges, param_name, 'differential_evolution')


### Fitting wings with Voigt and core with Gauss - MCMC

In [ ]:
reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

lower_lim = 10827
upper_lim = 10834.1

include_ranges = ((10829.6, 10830.5), (10831.5, 10834.1))

core_range = [10829.78, 10830.38]
left_wing_range = [10827, 10829.78]
right_wing_range = [10830.38, 10834.1]

lower_lim_ew = 10831.5
upper_lim_ew = 10834.1

ref_Si = AF.air2vac(10827.091)
c_light_m=299792458.

#guess_params = [-0.05, -10, 10833.3, 0.3, 1, 1, 1, 1]
#bounds_param = ((-1, -np.inf, 10833.2, 0, -np.inf, -np.inf, -np.inf, -np.inf),(0, np.inf, 10833.4, np.inf, np.inf, np.inf, np.inf, np.inf)) 

#guess_params = [10, 10833.3, 0.3, -10]

#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
#guess_params = [np.log10(17000), 4.8, 14.7, 17, 2500, -700, -500, 10, -0.002]

#guess_params with variable range
#guess_params = [np.log10(17000), 4.75, 14.7, 16, 2500, -700, -500, 10, -0.002, 10829.78, 10830.38]
#guess_params = [np.log10(17000), 4.75, 14.7, 16, 2500, -700, -500, 10, -0.002, 10829.5, 10830.5]
guess_params = [np.log10(17000), 4.75, 14.7, 16, 2500, -700, -500, 10, -0.002, 0.5]
#bounds_param = ((np.log10(10000), 13, 0.99, 0),(np.log10(20000), 14.998990975, 1.01, 1)) 
#Bounds used mainly by curve_fit - Worst fit

#bounds_param = ((np.log10(10000), np.log10(10000), 13, 15, 0, 0, 0, 0.99, 0),
                #(np.log10(20000), np.log10(30000), 16, 20, 1, np.inf, np.inf, 1.01, 1)) 

#Bounds for lmfit - Too loose priors
#bounds_param = ((0, 0, 0, 0, 0, -np.inf, -np.inf, -np.inf, -np.inf, -np.inf, -np.inf), (6, 6, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf))

#Better priors - 1st round                                                      #100
#bounds_param_better = ((3.9, 3, 10, 10, 0, -5000, -5000, 0, -1), (7, 7, 14.8, 20, 10000, 5000, 5000, 50, 1))

#Better priors - 2nd round
#bounds_param_better = ((3.9, 4, 14.3, 15.6, 2000, -1500, -1200, 0, -0.005, 10829, 10830.1), (4.5, 5, 15.5, 17.1, 5000, 200, 170, 50, 0, 10829.85, 10831))
bounds_param_better = ((4.1, 4, 14.3, 15.6, 2000, -1500, -1200, 0, -0.005, 0.1), (4.5, 5, 15, 17.1, 5000, 200, 170, 50, 0, 0.5))

#Better priors - 2nd round - with variable range

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    def planetary_model_HA(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HA)
        return full_model
    
    def planetary_model_HE(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HE)
        return full_model

    theta_HA, err_theta_HA = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                               rassine_total_norm_err_HA, 'r', wav_ranges)

    theta_HE, err_theta_HE = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                               rassine_total_norm_err_HE, 'k', wav_ranges)
    theta_rassine_HA[:, 0] = np.abs(theta_rassine_HA[:, 0])
    theta_rassine_HE[:, 0] = np.abs(theta_rassine_HE[:, 0])

#If there is one mode of observation
else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000

    core_lamda = AF.bound(core_range[0], core_range[1], total_lamda[0], total_lamda[0]) 
    left_wing_lamda = AF.bound(left_wing_range[0], left_wing_range[1], total_lamda[0], total_lamda[0])
    right_wing_lamda = AF.bound(right_wing_range[0], right_wing_range[1], total_lamda[0], total_lamda[0]) 


    wing_lambda = np.append(left_wing_lamda, right_wing_lamda)
    core_lambda = np.copy(core_lamda)

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A, B, ranges):
        #Describing the continuum with a polynomial
        poly = A+B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        lamda_offset_Si = (ref_Si * RV_offset_Si)/c_light_m
        for i, j in enumerate(x):
            if lower_lim <= j <= (ref_Si + lamda_offset_Si - ranges) or (ref_Si + lamda_offset_Si + ranges) <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif (ref_Si + lamda_offset_Si - ranges) <= j <= (ref_Si + lamda_offset_Si + ranges):
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        full_model = model_wings + model_core     
        return full_model


    param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'ranges']
    
    theta, err_theta, lmfit_thetas, err_lmfit_thetas = AF.fit_spctr_line(planetary_model, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, total_lamda, total_norm_spctr_Antaress, 
                                         total_norm_err_Antaress, wav_ranges, param_name, 'emcee')


### Testing

In [ ]:
%matplotlib notebook
reload(AF)
from scipy import optimize

z = 5
def toy_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A, B, ranges):

        lamda_offset_Si = (ref_Si * RV_offset_Si)/c_light_m

        cond_list = [np.logical_or(np.logical_and(lower_lim <= x, x <= (ref_Si + lamda_offset_Si - ranges)), np.logical_and((ref_Si + lamda_offset_Si + ranges) <= x, x <= upper_lim)), np.logical_and((ref_Si + lamda_offset_Si - ranges) <= x, x <= (ref_Si + lamda_offset_Si + ranges))]

        func_list = [lambda x: spec_mod.spectrum_he_and_Si(x, A+B*x,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power), 
                    lambda x: spec_mod.spectrum_he_and_Si_Gauss(x, A+B*x,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)]

        return np.piecewise(x, cond_list, func_list)

    
toy_x = np.linspace(10820, 10840, 100)
toy_y = toy_model(toy_x, 4.3422, 4.9999, 14.927, 16.094, 4839, -1329, -560, 11.66, -0.0009867, 0.3)
plt.plot(total_lamda[z], total_norm_spctr_Antaress[z], '.')
plt.plot(toy_x, toy_y, 'r')
plt.xlim(10827, 10834.1)

bou_x = AF.bound(lower_lim, upper_lim, total_lamda[z], total_lamda[z])
bou_y = AF.bound(lower_lim, upper_lim, total_lamda[z], total_norm_spctr_Antaress[z])
bou_y_err = AF.bound(lower_lim, upper_lim, total_lamda[z], total_norm_err_Antaress[z])

guess = [np.log10(17000), 4.75, 14.7, 16, 2500, -700, -500, 10, -0.002, 0.3]
bou = ((3.9, 4, 14.3, 15.6, 2000, -1500, -1200, 0, -0.005, 0.1), (4.5, 5, 15.5, 17.1, 5000, 200, 170, 50, 0, 1))
para, cov = curve_fit(toy_model, bou_x, bou_y, sigma = bou_y_err, p0 = guess, bounds = bou)

plt.plot(toy_x, toy_model(toy_x, *para), 'k')
plt.show()


print(total_norm_spctr_Antaress[z].shape)
new_flux = AF.bootstrap_generate(total_norm_spctr_Antaress[z], total_norm_err_Antaress[z], 1000)

for c in range(1000):
    plt.plot(total_lamda[z], new_flux[c])
    
plt.xlim(10830, 10835)
plt.show()

### Fitting a gaussian to the He triplet line - Rassine data + Basic Interpolation

In [ ]:
reload(AF)
lower_lim = 10831.5
upper_lim = 10834.1
#guess_params = [-0.05, -10, 10833.3, 0.3, 1, 1, 1, 1]
#bounds_param = ((-1, -np.inf, 10833.2, 0, -np.inf, -np.inf, -np.inf, -np.inf),(0, np.inf, 10833.4, np.inf, np.inf, np.inf, np.inf, np.inf)) 
guess_params = [10, 10833.3, 0.3, -10]
bounds_param = ((-np.inf, 10833.2, 0, -np.inf),(np.inf, 10833.4, np.inf, np.inf)) 

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

    theta_rassine_HA, err_theta_rassine_HA = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               bounds_param, rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                               rassine_total_norm_err_HA, 'r', wav_ranges)

    theta_rassine_HE, err_theta_rassine_HE = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               bounds_param, rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                               rassine_total_norm_err_HE, 'k', wav_ranges)
    theta_rassine_HA[:, 0] = np.abs(theta_rassine_HA[:, 0])
    theta_rassine_HE[:, 0] = np.abs(theta_rassine_HE[:, 0])

#If there is one mode of observation
else:
    theta_rassine, err_theta_rassine = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, lower_lim, upper_lim, guess_params, 
                                         bounds_param, rassine_total_lamda, rassine_total_norm_spctr, 
                                         rassine_total_norm_err, 'r', wav_ranges)

    theta_rassine[:, 0] = np.abs(theta_rassine[:, 0])
    

### Fitting a gaussian to the He triplet line - Normal data + Antaress

In [ ]:
reload(AF)
lower_lim = 10831.5
upper_lim = 10834.1
#guess_params = [-0.05, -10, 10833.3, 0.3, 1, 1, 1, 1]
#bounds_param = ((-1, -np.inf, 10833.2, 0, -np.inf, -np.inf, -np.inf, -np.inf),(0, np.inf, 10833.4, np.inf, np.inf, np.inf, np.inf, np.inf)) 
guess_params = [10, 10833.3, 0.3, 0.001, 0.001, -10]
bounds_param = ((-np.inf, 10833.2, 0, -np.inf, -np.inf, -np.inf),(np.inf, 10833.4, np.inf, np.inf, np.inf, np.inf)) 

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

    theta_Antaress_HA, err_theta_Antaress_HA = AF.fit_spctr_line(AF.new_best_model, lower_lim, upper_lim, guess_params, 
                                               bounds_param, total_lamda_HA, total_norm_spctr_Antaress_HA, 
                                               total_norm_err_Antaress_HA, 'r', wav_ranges)

    theta_Antaress_HE, err_theta_Antaress_HE = AF.fit_spctr_line(AF.new_best_model, lower_lim, upper_lim, guess_params, 
                                               bounds_param, total_lamda_HE, total_norm_spctr_Antaress_HE, 
                                               total_norm_err_Antaress_HE, 'k', wav_ranges)
    
    theta_Antaress_HA[:, 0] = np.abs(theta_Antaress_HA[:, 0])
    theta_Antaress_HE[:, 0] = np.abs(theta_Antaress_HE[:, 0])
#If there is one mode of observation
else:
    theta_Antaress, err_theta_Antaress = AF.fit_spctr_line(AF.new_best_model, lower_lim, upper_lim, lower_lim, upper_lim, guess_params, 
                                         bounds_param, total_lamda, total_norm_spctr_Antaress, 
                                         total_norm_err_Antaress, 'r', wav_ranges)

    theta_Antaress[:, 0] = np.abs(theta_Antaress[:, 0])
    

### Fitting wings with Voigt and core with Gauss - resolving continuity issues - Antaress

In [ ]:
reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

lower_lim = 10827
upper_lim = 10834.1

lower_lim_ew = [10831.5, 10827]
upper_lim_ew = [10834.1, 10831.5]

ref_Si = AF.air2vac(10827.091)

#guess_params = [-0.05, -10, 10833.3, 0.3, 1, 1, 1, 1]
#bounds_param = ((-1, -np.inf, 10833.2, 0, -np.inf, -np.inf, -np.inf, -np.inf),(0, np.inf, 10833.4, np.inf, np.inf, np.inf, np.inf, np.inf)) 

#guess_params = [10, 10833.3, 0.3, -10]

#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
#guess_params = [np.log10(17000), 4.8, 14.7, 17, 2500, -700, -500, 10, -0.002]

#guess_params with variable range
#guess_params = [np.log10(17000), 4.75, 14.7, 16, 2500, -700, -500, 10, -0.002, 10829.78, 10830.38]
#guess_params = [np.log10(17000), 4.75, 14.7, 16, 2500, -700, -500, 10, -0.002, 10829.5, 10830.5]

#Best one so far - to use if testing of newton doesn't work
#guess_params = [np.log10(17000), 4.75, 14.7, 16, 2500, -700, -500, 10, -0.002, 0.5]
#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
#guess_params = [np.log10(17000), 4.75, 14.7, 16, 2500, -700, -500, -70, 0.01, -5.7e-7]
#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
#guess_params = [np.log10(17000), 4.75, 14.7, 16, 2500, -700, -500, 10, -0.002]
#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
#guess_params = [np.log10(17000), 4.75, 14.7, 16, 2500, -700, -500, 1]
#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']
guess_params = [np.log10(17000), 4.75, 14.7, 16, 2500, -700, -500]

#bounds_param = ((np.log10(10000), 13, 0.99, 0),(np.log10(20000), 14.998990975, 1.01, 1)) 
#Bounds used mainly by curve_fit - Worst fit

#bounds_param = ((np.log10(10000), np.log10(10000), 13, 15, 0, 0, 0, 0.99, 0),
                #(np.log10(20000), np.log10(30000), 16, 20, 1, np.inf, np.inf, 1.01, 1)) 

#Bounds for lmfit - Too loose priors
#bounds_param = ((0, 0, 0, 0, 0, -np.inf, -np.inf, -np.inf, -np.inf, -np.inf, -np.inf), (6, 6, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf))

#Better priors - 1st round                                                      #100
#bounds_param_better = ((3.9, 3, 10, 10, 0, -5000, -5000, 0, -1), (7, 7, 14.8, 20, 10000, 5000, 5000, 50, 1))

#Better priors - 2nd round
#bounds_param_better = ((3.9, 4, 14.3, 15.6, 2000, -1500, -1200, 0, -0.005, 10829, 10830.1), (4.5, 5, 15.5, 17.1, 5000, 200, 170, 50, 0, 10829.85, 10831))

#Best priors - to use if newton doesn't work
#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']
bounds_param_better = ((4.1, 4, 14.3, 15.6, 2000, -1500, -1200), (4.8, 5, 15.5, 17.1, 7000, 200, 170))
#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
#bounds_param_better = ((4.1, 4, 14.3, 15.6, 2000, -1500, -1200, -100, -0.01, -1e-3), (4.5, 5, 15, 17.1, 6000, 200, 170, 100, 0.01, 0))
#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']
#bounds_param_better = ((4.1, 4, 14.3, 15.6, 2000, -1500, -1200, 0, -0.005), (4.5, 5, 15, 17.1, 6000, 200, 170, 50, 0))
#['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
#bounds_param_better = ((4.1, 3.5, 14.3, 15.4, 2000, -1500, -1200, 0.95), (4.6, 5, 15.5, 17.1, 6000, 200, 170, 1.05))


#Better priors - 2nd round - with variable range

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    def planetary_model_HA(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HA)
        return full_model
    
    def planetary_model_HE(x, log_temp, log_density, A, B):
        #Describing the continuum with a polynomial
        poly = A+B*x
        #Putting model together
        full_model = spec_mod.spectrum_he(x, poly, 10**log_temp, 10**log_density, None, 'VACUUM', R_pow = R_power_HE)
        return full_model

    theta_HA, err_theta_HA = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                               rassine_total_norm_err_HA, 'r', wav_ranges)

    theta_HE, err_theta_HE = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_lim, upper_lim, guess_params, 
                                               [], rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                               rassine_total_norm_err_HE, 'k', wav_ranges)
    theta_rassine_HA[:, 0] = np.abs(theta_rassine_HA[:, 0])
    theta_rassine_HE[:, 0] = np.abs(theta_rassine_HE[:, 0])

#If there is one mode of observation
else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000
        
    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power)
        #Describing the continuum with a polynomial
        #poly = A*np.ones(x.shape)
        #poly = p_continuum[0]+ p_continuum[1]*x + (p_continuum[2]*x**2)
        #poly = A + B*x + C*x**2
        poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        full_model = model_wings + model_core
        
        return full_model


    param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']
    
    theta, err_theta = AF.fit_spctr_line(planetary_model, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, total_lamda, total_norm_spctr_Antaress, 
                                         total_norm_err_Antaress, wav_ranges, param_name, '', R_power, poly_order)


## Fitting a Voigt profile to the Si line

### Fitting a Voigt profile to the Si line - Rassine data + Basic Interpolation

In [ ]:
reload(AF)

lower_l = 10828
upper_l = 10832
guess_para = [10, 10830, 0.3, 0.001, -10]
bound = ((-np.inf, 10828, 0, -np.inf, -np.inf),(np.inf, 10832, np.inf, np.inf, np.inf)) 

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

    theta_Si_rassine_HA, err_theta_Si_rassine_HA = AF.fit_spctr_line(AF.new_best_model_Voigt_Rassine, lower_l, upper_l, guess_para, bound, 
                  rassine_total_lamda_HA, rassine_total_norm_spctr_HA, rassine_total_norm_err_HA, 'r', wav_ranges)
    
    theta_Si_rassine_HE, err_theta_Si_rassine_HE = AF.fit_spctr_line(AF.new_best_model_Voigt_Rassine, lower_l, upper_l, guess_para, bound, 
                  rassine_total_lamda_HE, rassine_total_norm_spctr_HE, rassine_total_norm_err_HE, 'k', wav_ranges)

#If there is one mode of observation
else:
    
    theta_Si_rassine, err_theta_Si_rassine = AF.fit_spctr_line(AF.new_best_model_Voigt_Rassine, lower_l, upper_l, lower_l, upper_l, guess_para, bound, 
                  rassine_total_lamda, rassine_total_norm_spctr, rassine_total_norm_err, 'r', wav_ranges)
    

### Fitting a Voigt profile to the Si line - Normal data + Antaress

In [ ]:
reload(AF)

lower_l = 10828
upper_l = 10832
guess_para = [10, 10830, 0.3, 0.001, 0.001, 0.001, -10]
bound = ((-np.inf, 10828, 0, -np.inf, -np.inf, -np.inf, -np.inf),(np.inf, 10832, np.inf, np.inf, np.inf, np.inf, np.inf)) 

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

    theta_Si_Antaress_HA, err_theta_Si_Antaress_HA = AF.fit_spctr_line(AF.new_best_model_Voigt, lower_l, upper_l, guess_para, bound, 
                  total_lamda_HA, total_norm_spctr_Antaress_HA, total_norm_err_Antaress_HA, 'r', wav_ranges)
    
    theta_Si_Antaress_HE, err_theta_Si_Antaress_HE = AF.fit_spctr_line(AF.new_best_model_Voigt, lower_l, upper_l, guess_para, bound, 
                  total_lamda_HE, total_norm_spctr_Antaress_HE, total_norm_err_Antaress_HE, 'k', wav_ranges)

#If there is one mode of observation
else:
    
    theta_Si_Antaress, err_theta_Si_Antaress = AF.fit_spctr_line(AF.new_best_model_Voigt, lower_l, upper_l, lower_l, upper_l, guess_para, bound, 
                  total_lamda, total_norm_spctr_Antaress, total_norm_err_Antaress, 'r', wav_ranges)


## Fitting a Gaussian profile to the Si line

### Fitting a Gaussian profile to the Si line - Rassine data + Basic Interpolation

In [ ]:
reload(AF)

#lower_l = 10827.5
#upper_l = 10832
lower_l = 10828.7
upper_l = 10831.5
guess_para = [10, 10830, 0.3, -10]
bound = ((-np.inf, 10828, 0, -np.inf),(np.inf, 10832, np.inf, np.inf)) 

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

    thetaG_Si_rassine_HA, err_thetaG_Si_rassine_HA = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_l, upper_l, guess_para, bound, 
                  rassine_total_lamda_HA, rassine_total_norm_spctr_HA, rassine_total_norm_err_HA, 'r', wav_ranges)
    
    thetaG_Si_rassine_HE, err_thetaG_Si_rassine_HE = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_l, upper_l, guess_para, bound, 
                  rassine_total_lamda_HE, rassine_total_norm_spctr_HE, rassine_total_norm_err_HE, 'k', wav_ranges)

#If there is one mode of observation
else:
    
    thetaG_Si_rassine, err_thetaG_Si_rassine = AF.fit_spctr_line(AF.new_best_model_Rassine, lower_l, upper_l, lower_l, upper_l, guess_para, bound, 
                  rassine_total_lamda, rassine_total_norm_spctr, rassine_total_norm_err, 'r', wav_ranges)

### Fitting a Gaussian profile to the Si line - Normal data + Antaress

In [ ]:
reload(AF)

lower_l = 10827.5
upper_l = 10832
guess_para = [10, 10830, 0.3, 0.001, 0.0001, -10]
bound = ((-np.inf, 10828, 0, -np.inf, -np.inf, -np.inf),(np.inf, 10832, np.inf, np.inf, np.inf, np.inf)) 

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

    thetaG_Si_Antaress_HA, err_thetaG_Si_Antaress_HA = AF.fit_spctr_line(AF.new_best_model, lower_l, upper_l, guess_para, bound, 
                  total_lamda_HA, total_norm_spctr_Antaress_HA, total_norm_err_Antaress_HA, 'r', wav_ranges)
    
    thetaG_Si_Antaress_HE, err_thetaG_Si_Antaress_HE = AF.fit_spctr_line(AF.new_best_model, lower_l, upper_l, guess_para, bound, 
                  total_lamda_HE, total_norm_spctr_Antaress_HE, total_norm_err_Antaress_HE, 'k', wav_ranges)

#If there is one mode of observation
else:
    
    thetaG_Si_Antaress, err_thetaG_Si_Antaress = AF.fit_spctr_line(AF.new_best_model, lower_l, upper_l, lower_l, upper_l, guess_para, bound, 
                  total_lamda, total_norm_spctr_Antaress, total_norm_err_Antaress, 'r', wav_ranges)

## Plotting the positions, depths and widths of the fitted Voigt profile for Si

### Plotting the positions, depths and widths of the fitted Voigt profile for Si - Rassine data + Basic Interpolation

In [ ]:
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
save = False

ref_Si = AF.air2vac(10827.091) #From NIST database

ref_time = 59960

depth_index = 0
position_index = 1
width_index = 2
ew_index = -1

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    #HA
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Getting the scatter
    avg_depth = np.average(theta_Si_rassine_HA[:, depth_index], weights=1/err_theta_Si_rassine_HA[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Si_rassine_HA[:, depth_index] - avg_depth)**2, weights=1/err_theta_Si_rassine_HA[:, depth_index]**2))
    new_pos = (theta_Si_rassine_HA[:, position_index]-ref_Si)*3e8/ref_Si
    new_pos_err = err_theta_Si_rassine_HA[:, position_index]*3e8/ref_Si
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Si_rassine_HA[:, width_index], weights=1/err_theta_Si_rassine_HA[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Si_rassine_HA[:, width_index] - avg_width)**2, weights=1/err_theta_Si_rassine_HA[:, width_index]**2))
    avg_ew = np.average(theta_Si_rassine_HA[:, ew_index], weights=1/err_theta_Si_rassine_HA[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Si_rassine_HA[:, ew_index] - avg_ew)**2, weights=1/err_theta_Si_rassine_HA[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HA, date_HA, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HA-ref_time, theta_Si_rassine_HA[:, depth_index], yerr=err_theta_Si_rassine_HA[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_HA-ref_time, (theta_Si_rassine_HA[:, position_index]-ref_Si)*3e8/ref_Si, yerr=err_theta_Si_rassine_HA[:, position_index]*3e8/ref_Si, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_Si)*3e8/ref_Si, yerr=binned_pos_err*3e8/ref_Si, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HA-ref_time, theta_Si_rassine_HA[:, width_index], yerr=err_theta_Si_rassine_HA[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HA-ref_time, theta_Si_rassine_HA[:, ew_index], yerr=err_theta_Si_rassine_HA[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Voigt_Fit_Rassine_HA.pdf')
    plt.show()


    #HE
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Getting the scatter
    avg_depth = np.average(theta_Si_rassine_HE[:, depth_index], weights=1/err_theta_Si_rassine_HE[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Si_rassine_HE[:, depth_index] - avg_depth)**2, weights=1/err_theta_Si_rassine_HE[:, depth_index]**2))
    new_pos = (theta_Si_rassine_HE[:, position_index]-ref_Si)*3e8/ref_Si
    new_pos_err = err_theta_Si_rassine_HE[:, position_index]*3e8/ref_Si
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Si_rassine_HE[:, width_index], weights=1/err_theta_Si_rassine_HE[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Si_rassine_HE[:, width_index] - avg_width)**2, weights=1/err_theta_Si_rassine_HE[:, width_index]**2))
    avg_ew = np.average(theta_Si_rassine_HE[:, ew_index], weights=1/err_theta_Si_rassine_HE[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Si_rassine_HE[:, ew_index] - avg_ew)**2, weights=1/err_theta_Si_rassine_HE[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HE, date_HE, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HE-ref_time, theta_Si_rassine_HE[:, depth_index], yerr=err_theta_Si_rassine_HE[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_HE-ref_time, (theta_Si_rassine_HE[:, position_index]-ref_Si)*3e8/ref_Si, yerr=err_theta_Si_rassine_HE[:, position_index]*3e8/ref_Si, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_Si)*3e8/ref_Si, yerr=binned_pos_err*3e8/ref_Si, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HE-ref_time, theta_Si_rassine_HE[:, width_index], yerr=err_theta_Si_rassine_HE[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HE-ref_time, theta_Si_rassine_HE[:, ew_index], yerr=err_theta_Si_rassine_HE[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Voigt_Fit_Rassine_HE.pdf')
    plt.show()

#If there one mode of observation.   
else:
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Getting the scatter
    avg_depth = np.average(theta_Si_rassine[:, depth_index], weights=1/err_theta_Si_rassine[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Si_rassine[:, depth_index] - avg_depth)**2, weights=1/err_theta_Si_rassine[:, depth_index]**2))
    new_pos = (theta_Si_rassine[:, position_index]-ref_Si)*3e8/ref_Si
    new_pos_err = err_theta_Si_rassine[:, position_index]*3e8/ref_Si
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Si_rassine[:, width_index], weights=1/err_theta_Si_rassine[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Si_rassine[:, width_index] - avg_width)**2, weights=1/err_theta_Si_rassine[:, width_index]**2))
    avg_ew = np.average(theta_Si_rassine[:, ew_index], weights=1/err_theta_Si_rassine[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Si_rassine[:, ew_index] - avg_ew)**2, weights=1/err_theta_Si_rassine[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date, date, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date, theta_Si_rassine[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date, theta_Si_rassine[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date, theta_Si_rassine[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date, theta_Si_rassine[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date, theta_Si_rassine[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date, theta_Si_rassine[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date, theta_Si_rassine[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date, theta_Si_rassine[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date-ref_time, theta_Si_rassine[:, depth_index], yerr=err_theta_Si_rassine[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date-ref_time, (theta_Si_rassine[:, position_index]-ref_Si)*3e8/ref_Si, yerr=err_theta_Si_rassine[:, position_index]*3e8/ref_Si, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_Si)*3e8/ref_Si, yerr=binned_pos_err*3e8/ref_Si, fmt='r.', markersize=10)
    
    ax3.errorbar(date-ref_time, theta_Si_rassine[:, width_index], yerr=err_theta_Si_rassine[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date-ref_time, theta_Si_rassine[:, ew_index], yerr=err_theta_Si_rassine[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Voigt_Fit_Rassine.pdf')
    plt.show()

### Plotting the positions, depths and widths of the fitted Voigt profile for Si - Normal data + Antaress


In [ ]:
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
save = False

ref_Si = AF.air2vac(10827.091) #From NIST database

ref_time = 59960

depth_index = 0
position_index = 1
width_index = 2
ew_index = -1

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    #HA
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Getting the scatter
    avg_depth = np.average(theta_Si_Antaress_HA[:, depth_index], weights=1/err_theta_Si_Antaress_HA[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Si_Antaress_HA[:, depth_index] - avg_depth)**2, weights=1/err_theta_Si_Antaress_HA[:, depth_index]**2))
    new_pos = (theta_Si_Antaress_HA[:, position_index]-ref_Si)*3e8/ref_Si
    new_pos_err = err_theta_Si_Antaress_HA[:, position_index]*3e8/ref_Si
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Si_Antaress_HA[:, width_index], weights=1/err_theta_Si_Antaress_HA[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Si_Antaress_HA[:, width_index] - avg_width)**2, weights=1/err_theta_Si_Antaress_HA[:, width_index]**2))
    avg_ew = np.average(theta_Si_Antaress_HA[:, ew_index], weights=1/err_theta_Si_Antaress_HA[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Si_Antaress_HA[:, ew_index] - avg_ew)**2, weights=1/err_theta_Si_Antaress_HA[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HA, date_HA, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HA-ref_time, theta_Si_Antaress_HA[:, depth_index], yerr=err_theta_Si_Antaress_HA[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_HA-ref_time, (theta_Si_Antaress_HA[:, position_index]-ref_Si)*3e8/ref_Si, yerr=err_theta_Si_Antaress_HA[:, position_index]*3e8/ref_Si, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_Si)*3e8/ref_Si, yerr=binned_pos_err*3e8/ref_Si, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HA-ref_time, theta_Si_Antaress_HA[:, width_index], yerr=err_theta_Si_Antaress_HA[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HA-ref_time, theta_Si_Antaress_HA[:, ew_index], yerr=err_theta_Si_Antaress_HA[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Voigt_Fit_Antaress_HA.pdf')
    plt.show()


    #HE
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Getting the scatter
    avg_depth = np.average(theta_Si_Antaress_HE[:, depth_index], weights=1/err_theta_Si_Antaress_HE[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Si_Antaress_HE[:, depth_index] - avg_depth)**2, weights=1/err_theta_Si_Antaress_HE[:, depth_index]**2))
    new_pos = (theta_Si_Antaress_HE[:, position_index]-ref_Si)*3e8/ref_Si
    new_pos_err = err_theta_Si_Antaress_HE[:, position_index]*3e8/ref_Si
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Si_Antaress_HE[:, width_index], weights=1/err_theta_Si_Antaress_HE[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Si_Antaress_HE[:, width_index] - avg_width)**2, weights=1/err_theta_Si_Antaress_HE[:, width_index]**2))
    avg_ew = np.average(theta_Si_Antaress_HE[:, ew_index], weights=1/err_theta_Si_Antaress_HE[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Si_Antaress_HE[:, ew_index] - avg_ew)**2, weights=1/err_theta_Si_Antaress_HE[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HE, date_HE, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HE-ref_time, theta_Si_Antaress_HE[:, depth_index], yerr=err_theta_Si_Antaress_HE[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_HE-ref_time, (theta_Si_Antaress_HE[:, position_index]-ref_Si)*3e8/ref_Si, yerr=err_theta_Si_Antaress_HE[:, position_index]*3e8/ref_Si, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_Si)*3e8/ref_Si, yerr=binned_pos_err*3e8/ref_Si, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HE-ref_time, theta_Si_Antaress_HE[:, width_index], yerr=err_theta_Si_Antaress_HE[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HE-ref_time, theta_Si_Antaress_HE[:, ew_index], yerr=err_theta_Si_Antaress_HE[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Voigt_Fit_Antaress_HE.pdf')
    plt.show()

#If there one mode of observation.   
else:
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Getting the scatter
    avg_depth = np.average(theta_Si_Antaress[:, depth_index], weights=1/err_theta_Si_Antaress[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Si_Antaress[:, depth_index] - avg_depth)**2, weights=1/err_theta_Si_Antaress[:, depth_index]**2))
    new_pos = (theta_Si_Antaress[:, position_index]-ref_Si)*3e8/ref_Si
    new_pos_err = err_theta_Si_Antaress[:, position_index]*3e8/ref_Si
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Si_Antaress[:, width_index], weights=1/err_theta_Si_Antaress[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Si_Antaress[:, width_index] - avg_width)**2, weights=1/err_theta_Si_Antaress[:, width_index]**2))
    avg_ew = np.average(theta_Si_Antaress[:, ew_index], weights=1/err_theta_Si_Antaress[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Si_Antaress[:, ew_index] - avg_ew)**2, weights=1/err_theta_Si_Antaress[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date, date, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date, theta_Si_Antaress[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date, theta_Si_Antaress[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date, theta_Si_Antaress[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date, theta_Si_Antaress[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date, theta_Si_Antaress[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date, theta_Si_Antaress[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date, theta_Si_Antaress[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date, theta_Si_Antaress[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date-ref_time, theta_Si_Antaress[:, depth_index], yerr=err_theta_Si_Antaress[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date-ref_time, (theta_Si_Antaress[:, position_index]-ref_Si)*3e8/ref_Si, yerr=err_theta_Si_Antaress[:, position_index]*3e8/ref_Si, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_Si)*3e8/ref_Si, yerr=binned_pos_err*3e8/ref_Si, fmt='r.', markersize=10)
    
    ax3.errorbar(date-ref_time, theta_Si_Antaress[:, width_index], yerr=err_theta_Si_Antaress[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date-ref_time, theta_Si_Antaress[:, ew_index], yerr=err_theta_Si_Antaress[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Voigt_Fit_Antaress.pdf')
    plt.show()

### Plotting the positions, depths and widths of the fitted Voigt profile for Si - Overplot (Rassine + Antaress)


In [ ]:
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
save = False

ref_He = AF.air2vac((10830.33977+10830.25010)/2) #From NIST database

ref_time = 59960

depth_index = 0
position_index = 1
width_index = 2
ew_index = -1

opacity = 0.1

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    #HA
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Antaress
    #Getting the scatter
    avg_depth = np.average(theta_Si_Antaress_HA[:, depth_index], weights=1/err_theta_Si_Antaress_HA[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Si_Antaress_HA[:, depth_index] - avg_depth)**2, weights=1/err_theta_Si_Antaress_HA[:, depth_index]**2))
    new_pos = (theta_Si_Antaress_HA[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_theta_Si_Antaress_HA[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Si_Antaress_HA[:, width_index], weights=1/err_theta_Si_Antaress_HA[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Si_Antaress_HA[:, width_index] - avg_width)**2, weights=1/err_theta_Si_Antaress_HA[:, width_index]**2))
    avg_ew = np.average(theta_Si_Antaress_HA[:, ew_index], weights=1/err_theta_Si_Antaress_HA[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Si_Antaress_HA[:, ew_index] - avg_ew)**2, weights=1/err_theta_Si_Antaress_HA[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HA, date_HA, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HA, theta_Si_Antaress_HA[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HA-ref_time, theta_Si_Antaress_HA[:, depth_index], yerr=err_theta_Si_Antaress_HA[:, depth_index], fmt='r.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
  
    ax2.errorbar(date_HA-ref_time, (theta_Si_Antaress_HA[:, position_index]-ref_He)*3e8/ref_He, yerr=err_theta_Si_Antaress_HA[:, position_index]*3e8/ref_He, fmt='r.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HA-ref_time, theta_Si_Antaress_HA[:, width_index], yerr=err_theta_Si_Antaress_HA[:, width_index], fmt='r.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HA-ref_time, theta_Si_Antaress_HA[:, ew_index], yerr=err_theta_Si_Antaress_HA[:,ew_index], fmt='r.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
        
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    
    #Rassine
    #Getting the scatter
    avg_depth = np.average(theta_Si_rassine_HA[:, depth_index], weights=1/err_theta_Si_rassine_HA[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Si_rassine_HA[:, depth_index] - avg_depth)**2, weights=1/err_theta_Si_rassine_HA[:, depth_index]**2))
    new_pos = (theta_Si_rassine_HA[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_theta_Si_rassine_HA[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Si_rassine_HA[:, width_index], weights=1/err_theta_Si_rassine_HA[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Si_rassine_HA[:, width_index] - avg_width)**2, weights=1/err_theta_Si_rassine_HA[:, width_index]**2))
    avg_ew = np.average(theta_Si_rassine_HA[:, ew_index], weights=1/err_theta_Si_rassine_HA[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Si_rassine_HA[:, ew_index] - avg_ew)**2, weights=1/err_theta_Si_rassine_HA[:, ew_index]**2))

    #Getting the binned points
    num_bins = 20
        
    binned_depth = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HA, theta_Si_rassine_HA[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HA-ref_time, theta_Si_rassine_HA[:, depth_index], yerr=err_theta_Si_rassine_HA[:, depth_index], fmt='b.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='b.', markersize=10)
  
    ax2.errorbar(date_HA-ref_time, (theta_Si_rassine_HA[:, position_index]-ref_He)*3e8/ref_He, yerr=err_theta_Si_rassine_HA[:, position_index]*3e8/ref_He, fmt='b.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='b.', markersize=10)
    
    ax3.errorbar(date_HA-ref_time, theta_Si_rassine_HA[:, width_index], yerr=err_theta_Si_rassine_HA[:, width_index], fmt='b.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='b.', markersize=10)
    
    ax4.errorbar(date_HA-ref_time, theta_Si_rassine_HA[:, ew_index], yerr=err_theta_Si_rassine_HA[:,ew_index], fmt='b.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='b.', markersize=10)

    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)

    textstr12 = textstr1+'\n'+'$\sigma$ = %.3f'%std_depth
    textstr22 = textstr2+'\n'+'$\sigma$ = %.3f'%std_pos
    textstr32 = textstr3+'\n'+'$\sigma$ = %.3f'%std_width
    textstr42 = textstr4+'\n'+'$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.90, textstr12, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.90, textstr22, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.90, textstr32, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.90, textstr42, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Voigt_Fit_Overplot_HA.pdf')
    plt.show()


    #HE
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Antaress
    #Getting the scatter
    avg_depth = np.average(theta_Si_Antaress_HE[:, depth_index], weights=1/err_theta_Si_Antaress_HE[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Si_Antaress_HE[:, depth_index] - avg_depth)**2, weights=1/err_theta_Si_Antaress_HE[:, depth_index]**2))
    new_pos = (theta_Si_Antaress_HE[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_theta_Si_Antaress_HE[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Si_Antaress_HE[:, width_index], weights=1/err_theta_Si_Antaress_HE[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Si_Antaress_HE[:, width_index] - avg_width)**2, weights=1/err_theta_Si_Antaress_HE[:, width_index]**2))
    avg_ew = np.average(theta_Si_Antaress_HE[:, ew_index], weights=1/err_theta_Si_Antaress_HE[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Si_Antaress_HE[:, ew_index] - avg_ew)**2, weights=1/err_theta_Si_Antaress_HE[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HE, date_HE, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HE, theta_Si_Antaress_HE[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HE-ref_time, theta_Si_Antaress_HE[:, depth_index], yerr=err_theta_Si_Antaress_HE[:, depth_index], fmt='r.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
  
    ax2.errorbar(date_HE-ref_time, (theta_Si_Antaress_HE[:, position_index]-ref_He)*3e8/ref_He, yerr=err_theta_Si_Antaress_HE[:, position_index]*3e8/ref_He, fmt='r.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HE-ref_time, theta_Si_Antaress_HE[:, width_index], yerr=err_theta_Si_Antaress_HE[:, width_index], fmt='r.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HE-ref_time, theta_Si_Antaress_HE[:, ew_index], yerr=err_theta_Si_Antaress_HE[:,ew_index], fmt='r.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
        
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    
    #Rassine
    #Getting the scatter
    avg_depth = np.average(theta_Si_rassine_HE[:, depth_index], weights=1/err_theta_Si_rassine_HE[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Si_rassine_HE[:, depth_index] - avg_depth)**2, weights=1/err_theta_Si_rassine_HE[:, depth_index]**2))
    new_pos = (theta_Si_rassine_HE[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_theta_Si_rassine_HE[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Si_rassine_HE[:, width_index], weights=1/err_theta_Si_rassine_HE[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Si_rassine_HE[:, width_index] - avg_width)**2, weights=1/err_theta_Si_rassine_HE[:, width_index]**2))
    avg_ew = np.average(theta_Si_rassine_HE[:, ew_index], weights=1/err_theta_Si_rassine_HE[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Si_rassine_HE[:, ew_index] - avg_ew)**2, weights=1/err_theta_Si_rassine_HE[:, ew_index]**2))

    #Getting the binned points
    num_bins = 20
        
    binned_depth = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HE, theta_Si_rassine_HE[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HE-ref_time, theta_Si_rassine_HE[:, depth_index], yerr=err_theta_Si_rassine_HE[:, depth_index], fmt='b.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='b.', markersize=10)
  
    ax2.errorbar(date_HE-ref_time, (theta_Si_rassine_HE[:, position_index]-ref_He)*3e8/ref_He, yerr=err_theta_Si_rassine_HE[:, position_index]*3e8/ref_He, fmt='b.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='b.', markersize=10)
    
    ax3.errorbar(date_HE-ref_time, theta_Si_rassine_HE[:, width_index], yerr=err_theta_Si_rassine_HE[:, width_index], fmt='b.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='b.', markersize=10)
    
    ax4.errorbar(date_HE-ref_time, theta_Si_rassine_HE[:, ew_index], yerr=err_theta_Si_rassine_HE[:,ew_index], fmt='b.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='b.', markersize=10)

    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)

    textstr12 = textstr1+'\n'+'$\sigma$ = %.3f'%std_depth
    textstr22 = textstr2+'\n'+'$\sigma$ = %.3f'%std_pos
    textstr32 = textstr3+'\n'+'$\sigma$ = %.3f'%std_width
    textstr42 = textstr4+'\n'+'$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.90, textstr12, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.90, textstr22, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.90, textstr32, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.90, textstr42, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Voigt_Fit_Overplot_HE.pdf')
    plt.show()

#If there one mode of observation.   
else:
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Antaress
    #Getting the scatter
    avg_depth = np.average(theta_Si_Antaress[:, depth_index], weights=1/err_theta_Si_Antaress[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Si_Antaress[:, depth_index] - avg_depth)**2, weights=1/err_theta_Si_Antaress[:, depth_index]**2))
    new_pos = (theta_Si_Antaress[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_theta_Si_Antaress[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Si_Antaress[:, width_index], weights=1/err_theta_Si_Antaress[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Si_Antaress[:, width_index] - avg_width)**2, weights=1/err_theta_Si_Antaress[:, width_index]**2))
    avg_ew = np.average(theta_Si_Antaress[:, ew_index], weights=1/err_theta_Si_Antaress[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Si_Antaress[:, ew_index] - avg_ew)**2, weights=1/err_theta_Si_Antaress[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date, date, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date, theta_Si_Antaress[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date, theta_Si_Antaress[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date, theta_Si_Antaress[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date, theta_Si_Antaress[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date, theta_Si_Antaress[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date, theta_Si_Antaress[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date, theta_Si_Antaress[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date, theta_Si_Antaress[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date-ref_time, theta_Si_Antaress[:, depth_index], yerr=err_theta_Si_Antaress[:, depth_index], fmt='r.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10, label='Antaress')
  
    ax2.errorbar(date-ref_time, (theta_Si_Antaress[:, position_index]-ref_He)*3e8/ref_He, yerr=err_theta_Si_Antaress[:, position_index]*3e8/ref_He, fmt='r.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='r.', markersize=10)
    
    ax3.errorbar(date-ref_time, theta_Si_Antaress[:, width_index], yerr=err_theta_Si_Antaress[:, width_index], fmt='r.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date-ref_time, theta_Si_Antaress[:, ew_index], yerr=err_theta_Si_Antaress[:,ew_index], fmt='r.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
        
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    
    #Rassine
    #Getting the scatter
    avg_depth = np.average(theta_Si_rassine[:, depth_index], weights=1/err_theta_Si_rassine[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Si_rassine[:, depth_index] - avg_depth)**2, weights=1/err_theta_Si_rassine[:, depth_index]**2))
    new_pos = (theta_Si_rassine[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_theta_Si_rassine[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Si_rassine[:, width_index], weights=1/err_theta_Si_rassine[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Si_rassine[:, width_index] - avg_width)**2, weights=1/err_theta_Si_rassine[:, width_index]**2))
    avg_ew = np.average(theta_Si_rassine[:, ew_index], weights=1/err_theta_Si_rassine[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Si_rassine[:, ew_index] - avg_ew)**2, weights=1/err_theta_Si_rassine[:, ew_index]**2))

    #Getting the binned points
    num_bins = 20
        
    binned_depth = ss.binned_statistic(date, theta_Si_rassine[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date, theta_Si_rassine[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date, theta_Si_rassine[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date, theta_Si_rassine[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date, theta_Si_rassine[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date, theta_Si_rassine[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date, theta_Si_rassine[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date, theta_Si_rassine[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date-ref_time, theta_Si_rassine[:, depth_index], yerr=err_theta_Si_rassine[:, depth_index], fmt='b.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='b.', markersize=10, label='Rassine')
  
    ax2.errorbar(date-ref_time, (theta_Si_rassine[:, position_index]-ref_He)*3e8/ref_He, yerr=err_theta_Si_rassine[:, position_index]*3e8/ref_He, fmt='b.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='b.', markersize=10)
    
    ax3.errorbar(date-ref_time, theta_Si_rassine[:, width_index], yerr=err_theta_Si_rassine[:, width_index], fmt='b.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='b.', markersize=10)
    
    ax4.errorbar(date-ref_time, theta_Si_rassine[:, ew_index], yerr=err_theta_Si_rassine[:,ew_index], fmt='b.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='b.', markersize=10)

    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)

    textstr12 = textstr1+'\n'+'$\sigma$ = %.3f'%std_depth
    textstr22 = textstr2+'\n'+'$\sigma$ = %.3f'%std_pos
    textstr32 = textstr3+'\n'+'$\sigma$ = %.3f'%std_width
    textstr42 = textstr4+'\n'+'$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.90, textstr12, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.90, textstr22, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.90, textstr32, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.90, textstr42, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    ax1.legend(fontsize=14)
    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Voigt_Fit_Overplot.pdf')
    plt.show()

## Plotting the positions, depths and widths of the fitted Gaussian profile for Si

### Plotting the positions, depths and widths of the fitted Gaussian profile for Si - Rassine data + Basic Interpolation

In [ ]:
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
save = False

ref_Si = AF.air2vac(10827.091) #From NIST database

ref_time = 59960

depth_index = 0
position_index = 1
width_index = 2
ew_index = -1

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    #HA
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Getting the scatter
    avg_depth = np.average(thetaG_Si_rassine_HA[:, depth_index], weights=1/err_thetaG_Si_rassine_HA[:, depth_index]**2)
    std_depth = np.sqrt(np.average((thetaG_Si_rassine_HA[:, depth_index] - avg_depth)**2, weights=1/err_thetaG_Si_rassine_HA[:, depth_index]**2))
    new_pos = (thetaG_Si_rassine_HA[:, position_index]-ref_Si)*3e8/ref_Si
    new_pos_err = err_thetaG_Si_rassine_HA[:, position_index]*3e8/ref_Si
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(thetaG_Si_rassine_HA[:, width_index], weights=1/err_thetaG_Si_rassine_HA[:, width_index]**2)
    std_width = np.sqrt(np.average((thetaG_Si_rassine_HA[:, width_index] - avg_width)**2, weights=1/err_thetaG_Si_rassine_HA[:, width_index]**2))
    avg_ew = np.average(thetaG_Si_rassine_HA[:, ew_index], weights=1/err_thetaG_Si_rassine_HA[:, ew_index]**2)
    std_ew = np.sqrt(np.average((thetaG_Si_rassine_HA[:, ew_index] - avg_ew)**2, weights=1/err_thetaG_Si_rassine_HA[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HA, date_HA, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HA-ref_time, thetaG_Si_rassine_HA[:, depth_index], yerr=err_thetaG_Si_rassine_HA[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_HA-ref_time, (thetaG_Si_rassine_HA[:, position_index]-ref_Si)*3e8/ref_Si, yerr=err_thetaG_Si_rassine_HA[:, position_index]*3e8/ref_Si, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_Si)*3e8/ref_Si, yerr=binned_pos_err*3e8/ref_Si, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HA-ref_time, thetaG_Si_rassine_HA[:, width_index], yerr=err_thetaG_Si_rassine_HA[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HA-ref_time, thetaG_Si_rassine_HA[:, ew_index], yerr=err_thetaG_Si_rassine_HA[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Gaussian_Fit_Rassine_HA.pdf')
    plt.show()


    #HE
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Getting the scatter
    avg_depth = np.average(thetaG_Si_rassine_HE[:, depth_index], weights=1/err_thetaG_Si_rassine_HE[:, depth_index]**2)
    std_depth = np.sqrt(np.average((thetaG_Si_rassine_HE[:, depth_index] - avg_depth)**2, weights=1/err_thetaG_Si_rassine_HE[:, depth_index]**2))
    new_pos = (thetaG_Si_rassine_HE[:, position_index]-ref_Si)*3e8/ref_Si
    new_pos_err = err_thetaG_Si_rassine_HE[:, position_index]*3e8/ref_Si
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(thetaG_Si_rassine_HE[:, width_index], weights=1/err_thetaG_Si_rassine_HE[:, width_index]**2)
    std_width = np.sqrt(np.average((thetaG_Si_rassine_HE[:, width_index] - avg_width)**2, weights=1/err_thetaG_Si_rassine_HE[:, width_index]**2))
    avg_ew = np.average(thetaG_Si_rassine_HE[:, ew_index], weights=1/err_thetaG_Si_rassine_HE[:, ew_index]**2)
    std_ew = np.sqrt(np.average((thetaG_Si_rassine_HE[:, ew_index] - avg_ew)**2, weights=1/err_thetaG_Si_rassine_HE[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HE, date_HE, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HE-ref_time, thetaG_Si_rassine_HE[:, depth_index], yerr=err_thetaG_Si_rassine_HE[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_HE-ref_time, (thetaG_Si_rassine_HE[:, position_index]-ref_Si)*3e8/ref_Si, yerr=err_thetaG_Si_rassine_HE[:, position_index]*3e8/ref_Si, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_Si)*3e8/ref_Si, yerr=binned_pos_err*3e8/ref_Si, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HE-ref_time, thetaG_Si_rassine_HE[:, width_index], yerr=err_thetaG_Si_rassine_HE[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HE-ref_time, thetaG_Si_rassine_HE[:, ew_index], yerr=err_thetaG_Si_rassine_HE[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Gaussian_Fit_Rassine_HE.pdf')
    plt.show()

#If there one mode of observation.   
else:
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Getting the scatter
    avg_depth = np.average(thetaG_Si_rassine[:, depth_index], weights=1/err_thetaG_Si_rassine[:, depth_index]**2)
    std_depth = np.sqrt(np.average((thetaG_Si_rassine[:, depth_index] - avg_depth)**2, weights=1/err_thetaG_Si_rassine[:, depth_index]**2))
    new_pos = (thetaG_Si_rassine[:, position_index]-ref_Si)*3e8/ref_Si
    new_pos_err = err_thetaG_Si_rassine[:, position_index]*3e8/ref_Si
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(thetaG_Si_rassine[:, width_index], weights=1/err_thetaG_Si_rassine[:, width_index]**2)
    std_width = np.sqrt(np.average((thetaG_Si_rassine[:, width_index] - avg_width)**2, weights=1/err_thetaG_Si_rassine[:, width_index]**2))
    avg_ew = np.average(thetaG_Si_rassine[:, ew_index], weights=1/err_thetaG_Si_rassine[:, ew_index]**2)
    std_ew = np.sqrt(np.average((thetaG_Si_rassine[:, ew_index] - avg_ew)**2, weights=1/err_thetaG_Si_rassine[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date, date, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date, thetaG_Si_rassine[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date, thetaG_Si_rassine[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date, thetaG_Si_rassine[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date, thetaG_Si_rassine[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date, thetaG_Si_rassine[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date, thetaG_Si_rassine[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date, thetaG_Si_rassine[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date, thetaG_Si_rassine[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date-ref_time, thetaG_Si_rassine[:, depth_index], yerr=err_thetaG_Si_rassine[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date-ref_time, (thetaG_Si_rassine[:, position_index]-ref_Si)*3e8/ref_Si, yerr=err_thetaG_Si_rassine[:, position_index]*3e8/ref_Si, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_Si)*3e8/ref_Si, yerr=binned_pos_err*3e8/ref_Si, fmt='r.', markersize=10)
    
    ax3.errorbar(date-ref_time, thetaG_Si_rassine[:, width_index], yerr=err_thetaG_Si_rassine[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date-ref_time, thetaG_Si_rassine[:, ew_index], yerr=err_thetaG_Si_rassine[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Gaussian_Fit_Rassine.pdf')
    plt.show()

### Plotting the positions, depths and widths of the fitted Gaussian profile for Si - Normal data + Antaress


In [ ]:
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
save = False

ref_Si = AF.air2vac(10827.091) #From NIST database

ref_time = 59960

depth_index = 0
position_index = 1
width_index = 2
ew_index = -1

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    #HA
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Getting the scatter
    avg_depth = np.average(thetaG_Si_Antaress_HA[:, depth_index], weights=1/err_thetaG_Si_Antaress_HA[:, depth_index]**2)
    std_depth = np.sqrt(np.average((thetaG_Si_Antaress_HA[:, depth_index] - avg_depth)**2, weights=1/err_thetaG_Si_Antaress_HA[:, depth_index]**2))
    new_pos = (thetaG_Si_Antaress_HA[:, position_index]-ref_Si)*3e8/ref_Si
    new_pos_err = err_thetaG_Si_Antaress_HA[:, position_index]*3e8/ref_Si
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(thetaG_Si_Antaress_HA[:, width_index], weights=1/err_thetaG_Si_Antaress_HA[:, width_index]**2)
    std_width = np.sqrt(np.average((thetaG_Si_Antaress_HA[:, width_index] - avg_width)**2, weights=1/err_thetaG_Si_Antaress_HA[:, width_index]**2))
    avg_ew = np.average(thetaG_Si_Antaress_HA[:, ew_index], weights=1/err_thetaG_Si_Antaress_HA[:, ew_index]**2)
    std_ew = np.sqrt(np.average((thetaG_Si_Antaress_HA[:, ew_index] - avg_ew)**2, weights=1/err_thetaG_Si_Antaress_HA[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HA, date_HA, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HA-ref_time, thetaG_Si_Antaress_HA[:, depth_index], yerr=err_thetaG_Si_Antaress_HA[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_HA-ref_time, (thetaG_Si_Antaress_HA[:, position_index]-ref_Si)*3e8/ref_Si, yerr=err_thetaG_Si_Antaress_HA[:, position_index]*3e8/ref_Si, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_Si)*3e8/ref_Si, yerr=binned_pos_err*3e8/ref_Si, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HA-ref_time, thetaG_Si_Antaress_HA[:, width_index], yerr=err_thetaG_Si_Antaress_HA[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HA-ref_time, thetaG_Si_Antaress_HA[:, ew_index], yerr=err_thetaG_Si_Antaress_HA[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Gaussian_Fit_Antaress_HA.pdf')
    plt.show()


    #HE
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Getting the scatter
    avg_depth = np.average(thetaG_Si_Antaress_HE[:, depth_index], weights=1/err_thetaG_Si_Antaress_HE[:, depth_index]**2)
    std_depth = np.sqrt(np.average((thetaG_Si_Antaress_HE[:, depth_index] - avg_depth)**2, weights=1/err_thetaG_Si_Antaress_HE[:, depth_index]**2))
    new_pos = (thetaG_Si_Antaress_HE[:, position_index]-ref_Si)*3e8/ref_Si
    new_pos_err = err_thetaG_Si_Antaress_HE[:, position_index]*3e8/ref_Si
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(thetaG_Si_Antaress_HE[:, width_index], weights=1/err_thetaG_Si_Antaress_HE[:, width_index]**2)
    std_width = np.sqrt(np.average((thetaG_Si_Antaress_HE[:, width_index] - avg_width)**2, weights=1/err_thetaG_Si_Antaress_HE[:, width_index]**2))
    avg_ew = np.average(thetaG_Si_Antaress_HE[:, ew_index], weights=1/err_thetaG_Si_Antaress_HE[:, ew_index]**2)
    std_ew = np.sqrt(np.average((thetaG_Si_Antaress_HE[:, ew_index] - avg_ew)**2, weights=1/err_thetaG_Si_Antaress_HE[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HE, date_HE, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HE-ref_time, thetaG_Si_Antaress_HE[:, depth_index], yerr=err_thetaG_Si_Antaress_HE[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_HE-ref_time, (thetaG_Si_Antaress_HE[:, position_index]-ref_Si)*3e8/ref_Si, yerr=err_thetaG_Si_Antaress_HE[:, position_index]*3e8/ref_Si, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_Si)*3e8/ref_Si, yerr=binned_pos_err*3e8/ref_Si, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HE-ref_time, thetaG_Si_Antaress_HE[:, width_index], yerr=err_thetaG_Si_Antaress_HE[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HE-ref_time, thetaG_Si_Antaress_HE[:, ew_index], yerr=err_thetaG_Si_Antaress_HE[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Gaussian_Fit_Antaress_HE.pdf')
    plt.show()

#If there one mode of observation.   
else:
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Getting the scatter
    avg_depth = np.average(thetaG_Si_Antaress[:, depth_index], weights=1/err_thetaG_Si_Antaress[:, depth_index]**2)
    std_depth = np.sqrt(np.average((thetaG_Si_Antaress[:, depth_index] - avg_depth)**2, weights=1/err_thetaG_Si_Antaress[:, depth_index]**2))
    new_pos = (thetaG_Si_Antaress[:, position_index]-ref_Si)*3e8/ref_Si
    new_pos_err = err_thetaG_Si_Antaress[:, position_index]*3e8/ref_Si
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(thetaG_Si_Antaress[:, width_index], weights=1/err_thetaG_Si_Antaress[:, width_index]**2)
    std_width = np.sqrt(np.average((thetaG_Si_Antaress[:, width_index] - avg_width)**2, weights=1/err_thetaG_Si_Antaress[:, width_index]**2))
    avg_ew = np.average(thetaG_Si_Antaress[:, ew_index], weights=1/err_thetaG_Si_Antaress[:, ew_index]**2)
    std_ew = np.sqrt(np.average((thetaG_Si_Antaress[:, ew_index] - avg_ew)**2, weights=1/err_thetaG_Si_Antaress[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date, date, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date, thetaG_Si_Antaress[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date, thetaG_Si_Antaress[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date, thetaG_Si_Antaress[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date, thetaG_Si_Antaress[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date, thetaG_Si_Antaress[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date, thetaG_Si_Antaress[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date, thetaG_Si_Antaress[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date, thetaG_Si_Antaress[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date-ref_time, thetaG_Si_Antaress[:, depth_index], yerr=err_thetaG_Si_Antaress[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date-ref_time, (thetaG_Si_Antaress[:, position_index]-ref_Si)*3e8/ref_Si, yerr=err_thetaG_Si_Antaress[:, position_index]*3e8/ref_Si, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_Si)*3e8/ref_Si, yerr=binned_pos_err*3e8/ref_Si, fmt='r.', markersize=10)
    
    ax3.errorbar(date-ref_time, thetaG_Si_Antaress[:, width_index], yerr=err_thetaG_Si_Antaress[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date-ref_time, thetaG_Si_Antaress[:, ew_index], yerr=err_thetaG_Si_Antaress[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Gaussian_Fit_Antaress.pdf')
    plt.show()

### Plotting the positions, depths and widths of the fitted Gaussian profile for Si - Overplot (Rassine + Antaress)


In [ ]:
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
save = False

ref_He = AF.air2vac((10830.33977+10830.25010)/2) #From NIST database

ref_time = 59960

depth_index = 0
position_index = 1
width_index = 2
ew_index = -1

opacity = 0.1

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    #HA
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Antaress
    #Getting the scatter
    avg_depth = np.average(thetaG_Si_Antaress_HA[:, depth_index], weights=1/err_thetaG_Si_Antaress_HA[:, depth_index]**2)
    std_depth = np.sqrt(np.average((thetaG_Si_Antaress_HA[:, depth_index] - avg_depth)**2, weights=1/err_thetaG_Si_Antaress_HA[:, depth_index]**2))
    new_pos = (thetaG_Si_Antaress_HA[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_thetaG_Si_Antaress_HA[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(thetaG_Si_Antaress_HA[:, width_index], weights=1/err_thetaG_Si_Antaress_HA[:, width_index]**2)
    std_width = np.sqrt(np.average((thetaG_Si_Antaress_HA[:, width_index] - avg_width)**2, weights=1/err_thetaG_Si_Antaress_HA[:, width_index]**2))
    avg_ew = np.average(thetaG_Si_Antaress_HA[:, ew_index], weights=1/err_thetaG_Si_Antaress_HA[:, ew_index]**2)
    std_ew = np.sqrt(np.average((thetaG_Si_Antaress_HA[:, ew_index] - avg_ew)**2, weights=1/err_thetaG_Si_Antaress_HA[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HA, date_HA, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HA, thetaG_Si_Antaress_HA[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HA-ref_time, thetaG_Si_Antaress_HA[:, depth_index], yerr=err_thetaG_Si_Antaress_HA[:, depth_index], fmt='r.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
  
    ax2.errorbar(date_HA-ref_time, (thetaG_Si_Antaress_HA[:, position_index]-ref_He)*3e8/ref_He, yerr=err_thetaG_Si_Antaress_HA[:, position_index]*3e8/ref_He, fmt='r.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HA-ref_time, thetaG_Si_Antaress_HA[:, width_index], yerr=err_thetaG_Si_Antaress_HA[:, width_index], fmt='r.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HA-ref_time, thetaG_Si_Antaress_HA[:, ew_index], yerr=err_thetaG_Si_Antaress_HA[:,ew_index], fmt='r.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
        
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    
    #Rassine
    #Getting the scatter
    avg_depth = np.average(thetaG_Si_rassine_HA[:, depth_index], weights=1/err_thetaG_Si_rassine_HA[:, depth_index]**2)
    std_depth = np.sqrt(np.average((thetaG_Si_rassine_HA[:, depth_index] - avg_depth)**2, weights=1/err_thetaG_Si_rassine_HA[:, depth_index]**2))
    new_pos = (thetaG_Si_rassine_HA[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_thetaG_Si_rassine_HA[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(thetaG_Si_rassine_HA[:, width_index], weights=1/err_thetaG_Si_rassine_HA[:, width_index]**2)
    std_width = np.sqrt(np.average((thetaG_Si_rassine_HA[:, width_index] - avg_width)**2, weights=1/err_thetaG_Si_rassine_HA[:, width_index]**2))
    avg_ew = np.average(thetaG_Si_rassine_HA[:, ew_index], weights=1/err_thetaG_Si_rassine_HA[:, ew_index]**2)
    std_ew = np.sqrt(np.average((thetaG_Si_rassine_HA[:, ew_index] - avg_ew)**2, weights=1/err_thetaG_Si_rassine_HA[:, ew_index]**2))

    #Getting the binned points
    num_bins = 20
        
    binned_depth = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HA, thetaG_Si_rassine_HA[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HA-ref_time, thetaG_Si_rassine_HA[:, depth_index], yerr=err_thetaG_Si_rassine_HA[:, depth_index], fmt='b.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='b.', markersize=10)
  
    ax2.errorbar(date_HA-ref_time, (thetaG_Si_rassine_HA[:, position_index]-ref_He)*3e8/ref_He, yerr=err_thetaG_Si_rassine_HA[:, position_index]*3e8/ref_He, fmt='b.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='b.', markersize=10)
    
    ax3.errorbar(date_HA-ref_time, thetaG_Si_rassine_HA[:, width_index], yerr=err_thetaG_Si_rassine_HA[:, width_index], fmt='b.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='b.', markersize=10)
    
    ax4.errorbar(date_HA-ref_time, thetaG_Si_rassine_HA[:, ew_index], yerr=err_thetaG_Si_rassine_HA[:,ew_index], fmt='b.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='b.', markersize=10)

    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)

    textstr12 = textstr1+'\n'+'$\sigma$ = %.3f'%std_depth
    textstr22 = textstr2+'\n'+'$\sigma$ = %.3f'%std_pos
    textstr32 = textstr3+'\n'+'$\sigma$ = %.3f'%std_width
    textstr42 = textstr4+'\n'+'$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.90, textstr12, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.90, textstr22, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.90, textstr32, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.90, textstr42, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Gaussian_Fit_Overplot_HA.pdf')
    plt.show()


    #HE
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Antaress
    #Getting the scatter
    avg_depth = np.average(thetaG_Si_Antaress_HE[:, depth_index], weights=1/err_thetaG_Si_Antaress_HE[:, depth_index]**2)
    std_depth = np.sqrt(np.average((thetaG_Si_Antaress_HE[:, depth_index] - avg_depth)**2, weights=1/err_thetaG_Si_Antaress_HE[:, depth_index]**2))
    new_pos = (thetaG_Si_Antaress_HE[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_thetaG_Si_Antaress_HE[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(thetaG_Si_Antaress_HE[:, width_index], weights=1/err_thetaG_Si_Antaress_HE[:, width_index]**2)
    std_width = np.sqrt(np.average((thetaG_Si_Antaress_HE[:, width_index] - avg_width)**2, weights=1/err_thetaG_Si_Antaress_HE[:, width_index]**2))
    avg_ew = np.average(thetaG_Si_Antaress_HE[:, ew_index], weights=1/err_thetaG_Si_Antaress_HE[:, ew_index]**2)
    std_ew = np.sqrt(np.average((thetaG_Si_Antaress_HE[:, ew_index] - avg_ew)**2, weights=1/err_thetaG_Si_Antaress_HE[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HE, date_HE, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HE, thetaG_Si_Antaress_HE[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HE-ref_time, thetaG_Si_Antaress_HE[:, depth_index], yerr=err_thetaG_Si_Antaress_HE[:, depth_index], fmt='r.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
  
    ax2.errorbar(date_HE-ref_time, (thetaG_Si_Antaress_HE[:, position_index]-ref_He)*3e8/ref_He, yerr=err_thetaG_Si_Antaress_HE[:, position_index]*3e8/ref_He, fmt='r.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HE-ref_time, thetaG_Si_Antaress_HE[:, width_index], yerr=err_thetaG_Si_Antaress_HE[:, width_index], fmt='r.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HE-ref_time, thetaG_Si_Antaress_HE[:, ew_index], yerr=err_thetaG_Si_Antaress_HE[:,ew_index], fmt='r.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
        
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    
    #Rassine
    #Getting the scatter
    avg_depth = np.average(thetaG_Si_rassine_HE[:, depth_index], weights=1/err_thetaG_Si_rassine_HE[:, depth_index]**2)
    std_depth = np.sqrt(np.average((thetaG_Si_rassine_HE[:, depth_index] - avg_depth)**2, weights=1/err_thetaG_Si_rassine_HE[:, depth_index]**2))
    new_pos = (thetaG_Si_rassine_HE[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_thetaG_Si_rassine_HE[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(thetaG_Si_rassine_HE[:, width_index], weights=1/err_thetaG_Si_rassine_HE[:, width_index]**2)
    std_width = np.sqrt(np.average((thetaG_Si_rassine_HE[:, width_index] - avg_width)**2, weights=1/err_thetaG_Si_rassine_HE[:, width_index]**2))
    avg_ew = np.average(thetaG_Si_rassine_HE[:, ew_index], weights=1/err_thetaG_Si_rassine_HE[:, ew_index]**2)
    std_ew = np.sqrt(np.average((thetaG_Si_rassine_HE[:, ew_index] - avg_ew)**2, weights=1/err_thetaG_Si_rassine_HE[:, ew_index]**2))

    #Getting the binned points
    num_bins = 20
        
    binned_depth = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HE, thetaG_Si_rassine_HE[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HE-ref_time, thetaG_Si_rassine_HE[:, depth_index], yerr=err_thetaG_Si_rassine_HE[:, depth_index], fmt='b.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='b.', markersize=10)
  
    ax2.errorbar(date_HE-ref_time, (thetaG_Si_rassine_HE[:, position_index]-ref_He)*3e8/ref_He, yerr=err_thetaG_Si_rassine_HE[:, position_index]*3e8/ref_He, fmt='b.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='b.', markersize=10)
    
    ax3.errorbar(date_HE-ref_time, thetaG_Si_rassine_HE[:, width_index], yerr=err_thetaG_Si_rassine_HE[:, width_index], fmt='b.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='b.', markersize=10)
    
    ax4.errorbar(date_HE-ref_time, thetaG_Si_rassine_HE[:, ew_index], yerr=err_thetaG_Si_rassine_HE[:,ew_index], fmt='b.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='b.', markersize=10)

    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)

    textstr12 = textstr1+'\n'+'$\sigma$ = %.3f'%std_depth
    textstr22 = textstr2+'\n'+'$\sigma$ = %.3f'%std_pos
    textstr32 = textstr3+'\n'+'$\sigma$ = %.3f'%std_width
    textstr42 = textstr4+'\n'+'$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.90, textstr12, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.90, textstr22, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.90, textstr32, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.90, textstr42, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Gaussian_Fit_Overplot_HE.pdf')
    plt.show()

#If there one mode of observation.   
else:
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Antaress
    #Getting the scatter
    avg_depth = np.average(thetaG_Si_Antaress[:, depth_index], weights=1/err_thetaG_Si_Antaress[:, depth_index]**2)
    std_depth = np.sqrt(np.average((thetaG_Si_Antaress[:, depth_index] - avg_depth)**2, weights=1/err_thetaG_Si_Antaress[:, depth_index]**2))
    new_pos = (thetaG_Si_Antaress[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_thetaG_Si_Antaress[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(thetaG_Si_Antaress[:, width_index], weights=1/err_thetaG_Si_Antaress[:, width_index]**2)
    std_width = np.sqrt(np.average((thetaG_Si_Antaress[:, width_index] - avg_width)**2, weights=1/err_thetaG_Si_Antaress[:, width_index]**2))
    avg_ew = np.average(thetaG_Si_Antaress[:, ew_index], weights=1/err_thetaG_Si_Antaress[:, ew_index]**2)
    std_ew = np.sqrt(np.average((thetaG_Si_Antaress[:, ew_index] - avg_ew)**2, weights=1/err_thetaG_Si_Antaress[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date, date, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date, thetaG_Si_Antaress[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date, thetaG_Si_Antaress[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date, thetaG_Si_Antaress[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date, thetaG_Si_Antaress[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date, thetaG_Si_Antaress[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date, thetaG_Si_Antaress[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date, thetaG_Si_Antaress[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date, thetaG_Si_Antaress[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date-ref_time, thetaG_Si_Antaress[:, depth_index], yerr=err_thetaG_Si_Antaress[:, depth_index], fmt='r.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10, label='Antaress')
  
    ax2.errorbar(date-ref_time, (thetaG_Si_Antaress[:, position_index]-ref_He)*3e8/ref_He, yerr=err_thetaG_Si_Antaress[:, position_index]*3e8/ref_He, fmt='r.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='r.', markersize=10)
    
    ax3.errorbar(date-ref_time, thetaG_Si_Antaress[:, width_index], yerr=err_thetaG_Si_Antaress[:, width_index], fmt='r.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date-ref_time, thetaG_Si_Antaress[:, ew_index], yerr=err_thetaG_Si_Antaress[:,ew_index], fmt='r.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
        
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    
    #Rassine
    #Getting the scatter
    avg_depth = np.average(thetaG_Si_rassine[:, depth_index], weights=1/err_thetaG_Si_rassine[:, depth_index]**2)
    std_depth = np.sqrt(np.average((thetaG_Si_rassine[:, depth_index] - avg_depth)**2, weights=1/err_thetaG_Si_rassine[:, depth_index]**2))
    new_pos = (thetaG_Si_rassine[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_thetaG_Si_rassine[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(thetaG_Si_rassine[:, width_index], weights=1/err_thetaG_Si_rassine[:, width_index]**2)
    std_width = np.sqrt(np.average((thetaG_Si_rassine[:, width_index] - avg_width)**2, weights=1/err_thetaG_Si_rassine[:, width_index]**2))
    avg_ew = np.average(thetaG_Si_rassine[:, ew_index], weights=1/err_thetaG_Si_rassine[:, ew_index]**2)
    std_ew = np.sqrt(np.average((thetaG_Si_rassine[:, ew_index] - avg_ew)**2, weights=1/err_thetaG_Si_rassine[:, ew_index]**2))

    #Getting the binned points
    num_bins = 20
        
    binned_depth = ss.binned_statistic(date, thetaG_Si_rassine[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date, thetaG_Si_rassine[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date, thetaG_Si_rassine[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date, thetaG_Si_rassine[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date, thetaG_Si_rassine[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date, thetaG_Si_rassine[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date, thetaG_Si_rassine[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date, thetaG_Si_rassine[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date-ref_time, thetaG_Si_rassine[:, depth_index], yerr=err_thetaG_Si_rassine[:, depth_index], fmt='b.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='b.', markersize=10, label='Rassine')
  
    ax2.errorbar(date-ref_time, (thetaG_Si_rassine[:, position_index]-ref_He)*3e8/ref_He, yerr=err_thetaG_Si_rassine[:, position_index]*3e8/ref_He, fmt='b.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='b.', markersize=10)
    
    ax3.errorbar(date-ref_time, thetaG_Si_rassine[:, width_index], yerr=err_thetaG_Si_rassine[:, width_index], fmt='b.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='b.', markersize=10)
    
    ax4.errorbar(date-ref_time, thetaG_Si_rassine[:, ew_index], yerr=err_thetaG_Si_rassine[:,ew_index], fmt='b.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='b.', markersize=10)

    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)

    textstr12 = textstr1+'\n'+'$\sigma$ = %.3f'%std_depth
    textstr22 = textstr2+'\n'+'$\sigma$ = %.3f'%std_pos
    textstr32 = textstr3+'\n'+'$\sigma$ = %.3f'%std_width
    textstr42 = textstr4+'\n'+'$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.90, textstr12, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.90, textstr22, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.90, textstr32, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.90, textstr42, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    ax1.legend(fontsize=14)
    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Si_Gaussian_Fit_Overplot.pdf')
    plt.show()

## Plotting the He triplet for all the spectra of the day

### Plotting the test

In [ ]:
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
save = False

ref_He = AF.air2vac((10830.33977+10830.25010)/2) #From NIST database

ref_time = 59970

depth_index = 1 #actually temperature
position_index = 0 #actually log density
width_index = 2 #actually the offset
ew_index = -1 #actually the ew

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    #HA
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    theta_in_question = theta_HA
    err_in_question = err_theta_HA
    date_in_question = date_HA
    ref_in_question = ref_He
    #Getting the scatter
    avg_depth = np.average(theta_in_question[:, depth_index], weights=1/err_in_question[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_in_question[:, depth_index] - avg_depth)**2, weights=1/err_in_question[:, depth_index]**2))
    new_pos = (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question
    new_pos_err = err_in_question[:, position_index]*3e8/ref_in_question
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_in_question[:, width_index], weights=1/err_in_question[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_in_question[:, width_index] - avg_width)**2, weights=1/err_in_question[:, width_index]**2))
    avg_ew = np.average(theta_in_question[:, ew_index], weights=1/err_in_question[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_in_question[:, ew_index] - avg_ew)**2, weights=1/err_in_question[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_in_question, date_in_question, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_in_question-ref_time, theta_in_question[:, depth_index], yerr=err_in_question[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_in_question-ref_time, (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question, yerr=err_in_question[:, position_index]*3e8/ref_in_question, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_in_question)*3e8/ref_in_question, yerr=binned_pos_err*3e8/ref_in_question, fmt='r.', markersize=10)
    
    ax3.errorbar(date_in_question-ref_time, theta_in_question[:, width_index], yerr=err_in_question[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_in_question-ref_time, theta_in_question[:, ew_index], yerr=err_in_question[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/test_fit_HA.pdf')
    plt.show()


    #HE
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    theta_in_question = theta_HE
    err_in_question = err_theta_HE
    date_in_question = date_HE
    ref_in_question = ref_He
    #Getting the scatter
    avg_depth = np.average(theta_in_question[:, depth_index], weights=1/err_in_question[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_in_question[:, depth_index] - avg_depth)**2, weights=1/err_in_question[:, depth_index]**2))
    new_pos = (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question
    new_pos_err = err_in_question[:, position_index]*3e8/ref_in_question
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_in_question[:, width_index], weights=1/err_in_question[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_in_question[:, width_index] - avg_width)**2, weights=1/err_in_question[:, width_index]**2))
    avg_ew = np.average(theta_in_question[:, ew_index], weights=1/err_in_question[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_in_question[:, ew_index] - avg_ew)**2, weights=1/err_in_question[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_in_question, date_in_question, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_in_question-ref_time, theta_in_question[:, depth_index], yerr=err_in_question[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_in_question-ref_time, (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question, yerr=err_in_question[:, position_index]*3e8/ref_in_question, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_in_question)*3e8/ref_in_question, yerr=binned_pos_err*3e8/ref_in_question, fmt='r.', markersize=10)
    
    ax3.errorbar(date_in_question-ref_time, theta_in_question[:, width_index], yerr=err_in_question[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_in_question-ref_time, theta_in_question[:, ew_index], yerr=err_in_question[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/test_fit_HE.pdf')
    plt.show()

#If there one mode of observation.   
else:
    '''
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    theta_in_question = theta
    err_in_question = err_theta
    date_in_question = date
    ref_in_question = ref_He
    #Getting the scatter
    avg_depth = np.average(theta_in_question[:, depth_index], weights=1/err_in_question[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_in_question[:, depth_index] - avg_depth)**2, weights=1/err_in_question[:, depth_index]**2))
    new_pos = theta_in_question[:, position_index]
    new_pos_err = err_in_question[:, position_index]
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_in_question[:, width_index], weights=1/err_in_question[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_in_question[:, width_index] - avg_width)**2, weights=1/err_in_question[:, width_index]**2))
    avg_ew = np.average(theta_in_question[:, ew_index], weights=1/err_in_question[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_in_question[:, ew_index] - avg_ew)**2, weights=1/err_in_question[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_in_question, date_in_question, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_in_question-ref_time, theta_in_question[:, depth_index], yerr=err_in_question[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_in_question-ref_time, theta_in_question[:, position_index], yerr=err_in_question[:, position_index], fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, binned_pos, yerr=binned_pos_err, fmt='r.', markersize=10)
    
    ax3.errorbar(date_in_question-ref_time, theta_in_question[:, width_index], yerr=err_in_question[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_in_question-ref_time, theta_in_question[:, ew_index], yerr=err_in_question[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax1.plot(date_in_question-ref_time, theta_in_question[:, depth_index], '.', alpha=0.2)
    #ax1.plot(binned_date-ref_time, binned_depth-15, 'r.', markersize=10)
    #ax1.plot(binned_date-ref_time, binned_depth, 'r.', markersize=10)
    
    ax2.plot(date_in_question-ref_time, theta_in_question[:, position_index], '.', alpha=0.2)
    #ax2.plot(binned_date-ref_time, binned_pos-20000, 'r.', markersize=10)
    ax2.plot(binned_date-ref_time, binned_pos, 'r.', markersize=10)
    
    ax3.plot(date_in_question-ref_time, theta_in_question[:, width_index], '.', alpha=0.2)
    ax3.plot(binned_date-ref_time, binned_width, 'r.', markersize=10)
    
    ax4.plot(date_in_question-ref_time, theta_in_question[:, ew_index], '.', alpha=0.2)
    ax4.plot(binned_date-ref_time, binned_ew, 'r.', markersize=10)

    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Density ($m^{-3}$)', fontsize=14)
    ax2.set_ylabel('Temperature (K)', fontsize=14)
    ax3.set_ylabel('Offset (flux)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/test_fit.pdf')
    plt.show()
    '''

    T_He_index = 0 #actually temperature of He
    T_Si_index = 1 #actually temperature of Si
    n_He_index = 2 #actually density of He
    n_Si_index = 3 #actually density of Si
    d_Si_index = 4 # actually damping parameter of Si
    ew_index = -1 #actually the ew

    fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots (3, 2, sharex=True, figsize=[12, 10])

    theta_in_question = lmfit_thetas
    err_in_question = np.ones(lmfit_thetas.shape)*0.00001
    date_in_question = date
    
    #Getting the scatter
    avg_T_He = np.average(theta_in_question[:, T_He_index])
    std_T_He = np.sqrt(np.average((theta_in_question[:, T_He_index] - avg_T_He)**2))
    avg_T_Si = np.average(theta_in_question[:, T_Si_index])
    std_T_Si = np.sqrt(np.average((theta_in_question[:, T_Si_index] - avg_T_Si)**2))
    avg_n_He = np.average(theta_in_question[:, n_He_index])
    std_n_He = np.sqrt(np.average((theta_in_question[:, n_He_index] - avg_n_He)**2))
    avg_n_Si = np.average(theta_in_question[:, n_Si_index])
    std_n_Si = np.sqrt(np.average((theta_in_question[:, n_Si_index] - avg_n_Si)**2))
    avg_d_Si = np.average(theta_in_question[:, d_Si_index])
    std_d_Si = np.sqrt(np.average((theta_in_question[:, d_Si_index] - avg_d_Si)**2))   
    avg_ew = np.average(theta_in_question[:, ew_index])
    std_ew = np.sqrt(np.average((theta_in_question[:, ew_index] - avg_ew)**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_in_question, date_in_question, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_T_He = ss.binned_statistic(date_in_question, theta_in_question[:, T_He_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_n_He = ss.binned_statistic(date_in_question, theta_in_question[:, n_He_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_T_Si = ss.binned_statistic(date_in_question, theta_in_question[:, T_Si_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_n_Si = ss.binned_statistic(date_in_question, theta_in_question[:, n_Si_index] , statistic=np.ma.median, bins=num_bins)[0]
    binned_d_Si = ss.binned_statistic(date_in_question, theta_in_question[:, d_Si_index] , statistic=np.ma.median, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]

    ax1.plot(date_in_question-ref_time, theta_in_question[:, T_He_index], '.', alpha=0.2)
    ax1.plot(binned_date-ref_time, binned_T_He, 'r.', markersize=10)
    ax1twin = ax1.twinx()
    ax1twin.plot(date_in_question-ref_time, 10**theta_in_question[:, T_He_index], '.', alpha=0)
    
    ax2.plot(date_in_question-ref_time, theta_in_question[:, T_Si_index], '.', alpha=0.2)
    ax2.plot(binned_date-ref_time, binned_T_Si, 'r.', markersize=10)
    ax2twin = ax2.twinx()
    ax2twin.plot(date_in_question-ref_time, 10**theta_in_question[:, T_Si_index], '.', alpha=0)

    ax3.plot(date_in_question-ref_time, theta_in_question[:, n_He_index], '.', alpha=0.2)
    ax3.plot(binned_date-ref_time, binned_n_He, 'r.', markersize=10)
 
    ax4.plot(date_in_question-ref_time, theta_in_question[:, n_Si_index], '.', alpha=0.2)
    ax4.plot(binned_date-ref_time, binned_n_Si, 'r.', markersize=10)

    ax5.plot(date_in_question-ref_time, theta_in_question[:, d_Si_index], '.', alpha=0.2)
    ax5.plot(binned_date-ref_time, binned_d_Si, 'r.', markersize=10)
    
    ax6.plot(date_in_question-ref_time, theta_in_question[:, ew_index], '.', alpha=0.2)
    ax6.plot(binned_date-ref_time, binned_ew, 'r.', markersize=10)
    
    ax5.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax6.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Log Temperature He (K)', fontsize=14)
    ax2.set_ylabel('Log Temperature Si (K)', fontsize=14)
    ax3.set_ylabel('Log Density He ($m^{-3}$)', fontsize=14)
    ax4.set_ylabel('Log Density Si ($m^{-3}$)', fontsize=14)
    ax5.set_ylabel('Damping Si', fontsize=14)
    ax6.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='y', labelsize=14)
    ax4.tick_params(axis='y', labelsize=14)
    ax5.tick_params(axis='both', labelsize=14)
    ax6.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_T_He
    textstr2 = '$\sigma$ = %.3f'%std_T_Si
    textstr3 = '$\sigma$ = %.3f'%std_n_He
    textstr4 = '$\sigma$ = %.3f'%std_n_Si
    textstr5 = '$\sigma$ = %.3f'%std_d_Si
    textstr6 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax5.text(0.67, 0.92, textstr5, transform=ax5.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax6.text(0.67, 0.92, textstr6, transform=ax6.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/test_fit.pdf')
    plt.show()
    

### Plotting the positions, depths and widths of the fitted Gaussian lines - Rassine data + Basic Interpolation


In [ ]:
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
save = False

ref_He = AF.air2vac((10830.33977+10830.25010)/2) #From NIST database

ref_time = 59960

depth_index = 0
position_index = 1
width_index = 2
ew_index = -1

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    #HA
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    theta_in_question = theta_rassine_HA
    err_in_question = err_theta_rassine_HA
    date_in_question = date_HA
    ref_in_question = ref_He
    #Getting the scatter
    avg_depth = np.average(theta_in_question[:, depth_index], weights=1/err_in_question[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_in_question[:, depth_index] - avg_depth)**2, weights=1/err_in_question[:, depth_index]**2))
    new_pos = (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question
    new_pos_err = err_in_question[:, position_index]*3e8/ref_in_question
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_in_question[:, width_index], weights=1/err_in_question[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_in_question[:, width_index] - avg_width)**2, weights=1/err_in_question[:, width_index]**2))
    avg_ew = np.average(theta_in_question[:, ew_index], weights=1/err_in_question[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_in_question[:, ew_index] - avg_ew)**2, weights=1/err_in_question[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_in_question, date_in_question, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_in_question-ref_time, theta_in_question[:, depth_index], yerr=err_in_question[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_in_question-ref_time, (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question, yerr=err_in_question[:, position_index]*3e8/ref_in_question, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_in_question)*3e8/ref_in_question, yerr=binned_pos_err*3e8/ref_in_question, fmt='r.', markersize=10)
    
    ax3.errorbar(date_in_question-ref_time, theta_in_question[:, width_index], yerr=err_in_question[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_in_question-ref_time, theta_in_question[:, ew_index], yerr=err_in_question[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/He_Gaussian_Fit_Rassine_HA.pdf')
    plt.show()


    #HE
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    theta_in_question = theta_rassine_HE
    err_in_question = err_theta_rassine_HE
    date_in_question = date_HE
    ref_in_question = ref_He
    #Getting the scatter
    avg_depth = np.average(theta_in_question[:, depth_index], weights=1/err_in_question[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_in_question[:, depth_index] - avg_depth)**2, weights=1/err_in_question[:, depth_index]**2))
    new_pos = (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question
    new_pos_err = err_in_question[:, position_index]*3e8/ref_in_question
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_in_question[:, width_index], weights=1/err_in_question[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_in_question[:, width_index] - avg_width)**2, weights=1/err_in_question[:, width_index]**2))
    avg_ew = np.average(theta_in_question[:, ew_index], weights=1/err_in_question[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_in_question[:, ew_index] - avg_ew)**2, weights=1/err_in_question[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_in_question, date_in_question, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_in_question-ref_time, theta_in_question[:, depth_index], yerr=err_in_question[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_in_question-ref_time, (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question, yerr=err_in_question[:, position_index]*3e8/ref_in_question, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_in_question)*3e8/ref_in_question, yerr=binned_pos_err*3e8/ref_in_question, fmt='r.', markersize=10)
    
    ax3.errorbar(date_in_question-ref_time, theta_in_question[:, width_index], yerr=err_in_question[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_in_question-ref_time, theta_in_question[:, ew_index], yerr=err_in_question[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/He_Gaussian_Fit_Rassine_HE.pdf')
    plt.show()

#If there one mode of observation.   
else:
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    theta_in_question = theta_rassine
    err_in_question = err_theta_rassine
    date_in_question = date
    ref_in_question = ref_He
    #Getting the scatter
    avg_depth = np.average(theta_in_question[:, depth_index], weights=1/err_in_question[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_in_question[:, depth_index] - avg_depth)**2, weights=1/err_in_question[:, depth_index]**2))
    new_pos = (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question
    new_pos_err = err_in_question[:, position_index]*3e8/ref_in_question
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_in_question[:, width_index], weights=1/err_in_question[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_in_question[:, width_index] - avg_width)**2, weights=1/err_in_question[:, width_index]**2))
    avg_ew = np.average(theta_in_question[:, ew_index], weights=1/err_in_question[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_in_question[:, ew_index] - avg_ew)**2, weights=1/err_in_question[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_in_question, date_in_question, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_in_question-ref_time, theta_in_question[:, depth_index], yerr=err_in_question[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_in_question-ref_time, (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question, yerr=err_in_question[:, position_index]*3e8/ref_in_question, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_in_question)*3e8/ref_in_question, yerr=binned_pos_err*3e8/ref_in_question, fmt='r.', markersize=10)
    
    ax3.errorbar(date_in_question-ref_time, theta_in_question[:, width_index], yerr=err_in_question[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_in_question-ref_time, theta_in_question[:, ew_index], yerr=err_in_question[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/He_Gaussian_Fit_Rassine.pdf')
    plt.show()

### Plotting the positions, depths and widths of the fitted Gaussian lines - Normal data + Antaress

In [ ]:
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
save = False

ref_He = AF.air2vac((10830.33977+10830.25010)/2) #From NIST database

ref_time = 59960

depth_index = 0
position_index = 1
width_index = 2
ew_index = -1

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    #HA
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    theta_in_question = theta_Antaress_HA
    err_in_question = err_theta_Antaress_HA
    date_in_question = date_HA
    ref_in_question = ref_He
    #Getting the scatter
    avg_depth = np.average(theta_in_question[:, depth_index], weights=1/err_in_question[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_in_question[:, depth_index] - avg_depth)**2, weights=1/err_in_question[:, depth_index]**2))
    new_pos = (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question
    new_pos_err = err_in_question[:, position_index]*3e8/ref_in_question
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_in_question[:, width_index], weights=1/err_in_question[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_in_question[:, width_index] - avg_width)**2, weights=1/err_in_question[:, width_index]**2))
    avg_ew = np.average(theta_in_question[:, ew_index], weights=1/err_in_question[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_in_question[:, ew_index] - avg_ew)**2, weights=1/err_in_question[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_in_question, date_in_question, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_in_question-ref_time, theta_in_question[:, depth_index], yerr=err_in_question[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_in_question-ref_time, (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question, yerr=err_in_question[:, position_index]*3e8/ref_in_question, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_in_question)*3e8/ref_in_question, yerr=binned_pos_err*3e8/ref_in_question, fmt='r.', markersize=10)
    
    ax3.errorbar(date_in_question-ref_time, theta_in_question[:, width_index], yerr=err_in_question[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_in_question-ref_time, theta_in_question[:, ew_index], yerr=err_in_question[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/He_Gaussian_Fit_Antaress_HA.pdf')
    plt.show()


    #HE
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    theta_in_question = theta_Antaress_HE
    err_in_question = err_theta_Antaress_HE
    date_in_question = date_HE
    ref_in_question = ref_He
    #Getting the scatter
    avg_depth = np.average(theta_in_question[:, depth_index], weights=1/err_in_question[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_in_question[:, depth_index] - avg_depth)**2, weights=1/err_in_question[:, depth_index]**2))
    new_pos = (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question
    new_pos_err = err_in_question[:, position_index]*3e8/ref_in_question
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_in_question[:, width_index], weights=1/err_in_question[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_in_question[:, width_index] - avg_width)**2, weights=1/err_in_question[:, width_index]**2))
    avg_ew = np.average(theta_in_question[:, ew_index], weights=1/err_in_question[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_in_question[:, ew_index] - avg_ew)**2, weights=1/err_in_question[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_in_question, date_in_question, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_in_question-ref_time, theta_in_question[:, depth_index], yerr=err_in_question[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_in_question-ref_time, (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question, yerr=err_in_question[:, position_index]*3e8/ref_in_question, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_in_question)*3e8/ref_in_question, yerr=binned_pos_err*3e8/ref_in_question, fmt='r.', markersize=10)
    
    ax3.errorbar(date_in_question-ref_time, theta_in_question[:, width_index], yerr=err_in_question[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_in_question-ref_time, theta_in_question[:, ew_index], yerr=err_in_question[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/He_Gaussian_Fit_Antaress_HE.pdf')
    plt.show()

#If there one mode of observation.   
else:
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    theta_in_question = theta_Antaress
    err_in_question = err_theta_Antaress
    date_in_question = date
    ref_in_question = ref_He
    #Getting the scatter
    avg_depth = np.average(theta_in_question[:, depth_index], weights=1/err_in_question[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_in_question[:, depth_index] - avg_depth)**2, weights=1/err_in_question[:, depth_index]**2))
    new_pos = (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question
    new_pos_err = err_in_question[:, position_index]*3e8/ref_in_question
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_in_question[:, width_index], weights=1/err_in_question[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_in_question[:, width_index] - avg_width)**2, weights=1/err_in_question[:, width_index]**2))
    avg_ew = np.average(theta_in_question[:, ew_index], weights=1/err_in_question[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_in_question[:, ew_index] - avg_ew)**2, weights=1/err_in_question[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_in_question, date_in_question, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_in_question, theta_in_question[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_in_question, theta_in_question[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_in_question, theta_in_question[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_in_question, theta_in_question[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_in_question-ref_time, theta_in_question[:, depth_index], yerr=err_in_question[:, depth_index], fmt='.', alpha=0.2)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
    
    ax2.errorbar(date_in_question-ref_time, (theta_in_question[:, position_index]-ref_in_question)*3e8/ref_in_question, yerr=err_in_question[:, position_index]*3e8/ref_in_question, fmt='.', alpha=0.2)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_in_question)*3e8/ref_in_question, yerr=binned_pos_err*3e8/ref_in_question, fmt='r.', markersize=10)
    
    ax3.errorbar(date_in_question-ref_time, theta_in_question[:, width_index], yerr=err_in_question[:, width_index], fmt='.', alpha=0.2)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_in_question-ref_time, theta_in_question[:, ew_index], yerr=err_in_question[:,ew_index], fmt='.', alpha=0.2)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
    
    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.92, textstr1, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.92, textstr2, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.92, textstr3, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.92, textstr4, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/He_Gaussian_Fit_Antaress.pdf')
    plt.show()

### Plotting the positions, depths and widths of the fitted Gaussian lines - Overplotted (Rassine + Antaress)

In [ ]:
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
save = False

ref_He = AF.air2vac((10830.33977+10830.25010)/2) #From NIST database

ref_time = 59960

depth_index = 0
position_index = 1
width_index = 2
ew_index = -1

opacity = 0.1

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    #HA
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Antaress
    #Getting the scatter
    avg_depth = np.average(theta_Antaress_HA[:, depth_index], weights=1/err_theta_Antaress_HA[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Antaress_HA[:, depth_index] - avg_depth)**2, weights=1/err_theta_Antaress_HA[:, depth_index]**2))
    new_pos = (theta_Antaress_HA[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_theta_Antaress_HA[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Antaress_HA[:, width_index], weights=1/err_theta_Antaress_HA[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Antaress_HA[:, width_index] - avg_width)**2, weights=1/err_theta_Antaress_HA[:, width_index]**2))
    avg_ew = np.average(theta_Antaress_HA[:, ew_index], weights=1/err_theta_Antaress_HA[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Antaress_HA[:, ew_index] - avg_ew)**2, weights=1/err_theta_Antaress_HA[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HA, date_HA, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HA, theta_Antaress_HA[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HA, theta_Antaress_HA[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HA, theta_Antaress_HA[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HA, theta_Antaress_HA[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HA, theta_Antaress_HA[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HA, theta_Antaress_HA[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HA, theta_Antaress_HA[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HA, theta_Antaress_HA[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HA-ref_time, theta_Antaress_HA[:, depth_index], yerr=err_theta_Antaress_HA[:, depth_index], fmt='r.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
  
    ax2.errorbar(date_HA-ref_time, (theta_Antaress_HA[:, position_index]-ref_He)*3e8/ref_He, yerr=err_theta_Antaress_HA[:, position_index]*3e8/ref_He, fmt='r.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HA-ref_time, theta_Antaress_HA[:, width_index], yerr=err_theta_Antaress_HA[:, width_index], fmt='r.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HA-ref_time, theta_Antaress_HA[:, ew_index], yerr=err_theta_Antaress_HA[:,ew_index], fmt='r.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
        
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    
    #Rassine
    #Getting the scatter
    avg_depth = np.average(theta_rassine_HA[:, depth_index], weights=1/err_theta_rassine_HA[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_rassine_HA[:, depth_index] - avg_depth)**2, weights=1/err_theta_rassine_HA[:, depth_index]**2))
    new_pos = (theta_rassine_HA[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_theta_rassine_HA[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_rassine_HA[:, width_index], weights=1/err_theta_rassine_HA[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_rassine_HA[:, width_index] - avg_width)**2, weights=1/err_theta_rassine_HA[:, width_index]**2))
    avg_ew = np.average(theta_rassine_HA[:, ew_index], weights=1/err_theta_rassine_HA[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_rassine_HA[:, ew_index] - avg_ew)**2, weights=1/err_theta_rassine_HA[:, ew_index]**2))

    #Getting the binned points
    num_bins = 20
        
    binned_depth = ss.binned_statistic(date_HA, theta_rassine_HA[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HA, theta_rassine_HA[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HA, theta_rassine_HA[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HA, theta_rassine_HA[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HA, theta_rassine_HA[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HA, theta_rassine_HA[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HA, theta_rassine_HA[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HA, theta_rassine_HA[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HA-ref_time, theta_rassine_HA[:, depth_index], yerr=err_theta_rassine_HA[:, depth_index], fmt='b.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='b.', markersize=10)
  
    ax2.errorbar(date_HA-ref_time, (theta_rassine_HA[:, position_index]-ref_He)*3e8/ref_He, yerr=err_theta_rassine_HA[:, position_index]*3e8/ref_He, fmt='b.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='b.', markersize=10)
    
    ax3.errorbar(date_HA-ref_time, theta_rassine_HA[:, width_index], yerr=err_theta_rassine_HA[:, width_index], fmt='b.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='b.', markersize=10)
    
    ax4.errorbar(date_HA-ref_time, theta_rassine_HA[:, ew_index], yerr=err_theta_rassine_HA[:,ew_index], fmt='b.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='b.', markersize=10)

    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)

    textstr12 = textstr1+'\n'+'$\sigma$ = %.3f'%std_depth
    textstr22 = textstr2+'\n'+'$\sigma$ = %.3f'%std_pos
    textstr32 = textstr3+'\n'+'$\sigma$ = %.3f'%std_width
    textstr42 = textstr4+'\n'+'$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.90, textstr12, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.90, textstr22, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.90, textstr32, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.90, textstr42, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/He_Gaussian_Fit_Overplot_HA.pdf')
    plt.show()


    #HE
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Antaress
    #Getting the scatter
    avg_depth = np.average(theta_Antaress_HE[:, depth_index], weights=1/err_theta_Antaress_HE[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Antaress_HE[:, depth_index] - avg_depth)**2, weights=1/err_theta_Antaress_HE[:, depth_index]**2))
    new_pos = (theta_Antaress_HE[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_theta_Antaress_HE[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Antaress_HE[:, width_index], weights=1/err_theta_Antaress_HE[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Antaress_HE[:, width_index] - avg_width)**2, weights=1/err_theta_Antaress_HE[:, width_index]**2))
    avg_ew = np.average(theta_Antaress_HE[:, ew_index], weights=1/err_theta_Antaress_HE[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Antaress_HE[:, ew_index] - avg_ew)**2, weights=1/err_theta_Antaress_HE[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date_HE, date_HE, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date_HE, theta_Antaress_HE[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HE, theta_Antaress_HE[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HE, theta_Antaress_HE[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HE, theta_Antaress_HE[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HE, theta_Antaress_HE[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HE, theta_Antaress_HE[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HE, theta_Antaress_HE[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HE, theta_Antaress_HE[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HE-ref_time, theta_Antaress_HE[:, depth_index], yerr=err_theta_Antaress_HE[:, depth_index], fmt='r.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10)
  
    ax2.errorbar(date_HE-ref_time, (theta_Antaress_HE[:, position_index]-ref_He)*3e8/ref_He, yerr=err_theta_Antaress_HE[:, position_index]*3e8/ref_He, fmt='r.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='r.', markersize=10)
    
    ax3.errorbar(date_HE-ref_time, theta_Antaress_HE[:, width_index], yerr=err_theta_Antaress_HE[:, width_index], fmt='r.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date_HE-ref_time, theta_Antaress_HE[:, ew_index], yerr=err_theta_Antaress_HE[:,ew_index], fmt='r.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
        
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    
    #Rassine
    #Getting the scatter
    avg_depth = np.average(theta_rassine_HE[:, depth_index], weights=1/err_theta_rassine_HE[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_rassine_HE[:, depth_index] - avg_depth)**2, weights=1/err_theta_rassine_HE[:, depth_index]**2))
    new_pos = (theta_rassine_HE[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_theta_rassine_HE[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_rassine_HE[:, width_index], weights=1/err_theta_rassine_HE[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_rassine_HE[:, width_index] - avg_width)**2, weights=1/err_theta_rassine_HE[:, width_index]**2))
    avg_ew = np.average(theta_rassine_HE[:, ew_index], weights=1/err_theta_rassine_HE[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_rassine_HE[:, ew_index] - avg_ew)**2, weights=1/err_theta_rassine_HE[:, ew_index]**2))

    #Getting the binned points
    num_bins = 20
        
    binned_depth = ss.binned_statistic(date_HE, theta_rassine_HE[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date_HE, theta_rassine_HE[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date_HE, theta_rassine_HE[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date_HE, theta_rassine_HE[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date_HE, theta_rassine_HE[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date_HE, theta_rassine_HE[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date_HE, theta_rassine_HE[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date_HE, theta_rassine_HE[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date_HE-ref_time, theta_rassine_HE[:, depth_index], yerr=err_theta_rassine_HE[:, depth_index], fmt='b.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='b.', markersize=10)
  
    ax2.errorbar(date_HE-ref_time, (theta_rassine_HE[:, position_index]-ref_He)*3e8/ref_He, yerr=err_theta_rassine_HE[:, position_index]*3e8/ref_He, fmt='b.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='b.', markersize=10)
    
    ax3.errorbar(date_HE-ref_time, theta_rassine_HE[:, width_index], yerr=err_theta_rassine_HE[:, width_index], fmt='b.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='b.', markersize=10)
    
    ax4.errorbar(date_HE-ref_time, theta_rassine_HE[:, ew_index], yerr=err_theta_rassine_HE[:,ew_index], fmt='b.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='b.', markersize=10)

    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)

    textstr12 = textstr1+'\n'+'$\sigma$ = %.3f'%std_depth
    textstr22 = textstr2+'\n'+'$\sigma$ = %.3f'%std_pos
    textstr32 = textstr3+'\n'+'$\sigma$ = %.3f'%std_width
    textstr42 = textstr4+'\n'+'$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.90, textstr12, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.90, textstr22, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.90, textstr32, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.90, textstr42, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/He_Gaussian_Fit_Overplot_HE.pdf')
    plt.show()

#If there one mode of observation.   
else:
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots (2, 2, sharex=True, figsize=[12, 10])

    #Antaress
    #Getting the scatter
    avg_depth = np.average(theta_Antaress[:, depth_index], weights=1/err_theta_Antaress[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_Antaress[:, depth_index] - avg_depth)**2, weights=1/err_theta_Antaress[:, depth_index]**2))
    new_pos = (theta_Antaress[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_theta_Antaress[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_Antaress[:, width_index], weights=1/err_theta_Antaress[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_Antaress[:, width_index] - avg_width)**2, weights=1/err_theta_Antaress[:, width_index]**2))
    avg_ew = np.average(theta_Antaress[:, ew_index], weights=1/err_theta_Antaress[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_Antaress[:, ew_index] - avg_ew)**2, weights=1/err_theta_Antaress[:, ew_index]**2))
    
    #Getting the binned points
    num_bins = 20
    
    binned_date = ss.binned_statistic(date, date, statistic=np.ma.median, bins=num_bins)[0]
    
    binned_depth = ss.binned_statistic(date, theta_Antaress[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date, theta_Antaress[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date, theta_Antaress[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date, theta_Antaress[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date, theta_Antaress[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date, theta_Antaress[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date, theta_Antaress[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date, theta_Antaress[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date-ref_time, theta_Antaress[:, depth_index], yerr=err_theta_Antaress[:, depth_index], fmt='r.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='r.', markersize=10, label='Antaress')
  
    ax2.errorbar(date-ref_time, (theta_Antaress[:, position_index]-ref_He)*3e8/ref_He, yerr=err_theta_Antaress[:, position_index]*3e8/ref_He, fmt='r.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='r.', markersize=10)
    
    ax3.errorbar(date-ref_time, theta_Antaress[:, width_index], yerr=err_theta_Antaress[:, width_index], fmt='r.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='r.', markersize=10)
    
    ax4.errorbar(date-ref_time, theta_Antaress[:, ew_index], yerr=err_theta_Antaress[:,ew_index], fmt='r.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='r.', markersize=10)
        
    textstr1 = '$\sigma$ = %.3f'%std_depth
    textstr2 = '$\sigma$ = %.3f'%std_pos
    textstr3 = '$\sigma$ = %.3f'%std_width
    textstr4 = '$\sigma$ = %.3f'%std_ew
    
    #Rassine
    #Getting the scatter
    avg_depth = np.average(theta_rassine[:, depth_index], weights=1/err_theta_rassine[:, depth_index]**2)
    std_depth = np.sqrt(np.average((theta_rassine[:, depth_index] - avg_depth)**2, weights=1/err_theta_rassine[:, depth_index]**2))
    new_pos = (theta_rassine[:, position_index]-ref_He)*3e8/ref_He
    new_pos_err = err_theta_rassine[:, position_index]*3e8/ref_He
    avg_pos = np.average(new_pos, weights=1/new_pos_err**2)
    std_pos = np.sqrt(np.average((new_pos - avg_pos)**2, weights=1/new_pos_err**2))
    avg_width = np.average(theta_rassine[:, width_index], weights=1/err_theta_rassine[:, width_index]**2)
    std_width = np.sqrt(np.average((theta_rassine[:, width_index] - avg_width)**2, weights=1/err_theta_rassine[:, width_index]**2))
    avg_ew = np.average(theta_rassine[:, ew_index], weights=1/err_theta_rassine[:, ew_index]**2)
    std_ew = np.sqrt(np.average((theta_rassine[:, ew_index] - avg_ew)**2, weights=1/err_theta_rassine[:, ew_index]**2))

    #Getting the binned points
    num_bins = 20
        
    binned_depth = ss.binned_statistic(date, theta_rassine[:,depth_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_depth_err = ss.binned_statistic(date, theta_rassine[:,depth_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_pos = ss.binned_statistic(date, theta_rassine[:,position_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_pos_err = ss.binned_statistic(date, theta_rassine[:,position_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_width = ss.binned_statistic(date, theta_rassine[:,width_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_width_err = ss.binned_statistic(date, theta_rassine[:,width_index], statistic=np.ma.std, bins=num_bins)[0]
    binned_ew = ss.binned_statistic(date, theta_rassine[:,ew_index], statistic=np.ma.median, bins=num_bins)[0]
    binned_ew_err = ss.binned_statistic(date, theta_rassine[:,ew_index], statistic=np.ma.std, bins=num_bins)[0]

    ax1.errorbar(date-ref_time, theta_rassine[:, depth_index], yerr=err_theta_rassine[:, depth_index], fmt='b.', alpha=opacity)
    ax1.errorbar(binned_date-ref_time, binned_depth, yerr=binned_depth_err, fmt='b.', markersize=10, label='Rassine')
  
    ax2.errorbar(date-ref_time, (theta_rassine[:, position_index]-ref_He)*3e8/ref_He, yerr=err_theta_rassine[:, position_index]*3e8/ref_He, fmt='b.', alpha=opacity)
    ax2.errorbar(binned_date-ref_time, (binned_pos-ref_He)*3e8/ref_He, yerr=binned_pos_err*3e8/ref_He, fmt='b.', markersize=10)
    
    ax3.errorbar(date-ref_time, theta_rassine[:, width_index], yerr=err_theta_rassine[:, width_index], fmt='b.', alpha=opacity)
    ax3.errorbar(binned_date-ref_time, binned_width, yerr=binned_width_err, fmt='b.', markersize=10)
    
    ax4.errorbar(date-ref_time, theta_rassine[:, ew_index], yerr=err_theta_rassine[:,ew_index], fmt='b.', alpha=opacity)
    ax4.errorbar(binned_date-ref_time, binned_ew, yerr=binned_ew_err, fmt='b.', markersize=10)

    ax3.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax4.set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
    ax1.set_ylabel('Depth (flux)', fontsize=14)
    ax2.set_ylabel('Position (m/s)', fontsize=14)
    ax3.set_ylabel('Line width ($\AA$)', fontsize=14)
    ax4.set_ylabel('Equivalent width ($\AA$)', fontsize=14)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='y', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)

    textstr12 = textstr1+'\n'+'$\sigma$ = %.3f'%std_depth
    textstr22 = textstr2+'\n'+'$\sigma$ = %.3f'%std_pos
    textstr32 = textstr3+'\n'+'$\sigma$ = %.3f'%std_width
    textstr42 = textstr4+'\n'+'$\sigma$ = %.3f'%std_ew
    ax1.text(0.04, 0.90, textstr12, transform=ax1.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax2.text(0.62, 0.90, textstr22, transform=ax2.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax3.text(0.67, 0.90, textstr32, transform=ax3.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    ax4.text(0.67, 0.90, textstr42, transform=ax4.transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

    ax1.legend(fontsize=14)
    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/He_Gaussian_Fit_Overplot.pdf')
    plt.show()

### Plotting the differential evolution + resolving continuity issues + Correlation plot (Antaress)

In [ ]:
ref_time = 59970
save=True 


theta_in_question = theta
err_in_question = err_theta
date_in_question = date

num_bins = 20

binned_date = ss.binned_statistic(date_in_question, date_in_question, statistic=np.ma.median, bins=num_bins)[0]

new_param_name = param_name + ['He EW', 'Si EW']

fig, axs = plt.subplots (mt.ceil(theta_in_question.shape[1]/2), 2, sharex=True, figsize=[12, 14])
ax = axs.ravel()

for i in range(theta_in_question.shape[1]):
    avg = np.average(theta_in_question[:, i])
    std = np.sqrt(np.average((theta_in_question[:, i] - avg)**2))
    
    binned_data = ss.binned_statistic(date_in_question, theta_in_question[:, i], statistic=np.ma.median, bins=num_bins)[0]
    
    ax[i].plot(date_in_question - ref_time, theta_in_question[:, i], '.', alpha=0.2)
    ax[i].plot(binned_date - ref_time, binned_data, 'r.', markersize=10)
    
    textstr = '$\sigma$ = %.4f'%std
    ax[i].text(0.04, 0.90, textstr, transform=ax[i].transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))
    
    ax[i].set_ylabel(new_param_name[i], fontsize=14)
    ax[i].tick_params(axis='y', labelsize=14)
    if i == theta.shape[1]-1 or i == theta.shape[1]-2:
        ax[i].tick_params(axis='x', labelsize=14)
        ax[i].set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)
        
if not ax[-1].lines: 
    ax[-1].set_visible(False)
        

fig.tight_layout()
plt.subplots_adjust(hspace=0)
if save:
    plt.savefig('/Users/samsonmercier/Downloads/Timeseries_'+str(poly_order)+'th_order.pdf')

In [ ]:
reload(AF)

AF.Correlation_Plot('', theta[:, 0], theta[:, 1], err_theta[:, 0], err_theta[:, 1], 'T He', 'T Si', 'Antaress Temperature', '2023-01-26')

AF.Correlation_Plot('', theta[:, 2], theta[:, 3], err_theta[:, 2], err_theta[:, 3], 'n He', 'n Si', 'Antaress Density', '2023-01-26')

AF.Correlation_Plot('', theta[:, 5], theta[:, 6], err_theta[:, 5], err_theta[:, 6], 'He RV', 'Si RV', 'Antaress RV', '2023-01-26')

AF.Correlation_Plot('', theta[:, -2], theta[:, -1], err_theta[:, -2], err_theta[:, -1], 'He EW', 'Si EW', 'Antaress Eq. Width', '2023-01-26')


### Plotting the old way

In [ ]:
save=False

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    #Antaress
    print('ANTARESS')
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[16, 6])

    for i in range(len(total_lamda_HA)):
        ax1.errorbar(total_lamda_HA[i], total_norm_spctr_Antaress_HA[i], yerr=total_norm_err_Antaress_HA[i])
        ax1.set_xlabel('Wavelength ($\AA$)')
        ax1.set_ylabel('Flux')
        ax1.set_title('Normalized TC Spectra around He triplet (in vacuum) for HA')
        ax1.set_xlim([10825, 10835])
        ax1.set_ylim([0.3, 1.1])
    
    for i in range(len(total_lamda_HE)):
        ax2.errorbar(total_lamda_HE[i], total_norm_spctr_Antaress_HE[i], yerr=total_norm_err_Antaress_HE[i])
        ax2.set_xlabel('Wavelength ($\AA$)')
        ax2.set_ylabel('Flux')
        ax2.set_title('Normalized TC Spectra around He triplet (in vacuum) for HE')
        ax2.set_xlim([10825, 10835])
        ax2.set_ylim([0.3, 1.1])

    ax1.axvline(10833.2, color='b', label='He triplet location')
    ax1.axvline(10833.3, color='b')
    ax1.axvline(10832.1, color='b')

    ax2.axvline(10833.2, color='b', label='He triplet location')
    ax2.axvline(10833.3, color='b')
    ax2.axvline(10832.1, color='b')

    ax1.legend()
    ax2.legend()
    if save:
        plt.savefig('Plot_Results/Superpose_He_triplet_Vacuum_Antaress.pdf')
    plt.show()
  
    #Rassine
    print('Rassine')
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[16, 6])

    for i in range(len(rassine_total_lamda_HA)):
        ax1.errorbar(rassine_total_lamda_HA[i], rassine_total_norm_spctr_HA[i], yerr=rassine_total_norm_err_HA[i])
        ax1.set_xlabel('Wavelength ($\AA$)')
        ax1.set_ylabel('Flux')
        ax1.set_title('Normalized TC Spectra around He triplet (in vacuum) for HA')
        ax1.set_xlim([10825, 10835])
        ax1.set_ylim([0.3, 1.1])
    
    for i in range(len(rassine_total_lamda_HE)):
        ax2.errorbar(rassine_total_lamda_HE[i], rassine_total_norm_spctr_HE[i], yerr=rassine_total_norm_err_HE[i])
        ax2.set_xlabel('Wavelength ($\AA$)')
        ax2.set_ylabel('Flux')
        ax2.set_title('Normalized TC Spectra around He triplet (in vacuum) for HE')
        ax2.set_xlim([10825, 10835])
        ax2.set_ylim([0.3, 1.1])

    ax1.axvline(10833.2, color='b', label='He triplet location')
    ax1.axvline(10833.3, color='b')
    ax1.axvline(10832.1, color='b')

    ax2.axvline(10833.2, color='b', label='He triplet location')
    ax2.axvline(10833.3, color='b')
    ax2.axvline(10832.1, color='b')

    ax1.legend()
    ax2.legend()
    if save:
        plt.savefig('Plot_Results/Superpose_He_triplet_Vacuum_Rassine.pdf')
    plt.show()

#If there is one mode of observation
else:
    #Antaress
    print('Antaress')
    plt.figure(figsize=[15, 8])
    for i in range(len(total_lamda)):
        plt.errorbar(total_lamda[i], total_norm_spctr_Antaress[i], yerr=total_norm_err_Antaress[i])
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Flux')
    plt.title('Normalized TC Spectra around He triplet (in vacuum)')
    plt.xlim([10825, 10835])
    plt.ylim([0.3, 1.1])
    
    plt.axvline(10833.2, color='b', label='He triplet location')
    plt.axvline(10833.3, color='b')
    plt.axvline(10832.1, color='b')

    plt.legend()
    if save:
        plt.savefig('Plot_Results/Superpose_He_triplet_Vacuum_Antaress.pdf')
    plt.show()
    
    #Rassine
    print('Rassine')
    plt.figure(figsize=[15, 8])
    for i in range(len(rassine_total_lamda)):
        plt.errorbar(rassine_total_lamda[i], rassine_total_norm_spctr[i], yerr=rassine_total_norm_err[i])
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Flux')
    plt.title('Normalized TC Spectra around He triplet (in vacuum)')
    plt.xlim([10825, 10835])
    plt.ylim([0.3, 1.1])
    
    plt.axvline(10833.2, color='b', label='He triplet location')
    plt.axvline(10833.3, color='b')
    plt.axvline(10832.1, color='b')

    plt.legend()
    if save:
        plt.savefig('Plot_Results/Superpose_He_triplet_Vacuum_Rassine.pdf')
    plt.show()

## Perform air-to-vacuum correction

In [ ]:
save=False

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    #Antaress
    print('ANTARESS')
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[16, 6])

    for i in range(len(total_lamda_HA)):
        ax1.errorbar(AF.vac2air(total_lamda_HA[i]), total_norm_spctr_Antaress_HA[i], yerr=total_norm_err_Antaress_HA[i])
        ax1.set_xlabel('Wavelength ($\AA$)')
        ax1.set_ylabel('Flux')
        ax1.set_title('Normalized TC Spectra around He triplet (in air) for HA')
        ax1.set_xlim([10825, 10835])
        ax1.set_ylim([0.3, 1.1])
    
    for i in range(len(total_lamda_HE)):
        ax2.errorbar(AF.vac2air(total_lamda_HE[i]), total_norm_spctr_Antaress_HE[i], yerr=total_norm_err_Antaress_HE[i])
        ax2.set_xlabel('Wavelength ($\AA$)')
        ax2.set_ylabel('Flux')
        ax2.set_title('Normalized TC Spectra around He triplet (in air) for HE')
        ax2.set_xlim([10825, 10835])
        ax2.set_ylim([0.3, 1.1])

    ax1.axvline(AF.vac2air(10833.2), color='b', label='He triplet location')
    ax1.axvline(AF.vac2air(10833.3), color='b')
    ax1.axvline(AF.vac2air(10832.1), color='b')

    ax2.axvline(AF.vac2air(10833.2), color='b', label='He triplet location')
    ax2.axvline(AF.vac2air(10833.3), color='b')
    ax2.axvline(AF.vac2air(10832.1), color='b')

    ax1.legend()
    ax2.legend()
    if save:
        plt.savefig('Plot_Results/Superpose_He_triplet_Vacuum_Antaress.pdf')
    plt.show()
  
    #Rassine
    print('Rassine')
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[16, 6])

    for i in range(len(rassine_total_lamda_HA)):
        ax1.errorbar(AF.vac2air(rassine_total_lamda_HA[i]), rassine_total_norm_spctr_HA[i], yerr=rassine_total_norm_err_HA[i])
        ax1.set_xlabel('Wavelength ($\AA$)')
        ax1.set_ylabel('Flux')
        ax1.set_title('Normalized TC Spectra around He triplet (in air) for HA')
        ax1.set_xlim([10825, 10835])
        ax1.set_ylim([0.3, 1.1])
    
    for i in range(len(rassine_total_lamda_HE)):
        ax2.errorbar(AF.vac2air(rassine_total_lamda_HE[i]), rassine_total_norm_spctr_HE[i], yerr=rassine_total_norm_err_HE[i])
        ax2.set_xlabel('Wavelength ($\AA$)')
        ax2.set_ylabel('Flux')
        ax2.set_title('Normalized TC Spectra around He triplet (in air) for HE')
        ax2.set_xlim([10825, 10835])
        ax2.set_ylim([0.3, 1.1])

    ax1.axvline(AF.vac2air(10833.2), color='b', label='He triplet location')
    ax1.axvline(AF.vac2air(10833.3), color='b')
    ax1.axvline(AF.vac2air(10832.1), color='b')

    ax2.axvline(AF.vac2air(10833.2), color='b', label='He triplet location')
    ax2.axvline(AF.vac2air(10833.3), color='b')
    ax2.axvline(AF.vac2air(10832.1), color='b')

    ax1.legend()
    ax2.legend()
    if save:
        plt.savefig('Plot_Results/Superpose_He_triplet_Vacuum_Rassine.pdf')
    plt.show()

#If there is one mode of observation
else:
    #Antaress
    print('Antaress')
    plt.figure(figsize=[15, 8])
    for i in range(len(total_lamda)):
        plt.errorbar(AF.vac2air(total_lamda[i]), total_norm_spctr_Antaress[i], yerr=total_norm_err_Antaress[i])
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Flux')
    plt.title('Normalized TC Spectra around He triplet (in air)')
    plt.xlim([10825, 10835])
    plt.ylim([0.3, 1.1])
    
    plt.axvline(AF.vac2air(10833.2), color='b', label='He triplet location')
    plt.axvline(AF.vac2air(10833.3), color='b')
    plt.axvline(AF.vac2air(10832.1), color='b')

    plt.legend()
    if save:
        plt.savefig('Plot_Results/Superpose_He_triplet_Vacuum_Antaress.pdf')
    plt.show()
    
    #Rassine
    print('Rassine')
    plt.figure(figsize=[15, 8])
    for i in range(len(rassine_total_lamda)):
        plt.errorbar(AF.vac2air(rassine_total_lamda[i]), rassine_total_norm_spctr[i], yerr=rassine_total_norm_err[i])
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Flux')
    plt.title('Normalized TC Spectra around He triplet (in air)')
    plt.xlim([10825, 10835])
    plt.ylim([0.3, 1.1])
    
    plt.axvline(AF.vac2air(10833.2), color='b', label='He triplet location')
    plt.axvline(AF.vac2air(10833.3), color='b')
    plt.axvline(AF.vac2air(10832.1), color='b')

    plt.legend()
    if save:
        plt.savefig('Plot_Results/Superpose_He_triplet_Vacuum_Rassine.pdf')
    plt.show()

## Plotting the continuum

In [ ]:
save=False
#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    #Antaress
    print('ANTARESS')
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[16, 6])

    for i in range(len(total_lamda_HA)):
        ax1.errorbar(total_lamda_HA[i], total_norm_spctr_Antaress_HA[i], yerr=total_norm_err_Antaress_HA[i])
        ax1.set_xlabel('Wavelength ($\AA$)')
        ax1.set_ylabel('Flux')
        ax1.set_title('Normalized TC Spectra of continuum (in vacuum) for HA')
        ax1.set_xlim([10767, 10770])
        ax1.set_ylim([0.3, 1.1])
    
    for i in range(len(total_lamda_HE)):
        ax2.errorbar(total_lamda_HE[i], total_norm_spctr_Antaress_HE[i], yerr=total_norm_err_Antaress_HE[i])
        ax2.set_xlabel('Wavelength ($\AA$)')
        ax2.set_ylabel('Flux')
        ax2.set_title('Normalized TC Spectra of continuum (in vacuum) for HE')
        ax2.set_xlim([10767, 10770])
        ax2.set_ylim([0.3, 1.1])

    ax1.axvline(10833.2, color='b', label='He triplet location')
    ax1.axvline(10833.3, color='b')
    ax1.axvline(10832.1, color='b')

    ax2.axvline(10833.2, color='b', label='He triplet location')
    ax2.axvline(10833.3, color='b')
    ax2.axvline(10832.1, color='b')

    ax1.legend()
    ax2.legend()
    if save:
        plt.savefig('Plot_Results/Superpose_He_triplet_Vacuum_Antaress.pdf')
    plt.show()
  
    #Rassine
    print('Rassine')
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[16, 6])

    for i in range(len(rassine_total_lamda_HA)):
        ax1.errorbar(rassine_total_lamda_HA[i], rassine_total_norm_spctr_HA[i], yerr=rassine_total_norm_err_HA[i])
        ax1.set_xlabel('Wavelength ($\AA$)')
        ax1.set_ylabel('Flux')
        ax1.set_title('Normalized TC Spectra of continuum (in vacuum) for HA')
        ax1.set_xlim([10767, 10770])
        ax1.set_ylim([0.3, 1.1])
    
    for i in range(len(rassine_total_lamda_HE)):
        ax2.errorbar(rassine_total_lamda_HE[i], rassine_total_norm_spctr_HE[i], yerr=rassine_total_norm_err_HE[i])
        ax2.set_xlabel('Wavelength ($\AA$)')
        ax2.set_ylabel('Flux')
        ax2.set_title('Normalized TC Spectra of continuum (in vacuum) for HE')
        ax2.set_xlim([10767, 10770])
        ax2.set_ylim([0.3, 1.1])

    ax1.axvline(10833.2, color='b', label='He triplet location')
    ax1.axvline(10833.3, color='b')
    ax1.axvline(10832.1, color='b')

    ax2.axvline(10833.2, color='b', label='He triplet location')
    ax2.axvline(10833.3, color='b')
    ax2.axvline(10832.1, color='b')

    ax1.legend()
    ax2.legend()
    if save:
        plt.savefig('Plot_Results/Superpose_He_triplet_Vacuum_Rassine.pdf')
    plt.show()

#If there is one mode of observation
else:
    #Antaress
    print('Antaress')
    plt.figure(figsize=[15, 8])
    for i in range(len(total_lamda)):
        plt.errorbar(total_lamda[i], total_norm_spctr_Antaress[i], yerr=total_norm_err_Antaress[i])
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Flux')
    plt.title('Normalized TC Spectra of continuum (in vacuum)')
    plt.xlim([10767, 10770])
    
    plt.axvline(10833.2, color='b', label='He triplet location')
    plt.axvline(10833.3, color='b')
    plt.axvline(10832.1, color='b')

    plt.legend()
    if save:
        plt.savefig('Plot_Results/Superpose_He_triplet_Vacuum_Antaress.pdf')
    plt.show()
    
    #Rassine
    print('Rassine')
    plt.figure(figsize=[15, 8])
    for i in range(len(rassine_total_lamda)):
        plt.errorbar(rassine_total_lamda[i], rassine_total_norm_spctr[i], yerr=rassine_total_norm_err[i])
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Flux')
    plt.title('Normalized TC Spectra of continuum (in vacuum)')
    plt.xlim([10767, 10770])
    
    plt.axvline(10833.2, color='b', label='He triplet location')
    plt.axvline(10833.3, color='b')
    plt.axvline(10832.1, color='b')

    plt.legend()
    if save:
        plt.savefig('Plot_Results/Superpose_He_triplet_Vacuum_Rassine.pdf')
    plt.show()

## Building a master spectrum and residuals

In [ ]:
save = True

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    master_lamda_HA = np.average(total_lamda_HA, axis=0)
    master_norm_spctr_HA = np.average(total_norm_spctr_Antaress_HA, axis=0, weights=1/total_norm_err_Antaress_HA**2)
    master_norm_err_HA = 1/np.sqrt(np.sum(1/total_norm_err_Antaress_HA**2, axis=0))

    master_lamda_HE = np.average(total_lamda_HE, axis=0)
    master_norm_spctr_HE = np.average(total_norm_spctr_Antaress_HE, axis=0, weights=1/total_norm_err_Antaress_HE**2)
    master_norm_err_HE = 1/np.sqrt(np.sum(1/total_norm_err_Antaress_HE**2, axis=0))

    #HA
    fig, ((ax1, ax3), (ax2, ax4)) = plt.subplots(1, 2, figsize=[16, 6])

    #Master spectrum - Antaress
    ax1.errorbar(master_lamda_HA, master_norm_spctr_HA, yerr=master_norm_err_HA, fmt='.', label='Master')
    ax1.axvline(10833.2, color='red', label='He triplet location')
    ax1.axvline(10833.3, color='red')
    ax1.axvline(10832.1, color='red')
    ax1.legend(fontsize=14) 
    ax1.set_ylabel('Normalized Flux', fontsize=14)
    ax1.set_title('Antaress Master')
    
    for i in range(0, len(total_norm_spctr_HA), 2):
        ax2.errorbar(master_lamda_HA, total_norm_spctr_Antaress_HA[i] - master_norm_spctr_HA, yerr=np.sqrt(total_norm_err_Antaress_HA[i]**2+master_norm_err_HA**2), fmt='.')
    ax2.set_xlabel('Wavelength ($\AA$)', fontsize=14)
    ax2.set_ylabel('Residuals', fontsize=14)
    
    ax1.tick_params(axis='both', labelsize=14)
    ax2.tick_params(axis='both', labelsize=13)
    
    ax2.axvline(10833.2, color='red', label='He triplet location')
    ax2.axvline(10833.3, color='red')
    ax2.axvline(10832.1, color='red')
    ax2.axvline(10830.1, color='orange')
    ax1.axvline(10830.1, color='orange', label='Si line location')
    ax1.set_xlim([10825, 10835])
    ax2.set_xlim([10825, 10835])
    
    #Master spectrum - Rassine
    ax3.plot(rassine_master_lamda_HA, rassine_master_spctr_HA, '.', label='Master')
    ax3.axvline(10833.2, color='red', label='He triplet location')
    ax3.axvline(10833.3, color='red')
    ax3.axvline(10832.1, color='red')
    ax3.axvline(10830.1, color='orange', label='Si line location')
    ax3.legend(fontsize=14) 
    ax3.set_ylabel('Normalized Flux', fontsize=14)
    ax3.set_title('Rassine Master')

    for i in range(0, len(rassine_total_norm_spctr_HA), 2):
        ax4.plot(rassine_master_lamda_HA, rassine_total_norm_spctr_HA[i] - rassine_master_spctr_HA, '.')
    ax4.set_xlabel('Wavelength ($\AA$)', fontsize=14)
    ax4.set_ylabel('Residuals', fontsize=14)

    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=13)
    
    ax4.axvline(10833.2, color='red')
    ax4.axvline(10833.3, color='red')
    ax4.axvline(10832.1, color='red')
    ax4.axvline(10830.1, color='orange')
    ax3.set_xlim([10825, 10835])
    ax4.set_xlim([10825, 10835])

    plt.subplots_adjust(hspace=0)
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/He_triplet_Master_HA.pdf')
    plt.show()
    
    
    #HE
    fig, ((ax1, ax3), (ax2, ax4)) = plt.subplots(1, 2, figsize=[16, 6])

    #Master spectrum - Antaress
    ax1.errorbar(master_lamda_HE, master_norm_spctr_HE, yerr=master_norm_err_HE, fmt='.', label='Master')
    ax1.axvline(10833.2, color='red', label='He triplet location')
    ax1.axvline(10833.3, color='red')
    ax1.axvline(10832.1, color='red')
    ax1.legend(fontsize=14) 
    ax1.set_ylabel('Normalized Flux', fontsize=14)
    ax1.set_title('Antaress Master')
    
    for i in range(0, len(total_norm_spctr_HE), 2):
        ax2.errorbar(master_lamda_HE, total_norm_spctr_Antaress_HE[i] - master_norm_spctr_HE, yerr=np.sqrt(total_norm_err_Antaress_HE[i]**2+master_norm_err_HE**2), fmt='.')
    ax2.set_xlabel('Wavelength ($\AA$)', fontsize=14)
    ax2.set_ylabel('Residuals', fontsize=14)
    
    ax1.tick_params(axis='both', labelsize=14)
    ax2.tick_params(axis='both', labelsize=13)
    
    ax2.axvline(10833.2, color='red', label='He triplet location')
    ax2.axvline(10833.3, color='red')
    ax2.axvline(10832.1, color='red')
    ax2.axvline(10830.1, color='orange')
    ax1.axvline(10830.1, color='orange', label='Si line location')
    ax1.set_xlim([10825, 10835])
    ax2.set_xlim([10825, 10835])
    
    #Master spectrum - Rassine
    ax3.plot(rassine_master_lamda_HE, rassine_master_spctr_HE, '.', label='Master')
    ax3.axvline(10833.2, color='red', label='He triplet location')
    ax3.axvline(10833.3, color='red')
    ax3.axvline(10832.1, color='red')
    ax3.axvline(10830.1, color='orange', label='Si line location')
    ax3.legend(fontsize=14) 
    ax3.set_ylabel('Normalized Flux', fontsize=14)
    ax3.set_title('Rassine Master')

    for i in range(0, len(rassine_total_norm_spctr_HE), 2):
        ax4.plot(rassine_master_lamda_HE, rassine_total_norm_spctr_HE[i] - rassine_master_spctr_HE, '.')
    ax4.set_xlabel('Wavelength ($\AA$)', fontsize=14)
    ax4.set_ylabel('Residuals', fontsize=14)

    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=13)
    
    ax4.axvline(10833.2, color='red')
    ax4.axvline(10833.3, color='red')
    ax4.axvline(10832.1, color='red')
    ax4.axvline(10830.1, color='orange')
    ax3.set_xlim([10825, 10835])
    ax4.set_xlim([10825, 10835])

    plt.subplots_adjust(hspace=0)
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/He_triplet_Master_HA.pdf')
    plt.show()    
    
    
    
#If there is one mode of observation
else:
    fig, ((ax1, ax3), (ax2, ax4)) = plt.subplots(2, 2, sharex=True, gridspec_kw={'height_ratios': [3, 1]}, figsize=[15, 8])
    
    #Master spectrum - Antaress
    master_lamda = np.average(total_lamda, axis=0) #-- All spectra are interpolated on the same wavelength solution
    master_norm_spctr = np.average(total_norm_spctr_Antaress, axis=0, weights = 1/total_norm_err_Antaress**2)
    master_norm_err = 1/np.sqrt(np.sum(1/total_norm_err_Antaress**2, axis=0))
    
    ax1.errorbar(master_lamda, master_norm_spctr, yerr=master_norm_err, fmt='.', label='Master')
    ax1.axvline(10833.2, color='red', label='He triplet location')
    ax1.axvline(10833.3, color='red')
    ax1.axvline(10832.1, color='red')
    ax1.legend(fontsize=14) 
    ax1.set_ylabel('Normalized Flux', fontsize=14)
    ax1.set_title('Antaress Master')
    
    for i in range(0, len(total_norm_spctr), 2):
        ax2.errorbar(master_lamda, total_norm_spctr_Antaress[i] - master_norm_spctr, yerr=np.sqrt(total_norm_err_Antaress[i]**2+master_norm_err**2), fmt='.')
    ax2.set_xlabel('Wavelength ($\AA$)', fontsize=14)
    ax2.set_ylabel('Residuals', fontsize=14)
    
    ax1.tick_params(axis='both', labelsize=14)
    ax2.tick_params(axis='both', labelsize=13)
    
    ax2.axvline(10833.2, color='red', label='He triplet location')
    ax2.axvline(10833.3, color='red')
    ax2.axvline(10832.1, color='red')
    ax2.axvline(10830.1, color='orange')
    ax1.axvline(10830.1, color='orange', label='Si line location')
    ax1.set_xlim([10825, 10835])
    ax2.set_xlim([10825, 10835])
    
    #Master spectrum - Rassine
    ax3.plot(rassine_master_lamda, rassine_master_spctr, '.', label='Master')
    ax3.axvline(10833.2, color='red', label='He triplet location')
    ax3.axvline(10833.3, color='red')
    ax3.axvline(10832.1, color='red')
    ax3.axvline(10830.1, color='orange', label='Si line location')
    ax3.legend(fontsize=14) 
    ax3.set_ylabel('Normalized Flux', fontsize=14)
    ax3.set_title('Rassine Master')

    for i in range(0, len(rassine_total_norm_spctr), 2):
        ax4.plot(rassine_master_lamda, rassine_total_norm_spctr[i] - rassine_master_spctr, '.')
    ax4.set_xlabel('Wavelength ($\AA$)', fontsize=14)
    ax4.set_ylabel('Residuals', fontsize=14)

    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=13)
    
    ax4.axvline(10833.2, color='red')
    ax4.axvline(10833.3, color='red')
    ax4.axvline(10832.1, color='red')
    ax4.axvline(10830.1, color='orange')
    ax3.set_xlim([10825, 10835])
    ax4.set_xlim([10825, 10835])

    plt.subplots_adjust(hspace=0)
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/He_triplet_Master.pdf')
    plt.show()

## Testing out some code for fitting

In [ ]:
l = 10

overarching_low = 10827.0
overarching_up = 10834.1

include_ranges = ((10827.0, 10829.5), (10830.6, 10834.1))
bound_wav = AF.bound(overarching_low, overarching_up, total_lamda[l], total_lamda[l])
bound_flux = AF.bound(overarching_low, overarching_up, total_lamda[l], total_norm_spctr_Antaress[l])
bound_err = AF.bound(overarching_low, overarching_up, total_lamda[l], total_norm_err_Antaress[l])

plt.errorbar(bound_wav, bound_flux, yerr=bound_err, fmt='b.')

includ_lamda = []
includ_flux = []
includ_err = []
for x in include_ranges:
    includ_lamda.append(AF.bound(x[0], x[1], bound_wav, bound_wav))
    includ_flux.append(AF.bound(x[0], x[1], bound_wav, bound_flux))
    includ_err.append(AF.bound(x[0], x[1], bound_wav, bound_err))
    
include_lamda = np.array(list(itertools.chain.from_iterable(includ_lamda)))
include_flux = np.array(list(itertools.chain.from_iterable(includ_flux)))
include_err = np.array(list(itertools.chain.from_iterable(includ_err)))

plt.errorbar(include_lamda, include_flux, yerr=include_err, fmt='r.')
plt.show()

## Periodogram for the position, depth and widths of the He line

In [ ]:
save = False
oscillation_freq = 24*60/5.4 #in 1/days!!

depth_index = 0
position_index = 1
width_index = 2
ew_index = -1

#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

    #HA
    print('HA')
    #Antaress
    theta_in_question = theta_Antaress_HA
    err_in_question = err_theta_Antaress_HA

    fig, ((ax1, ax2, ax3, ax4), (ax5, ax6, ax7, ax8)) = plt.subplots (2, 4, figsize=[20, 10])

    LS_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).autopower(nyquist_factor=1)
    LS_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).autopower(nyquist_factor=1)
    LS_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).autopower(nyquist_factor=1)
    LS_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).autopower(nyquist_factor=1)

    probas = [0.1, 0.01, 0.001]
    FAP_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  

    ax1.loglog(LS_depth[0], LS_depth[1])
    ax2.loglog(LS_position[0], LS_position[1])
    ax3.loglog(LS_width[0], LS_width[1])
    ax4.loglog(LS_ew[0], LS_ew[1])
    ax1.set_xlabel('Frequency (1/d)', fontsize=14)
    ax2.set_xlabel('Frequency (1/d)', fontsize=14)
    ax3.set_xlabel('Frequency (1/d)', fontsize=14)
    ax4.set_xlabel('Frequency (1/d)', fontsize=14)
    ax1.set_ylabel('Power', fontsize=14)
    ax2.set_ylabel('Power', fontsize=14)
    ax3.set_ylabel('Power', fontsize=14)
    ax4.set_ylabel('Power', fontsize=14)
    ax1.set_title('Depth', fontsize=14)
    ax2.set_title('Position', fontsize=14)
    ax3.set_title('Line Width', fontsize=14)
    ax4.set_title('Equivalent Width', fontsize=14)
    ax1.tick_params(axis='both', labelsize=14)
    ax2.tick_params(axis='both', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    ax1.axvline(oscillation_freq, color='r', label='Oscillation')
    ax2.axvline(oscillation_freq, color='r')
    ax3.axvline(oscillation_freq, color='r')
    ax4.axvline(oscillation_freq, color='r')
    ax1.axhline(FAP_depth[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax1.axhline(FAP_depth[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax1.axhline(FAP_depth[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax2.axhline(FAP_position[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax2.axhline(FAP_position[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax2.axhline(FAP_position[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax3.axhline(FAP_width[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax3.axhline(FAP_width[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax3.axhline(FAP_width[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax4.axhline(FAP_ew[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax4.axhline(FAP_ew[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax4.axhline(FAP_ew[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    if max(LS_depth[1])>FAP_depth[0]:
        max_power = np.where(LS_depth[1]==max(LS_depth[1]))
        ax1.axvline(LS_depth[0][max_power], color='darkblue', label='Maximum')
    if max(LS_position[1])>FAP_position[0]:
        max_power = np.where(LS_position[1]==max(LS_position[1]))
        ax2.axvline(LS_position[0][max_power], color='darkblue')
    if max(LS_width[1])>FAP_width[0]:
        max_power = np.where(LS_width[1]==max(LS_width[1]))
        ax3.axvline(LS_width[0][max_power], color='darkblue')
    if max(LS_ew[1])>FAP_ew[0]:
        max_power = np.where(LS_ew[1]==max(LS_ew[1]))
        ax4.axvline(LS_ew[0][max_power], color='darkblue')

    #Rassine
    theta_in_question = theta_rassine_HA
    err_in_question = err_theta_rassine_HA

    LS_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).autopower(nyquist_factor=1)
    LS_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).autopower(nyquist_factor=1)
    LS_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).autopower(nyquist_factor=1)
    LS_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).autopower(nyquist_factor=1)

    probas = [0.1, 0.01, 0.001]
    FAP_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  

    ax5.loglog(LS_depth[0], LS_depth[1])
    ax6.loglog(LS_position[0], LS_position[1])
    ax7.loglog(LS_width[0], LS_width[1])
    ax8.loglog(LS_ew[0], LS_ew[1])
    ax5.set_xlabel('Frequency (1/d)', fontsize=14)
    ax6.set_xlabel('Frequency (1/d)', fontsize=14)
    ax7.set_xlabel('Frequency (1/d)', fontsize=14)
    ax8.set_xlabel('Frequency (1/d)', fontsize=14)
    ax5.set_ylabel('Power', fontsize=14)
    ax6.set_ylabel('Power', fontsize=14)
    ax7.set_ylabel('Power', fontsize=14)
    ax8.set_ylabel('Power', fontsize=14)
    ax5.set_title('Depth', fontsize=14)
    ax6.set_title('Position', fontsize=14)
    ax7.set_title('Line Width', fontsize=14)
    ax8.set_title('Equivalent Width', fontsize=14)
    ax5.tick_params(axis='both', labelsize=14)
    ax6.tick_params(axis='both', labelsize=14)
    ax7.tick_params(axis='both', labelsize=14)
    ax8.tick_params(axis='both', labelsize=14)
    ax5.axvline(oscillation_freq, color='r', label='Oscillation')
    ax6.axvline(oscillation_freq, color='r')
    ax7.axvline(oscillation_freq, color='r')
    ax8.axvline(oscillation_freq, color='r')
    ax5.axhline(FAP_depth[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax5.axhline(FAP_depth[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax5.axhline(FAP_depth[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax6.axhline(FAP_position[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax6.axhline(FAP_position[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax6.axhline(FAP_position[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax7.axhline(FAP_width[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax7.axhline(FAP_width[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax7.axhline(FAP_width[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax8.axhline(FAP_ew[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax8.axhline(FAP_ew[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax8.axhline(FAP_ew[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    if max(LS_depth[1])>FAP_depth[0]:
        max_power = np.where(LS_depth[1]==max(LS_depth[1]))
        ax5.axvline(LS_depth[0][max_power], color='darkblue', label='Maximum')
    if max(LS_position[1])>FAP_position[0]:
        max_power = np.where(LS_position[1]==max(LS_position[1]))
        ax6.axvline(LS_position[0][max_power], color='darkblue')
    if max(LS_width[1])>FAP_width[0]:
        max_power = np.where(LS_width[1]==max(LS_width[1]))
        ax7.axvline(LS_width[0][max_power], color='darkblue')
    if max(LS_ew[1])>FAP_ew[0]:
        max_power = np.where(LS_ew[1]==max(LS_ew[1]))
        ax8.axvline(LS_ew[0][max_power], color='darkblue')

    ax1.annotate('ANTARESS', (-0.2, 0.5), xytext=(-45, 0), ha='right', va='center',
                size=20, rotation=90, xycoords='axes fraction',
                textcoords='offset points')
    ax5.annotate('RASSINE', (-0.2, 0.5), xytext=(-45, 0), ha='right', va='center',
                size=20, rotation=90, xycoords='axes fraction',
                textcoords='offset points')
    
    ax1.legend(fontsize=14)
    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Periodogram_He_fit_HA.pdf')
    plt.show()

    #HE
    print('HE')
    #Antaress
    theta_in_question = theta_Antaress_HE
    err_in_question = err_theta_Antaress_HE

    fig, ((ax1, ax2, ax3, ax4), (ax5, ax6, ax7, ax8)) = plt.subplots (2, 4, figsize=[20, 10])

    LS_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).autopower(nyquist_factor=1)
    LS_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).autopower(nyquist_factor=1)
    LS_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).autopower(nyquist_factor=1)
    LS_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).autopower(nyquist_factor=1)

    probas = [0.1, 0.01, 0.001]
    FAP_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  

    ax1.loglog(LS_depth[0], LS_depth[1])
    ax2.loglog(LS_position[0], LS_position[1])
    ax3.loglog(LS_width[0], LS_width[1])
    ax4.loglog(LS_ew[0], LS_ew[1])
    ax1.set_xlabel('Frequency (1/d)', fontsize=14)
    ax2.set_xlabel('Frequency (1/d)', fontsize=14)
    ax3.set_xlabel('Frequency (1/d)', fontsize=14)
    ax4.set_xlabel('Frequency (1/d)', fontsize=14)
    ax1.set_ylabel('Power', fontsize=14)
    ax2.set_ylabel('Power', fontsize=14)
    ax3.set_ylabel('Power', fontsize=14)
    ax4.set_ylabel('Power', fontsize=14)
    ax1.set_title('Depth', fontsize=14)
    ax2.set_title('Position', fontsize=14)
    ax3.set_title('Line Width', fontsize=14)
    ax4.set_title('Equivalent Width', fontsize=14)
    ax1.tick_params(axis='both', labelsize=14)
    ax2.tick_params(axis='both', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    ax1.axvline(oscillation_freq, color='r', label='Oscillation')
    ax2.axvline(oscillation_freq, color='r')
    ax3.axvline(oscillation_freq, color='r')
    ax4.axvline(oscillation_freq, color='r')
    ax1.axhline(FAP_depth[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax1.axhline(FAP_depth[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax1.axhline(FAP_depth[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax2.axhline(FAP_position[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax2.axhline(FAP_position[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax2.axhline(FAP_position[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax3.axhline(FAP_width[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax3.axhline(FAP_width[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax3.axhline(FAP_width[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax4.axhline(FAP_ew[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax4.axhline(FAP_ew[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax4.axhline(FAP_ew[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    if max(LS_depth[1])>FAP_depth[0]:
        max_power = np.where(LS_depth[1]==max(LS_depth[1]))
        ax1.axvline(LS_depth[0][max_power], color='darkblue', label='Maximum')
    if max(LS_position[1])>FAP_position[0]:
        max_power = np.where(LS_position[1]==max(LS_position[1]))
        ax2.axvline(LS_position[0][max_power], color='darkblue')
    if max(LS_width[1])>FAP_width[0]:
        max_power = np.where(LS_width[1]==max(LS_width[1]))
        ax3.axvline(LS_width[0][max_power], color='darkblue')
    if max(LS_ew[1])>FAP_ew[0]:
        max_power = np.where(LS_ew[1]==max(LS_ew[1]))
        ax4.axvline(LS_ew[0][max_power], color='darkblue')

    #Rassine
    theta_in_question = theta_rassine_HE
    err_in_question = err_theta_rassine_HE

    LS_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).autopower(nyquist_factor=1)
    LS_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).autopower(nyquist_factor=1)
    LS_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).autopower(nyquist_factor=1)
    LS_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).autopower(nyquist_factor=1)

    probas = [0.1, 0.01, 0.001]
    FAP_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  

    ax5.loglog(LS_depth[0], LS_depth[1])
    ax6.loglog(LS_position[0], LS_position[1])
    ax7.loglog(LS_width[0], LS_width[1])
    ax8.loglog(LS_ew[0], LS_ew[1])
    ax5.set_xlabel('Frequency (1/d)', fontsize=14)
    ax6.set_xlabel('Frequency (1/d)', fontsize=14)
    ax7.set_xlabel('Frequency (1/d)', fontsize=14)
    ax8.set_xlabel('Frequency (1/d)', fontsize=14)
    ax5.set_ylabel('Power', fontsize=14)
    ax6.set_ylabel('Power', fontsize=14)
    ax7.set_ylabel('Power', fontsize=14)
    ax8.set_ylabel('Power', fontsize=14)
    ax5.set_title('Depth', fontsize=14)
    ax6.set_title('Position', fontsize=14)
    ax7.set_title('Line Width', fontsize=14)
    ax8.set_title('Equivalent Width', fontsize=14)
    ax5.tick_params(axis='both', labelsize=14)
    ax6.tick_params(axis='both', labelsize=14)
    ax7.tick_params(axis='both', labelsize=14)
    ax8.tick_params(axis='both', labelsize=14)
    ax5.axvline(oscillation_freq, color='r', label='Oscillation')
    ax6.axvline(oscillation_freq, color='r')
    ax7.axvline(oscillation_freq, color='r')
    ax8.axvline(oscillation_freq, color='r')
    ax5.axhline(FAP_depth[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax5.axhline(FAP_depth[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax5.axhline(FAP_depth[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax6.axhline(FAP_position[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax6.axhline(FAP_position[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax6.axhline(FAP_position[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax7.axhline(FAP_width[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax7.axhline(FAP_width[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax7.axhline(FAP_width[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax8.axhline(FAP_ew[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax8.axhline(FAP_ew[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax8.axhline(FAP_ew[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    if max(LS_depth[1])>FAP_depth[0]:
        max_power = np.where(LS_depth[1]==max(LS_depth[1]))
        ax5.axvline(LS_depth[0][max_power], color='darkblue', label='Maximum')
    if max(LS_position[1])>FAP_position[0]:
        max_power = np.where(LS_position[1]==max(LS_position[1]))
        ax6.axvline(LS_position[0][max_power], color='darkblue')
    if max(LS_width[1])>FAP_width[0]:
        max_power = np.where(LS_width[1]==max(LS_width[1]))
        ax7.axvline(LS_width[0][max_power], color='darkblue')
    if max(LS_ew[1])>FAP_ew[0]:
        max_power = np.where(LS_ew[1]==max(LS_ew[1]))
        ax8.axvline(LS_ew[0][max_power], color='darkblue')

    ax1.annotate('ANTARESS', (-0.2, 0.5), xytext=(-45, 0), ha='right', va='center',
                size=20, rotation=90, xycoords='axes fraction',
                textcoords='offset points')
    ax5.annotate('RASSINE', (-0.2, 0.5), xytext=(-45, 0), ha='right', va='center',
                size=20, rotation=90, xycoords='axes fraction',
                textcoords='offset points')
    
    ax1.legend(fontsize=14)
    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Periodogram_He_fit_HE.pdf')
    plt.show()
    
#If there is one mode of observation.    
else:    
    #Antaress
    theta_in_question = theta_Antaress
    err_in_question = err_theta_Antaress

    fig, ((ax1, ax2, ax3, ax4), (ax5, ax6, ax7, ax8)) = plt.subplots (2, 4, figsize=[20, 10])

    LS_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).autopower(nyquist_factor=1)
    LS_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).autopower(nyquist_factor=1)
    LS_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).autopower(nyquist_factor=1)
    LS_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).autopower(nyquist_factor=1)

    probas = [0.1, 0.01, 0.001]
    FAP_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  

    ax1.loglog(LS_depth[0], LS_depth[1])
    ax2.loglog(LS_position[0], LS_position[1])
    ax3.loglog(LS_width[0], LS_width[1])
    ax4.loglog(LS_ew[0], LS_ew[1])
    ax1.set_xlabel('Frequency (1/d)', fontsize=14)
    ax2.set_xlabel('Frequency (1/d)', fontsize=14)
    ax3.set_xlabel('Frequency (1/d)', fontsize=14)
    ax4.set_xlabel('Frequency (1/d)', fontsize=14)
    ax1.set_ylabel('Power', fontsize=14)
    ax2.set_ylabel('Power', fontsize=14)
    ax3.set_ylabel('Power', fontsize=14)
    ax4.set_ylabel('Power', fontsize=14)
    ax1.set_title('Depth', fontsize=14)
    ax2.set_title('Position', fontsize=14)
    ax3.set_title('Line Width', fontsize=14)
    ax4.set_title('Equivalent Width', fontsize=14)
    ax1.tick_params(axis='both', labelsize=14)
    ax2.tick_params(axis='both', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    ax1.axvline(oscillation_freq, color='r', label='Oscillation')
    ax2.axvline(oscillation_freq, color='r')
    ax3.axvline(oscillation_freq, color='r')
    ax4.axvline(oscillation_freq, color='r')
    ax1.axhline(FAP_depth[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax1.axhline(FAP_depth[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax1.axhline(FAP_depth[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax2.axhline(FAP_position[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax2.axhline(FAP_position[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax2.axhline(FAP_position[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax3.axhline(FAP_width[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax3.axhline(FAP_width[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax3.axhline(FAP_width[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax4.axhline(FAP_ew[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax4.axhline(FAP_ew[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax4.axhline(FAP_ew[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    if max(LS_depth[1])>FAP_depth[0]:
        max_power = np.where(LS_depth[1]==max(LS_depth[1]))
        ax1.axvline(LS_depth[0][max_power], color='darkblue', label='Maximum')
    if max(LS_position[1])>FAP_position[0]:
        max_power = np.where(LS_position[1]==max(LS_position[1]))
        ax2.axvline(LS_position[0][max_power], color='darkblue')
    if max(LS_width[1])>FAP_width[0]:
        max_power = np.where(LS_width[1]==max(LS_width[1]))
        ax3.axvline(LS_width[0][max_power], color='darkblue')
    if max(LS_ew[1])>FAP_ew[0]:
        max_power = np.where(LS_ew[1]==max(LS_ew[1]))
        ax4.axvline(LS_ew[0][max_power], color='darkblue')

    #Rassine
    theta_in_question = theta_rassine
    err_in_question = err_theta_rassine

    LS_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).autopower(nyquist_factor=1)
    LS_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).autopower(nyquist_factor=1)
    LS_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).autopower(nyquist_factor=1)
    LS_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).autopower(nyquist_factor=1)

    probas = [0.1, 0.01, 0.001]
    FAP_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  

    ax5.loglog(LS_depth[0], LS_depth[1])
    ax6.loglog(LS_position[0], LS_position[1])
    ax7.loglog(LS_width[0], LS_width[1])
    ax8.loglog(LS_ew[0], LS_ew[1])
    ax5.set_xlabel('Frequency (1/d)', fontsize=14)
    ax6.set_xlabel('Frequency (1/d)', fontsize=14)
    ax7.set_xlabel('Frequency (1/d)', fontsize=14)
    ax8.set_xlabel('Frequency (1/d)', fontsize=14)
    ax5.set_ylabel('Power', fontsize=14)
    ax6.set_ylabel('Power', fontsize=14)
    ax7.set_ylabel('Power', fontsize=14)
    ax8.set_ylabel('Power', fontsize=14)
    ax5.set_title('Depth', fontsize=14)
    ax6.set_title('Position', fontsize=14)
    ax7.set_title('Line Width', fontsize=14)
    ax8.set_title('Equivalent Width', fontsize=14)
    ax5.tick_params(axis='both', labelsize=14)
    ax6.tick_params(axis='both', labelsize=14)
    ax7.tick_params(axis='both', labelsize=14)
    ax8.tick_params(axis='both', labelsize=14)
    ax5.axvline(oscillation_freq, color='r', label='Oscillation')
    ax6.axvline(oscillation_freq, color='r')
    ax7.axvline(oscillation_freq, color='r')
    ax8.axvline(oscillation_freq, color='r')
    ax5.axhline(FAP_depth[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax5.axhline(FAP_depth[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax5.axhline(FAP_depth[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax6.axhline(FAP_position[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax6.axhline(FAP_position[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax6.axhline(FAP_position[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax7.axhline(FAP_width[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax7.axhline(FAP_width[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax7.axhline(FAP_width[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax8.axhline(FAP_ew[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax8.axhline(FAP_ew[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax8.axhline(FAP_ew[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    if max(LS_depth[1])>FAP_depth[0]:
        max_power = np.where(LS_depth[1]==max(LS_depth[1]))
        ax5.axvline(LS_depth[0][max_power], color='darkblue', label='Maximum')
    if max(LS_position[1])>FAP_position[0]:
        max_power = np.where(LS_position[1]==max(LS_position[1]))
        ax6.axvline(LS_position[0][max_power], color='darkblue')
    if max(LS_width[1])>FAP_width[0]:
        max_power = np.where(LS_width[1]==max(LS_width[1]))
        ax7.axvline(LS_width[0][max_power], color='darkblue')
    if max(LS_ew[1])>FAP_ew[0]:
        max_power = np.where(LS_ew[1]==max(LS_ew[1]))
        ax8.axvline(LS_ew[0][max_power], color='darkblue')

    ax1.annotate('ANTARESS', (-0.2, 0.5), xytext=(-45, 0), ha='right', va='center',
                size=20, rotation=90, xycoords='axes fraction',
                textcoords='offset points')
    ax5.annotate('RASSINE', (-0.2, 0.5), xytext=(-45, 0), ha='right', va='center',
                size=20, rotation=90, xycoords='axes fraction',
                textcoords='offset points')
    
    ax1.legend(fontsize=14)
    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Periodogram_He_fit.pdf')
    plt.show()


## Plotting the timeseries and periodogram of the Radial Velocity

In [ ]:
reload(AF)
save = True
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    AF.plot_TS_Periodo(date_HA, total_RV_HA, total_RV_err_HA, 'Radial Velocity (km/s)', 'RV', '(HA)', save)
    
    AF.plot_TS_Periodo(date_HE, total_RV_HE, total_RV_err_HE, 'Radial Velocity (km/s)', 'RV', '(HE)', save)
#If there is one mode of observation.
else:
    AF.plot_TS_Periodo(date, total_RV, total_RV_err, 'Radial Velocity (km/s)', 'RV', '', save)

## Plotting the timeseries and periodogram of the FWHM

In [ ]:
reload(AF)
save=True
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    AF.plot_TS_Periodo(date_HA, total_FWHM_HA, total_FWHM_err_HA,'FWHM (km/s)', 'FWHM', '(HA)', save)
    
    AF.plot_TS_Periodo(date_HE, total_FWHM_HE, total_FWHM_err_HE,'FWHM (km/s)', 'FWHM', '(HE)', save)
#If there is one mode of observation.
else:
    AF.plot_TS_Periodo(date, total_FWHM, total_FWHM_err, 'FWHM (km/s)', 'FWHM', '', save)

## Plotting the timeseries and periodogram of the Bisector Span

In [ ]:
reload(AF)
save=True
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    AF.plot_TS_Periodo(date_HA, total_BIS_SPAN_HA, total_BIS_SPAN_err_HA, 'Bisector Span (km/s)', 'Bisector Span', '(HA)', save)
    
    AF.plot_TS_Periodo(date_HE, total_BIS_SPAN_HE, total_BIS_SPAN_err_HE, 'Bisector Span (km/s)', 'Bisector Span', '(HE)', save)
#If there is one mode of observation.
else:
    AF.plot_TS_Periodo(date, total_BIS_SPAN, total_BIS_SPAN_err, 'Bisector Span (km/s)', 'Bisector Span', '', save)

## Plotting the timeseries and periodogram of H2O IWV

In [ ]:
reload(AF)
save = True
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    AF.plot_TS_Periodo(date_HA, total_H2O_HA, total_H2O_err_HA, 'H2O IWV', 'H2O', '(HA)', save)
    
    AF.plot_TS_Periodo(date_HE, total_H2O_HE, total_H2O_err_HE, 'H2O IWV', 'H2O', '(HE)', save)
#If there is one mode of observation.
else:
    AF.plot_TS_Periodo(date, total_H2O, total_H2O_err, 'H2O IWV', 'H2O', '', save)

## Plotting the timeseries and periodogram of O2 IWV

In [ ]:
reload(AF)
save = True
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    AF.plot_TS_Periodo(date_HA, total_O2_HA, total_O2_err_HA, 'O2 IWV', 'O2', '(HA)', save, fit=True, order=4)
    
    AF.plot_TS_Periodo(date_HE, total_O2_HE, total_O2_err_HE, 'O2 IWV', 'O2', '(HE)', save, fit=True, order=4)
#If there is one mode of observation.
else:
    AF.plot_TS_Periodo(date, total_O2, total_O2_err, 'O2 IWV', 'O2', '', save, fit=True, order=4)

## Plotting the timeseries and periodogram of CO2 IWV

In [ ]:
reload(AF)
save = True
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    AF.plot_TS_Periodo(date_HA, total_CO2_HA, total_CO2_err_HA, 'CO2 IWV', 'CO2', '(HA)', save, fit=True, order=4)
    
    AF.plot_TS_Periodo(date_HE, total_CO2_HE, total_CO2_err_HE, 'CO2 IWV', 'CO2', '(HE)', save, fit=True, order=4)
#If there is one mode of observation.
else:
    AF.plot_TS_Periodo(date, total_CO2, total_CO2_err, 'CO2 IWV', 'CO2', '', save, fit=True, order=4)

## Periodogram for the position, depth and width of the Si line (Voigt)

In [ ]:
save = False
oscillation_freq = 24*60/5.4 #in 1/days!!

depth_index = 0
position_index = 1
width_index = 2
ew_index = -1

#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

    #HA
    print('HA')
    #Antaress
    theta_in_question = theta_Si_Antaress_HA
    err_in_question = err_theta_Si_Antaress_HA

    fig, ((ax1, ax2, ax3, ax4), (ax5, ax6, ax7, ax8)) = plt.subplots (2, 4, figsize=[20, 10])

    LS_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).autopower(nyquist_factor=1)
    LS_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).autopower(nyquist_factor=1)
    LS_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).autopower(nyquist_factor=1)
    LS_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).autopower(nyquist_factor=1)

    probas = [0.1, 0.01, 0.001]
    FAP_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  

    ax1.loglog(LS_depth[0], LS_depth[1])
    ax2.loglog(LS_position[0], LS_position[1])
    ax3.loglog(LS_width[0], LS_width[1])
    ax4.loglog(LS_ew[0], LS_ew[1])
    ax1.set_xlabel('Frequency (1/d)', fontsize=14)
    ax2.set_xlabel('Frequency (1/d)', fontsize=14)
    ax3.set_xlabel('Frequency (1/d)', fontsize=14)
    ax4.set_xlabel('Frequency (1/d)', fontsize=14)
    ax1.set_ylabel('Power', fontsize=14)
    ax2.set_ylabel('Power', fontsize=14)
    ax3.set_ylabel('Power', fontsize=14)
    ax4.set_ylabel('Power', fontsize=14)
    ax1.set_title('Depth', fontsize=14)
    ax2.set_title('Position', fontsize=14)
    ax3.set_title('Line Width', fontsize=14)
    ax4.set_title('Equivalent Width', fontsize=14)
    ax1.tick_params(axis='both', labelsize=14)
    ax2.tick_params(axis='both', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    ax1.axvline(oscillation_freq, color='r', label='Oscillation')
    ax2.axvline(oscillation_freq, color='r')
    ax3.axvline(oscillation_freq, color='r')
    ax4.axvline(oscillation_freq, color='r')
    ax1.axhline(FAP_depth[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax1.axhline(FAP_depth[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax1.axhline(FAP_depth[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax2.axhline(FAP_position[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax2.axhline(FAP_position[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax2.axhline(FAP_position[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax3.axhline(FAP_width[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax3.axhline(FAP_width[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax3.axhline(FAP_width[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax4.axhline(FAP_ew[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax4.axhline(FAP_ew[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax4.axhline(FAP_ew[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    if max(LS_depth[1])>FAP_depth[0]:
        max_power = np.where(LS_depth[1]==max(LS_depth[1]))
        ax1.axvline(LS_depth[0][max_power], color='darkblue', label='Maximum')
    if max(LS_position[1])>FAP_position[0]:
        max_power = np.where(LS_position[1]==max(LS_position[1]))
        ax2.axvline(LS_position[0][max_power], color='darkblue')
    if max(LS_width[1])>FAP_width[0]:
        max_power = np.where(LS_width[1]==max(LS_width[1]))
        ax3.axvline(LS_width[0][max_power], color='darkblue')
    if max(LS_ew[1])>FAP_ew[0]:
        max_power = np.where(LS_ew[1]==max(LS_ew[1]))
        ax4.axvline(LS_ew[0][max_power], color='darkblue')

    #Rassine
    theta_in_question = theta_Si_rassine_HA
    err_in_question = err_theta_Si_rassine_HA

    LS_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).autopower(nyquist_factor=1)
    LS_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).autopower(nyquist_factor=1)
    LS_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).autopower(nyquist_factor=1)
    LS_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).autopower(nyquist_factor=1)

    probas = [0.1, 0.01, 0.001]
    FAP_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  

    ax5.loglog(LS_depth[0], LS_depth[1])
    ax6.loglog(LS_position[0], LS_position[1])
    ax7.loglog(LS_width[0], LS_width[1])
    ax8.loglog(LS_ew[0], LS_ew[1])
    ax5.set_xlabel('Frequency (1/d)', fontsize=14)
    ax6.set_xlabel('Frequency (1/d)', fontsize=14)
    ax7.set_xlabel('Frequency (1/d)', fontsize=14)
    ax8.set_xlabel('Frequency (1/d)', fontsize=14)
    ax5.set_ylabel('Power', fontsize=14)
    ax6.set_ylabel('Power', fontsize=14)
    ax7.set_ylabel('Power', fontsize=14)
    ax8.set_ylabel('Power', fontsize=14)
    ax5.set_title('Depth', fontsize=14)
    ax6.set_title('Position', fontsize=14)
    ax7.set_title('Line Width', fontsize=14)
    ax8.set_title('Equivalent Width', fontsize=14)
    ax5.tick_params(axis='both', labelsize=14)
    ax6.tick_params(axis='both', labelsize=14)
    ax7.tick_params(axis='both', labelsize=14)
    ax8.tick_params(axis='both', labelsize=14)
    ax5.axvline(oscillation_freq, color='r', label='Oscillation')
    ax6.axvline(oscillation_freq, color='r')
    ax7.axvline(oscillation_freq, color='r')
    ax8.axvline(oscillation_freq, color='r')
    ax5.axhline(FAP_depth[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax5.axhline(FAP_depth[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax5.axhline(FAP_depth[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax6.axhline(FAP_position[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax6.axhline(FAP_position[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax6.axhline(FAP_position[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax7.axhline(FAP_width[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax7.axhline(FAP_width[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax7.axhline(FAP_width[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax8.axhline(FAP_ew[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax8.axhline(FAP_ew[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax8.axhline(FAP_ew[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    if max(LS_depth[1])>FAP_depth[0]:
        max_power = np.where(LS_depth[1]==max(LS_depth[1]))
        ax5.axvline(LS_depth[0][max_power], color='darkblue', label='Maximum')
    if max(LS_position[1])>FAP_position[0]:
        max_power = np.where(LS_position[1]==max(LS_position[1]))
        ax6.axvline(LS_position[0][max_power], color='darkblue')
    if max(LS_width[1])>FAP_width[0]:
        max_power = np.where(LS_width[1]==max(LS_width[1]))
        ax7.axvline(LS_width[0][max_power], color='darkblue')
    if max(LS_ew[1])>FAP_ew[0]:
        max_power = np.where(LS_ew[1]==max(LS_ew[1]))
        ax8.axvline(LS_ew[0][max_power], color='darkblue')

    ax1.annotate('ANTARESS', (-0.2, 0.5), xytext=(-45, 0), ha='right', va='center',
                size=20, rotation=90, xycoords='axes fraction',
                textcoords='offset points')
    ax5.annotate('RASSINE', (-0.2, 0.5), xytext=(-45, 0), ha='right', va='center',
                size=20, rotation=90, xycoords='axes fraction',
                textcoords='offset points')
    
    ax1.legend(fontsize=14)
    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Periodogram_Si_Voigt_fit_HA.pdf')
    plt.show()

    #HE
    print('HE')
    #Antaress
    theta_in_question = theta_Si_Antaress_HE
    err_in_question = err_theta_Si_Antaress_HE

    fig, ((ax1, ax2, ax3, ax4), (ax5, ax6, ax7, ax8)) = plt.subplots (2, 4, figsize=[20, 10])

    LS_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).autopower(nyquist_factor=1)
    LS_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).autopower(nyquist_factor=1)
    LS_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).autopower(nyquist_factor=1)
    LS_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).autopower(nyquist_factor=1)

    probas = [0.1, 0.01, 0.001]
    FAP_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  

    ax1.loglog(LS_depth[0], LS_depth[1])
    ax2.loglog(LS_position[0], LS_position[1])
    ax3.loglog(LS_width[0], LS_width[1])
    ax4.loglog(LS_ew[0], LS_ew[1])
    ax1.set_xlabel('Frequency (1/d)', fontsize=14)
    ax2.set_xlabel('Frequency (1/d)', fontsize=14)
    ax3.set_xlabel('Frequency (1/d)', fontsize=14)
    ax4.set_xlabel('Frequency (1/d)', fontsize=14)
    ax1.set_ylabel('Power', fontsize=14)
    ax2.set_ylabel('Power', fontsize=14)
    ax3.set_ylabel('Power', fontsize=14)
    ax4.set_ylabel('Power', fontsize=14)
    ax1.set_title('Depth', fontsize=14)
    ax2.set_title('Position', fontsize=14)
    ax3.set_title('Line Width', fontsize=14)
    ax4.set_title('Equivalent Width', fontsize=14)
    ax1.tick_params(axis='both', labelsize=14)
    ax2.tick_params(axis='both', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    ax1.axvline(oscillation_freq, color='r', label='Oscillation')
    ax2.axvline(oscillation_freq, color='r')
    ax3.axvline(oscillation_freq, color='r')
    ax4.axvline(oscillation_freq, color='r')
    ax1.axhline(FAP_depth[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax1.axhline(FAP_depth[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax1.axhline(FAP_depth[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax2.axhline(FAP_position[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax2.axhline(FAP_position[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax2.axhline(FAP_position[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax3.axhline(FAP_width[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax3.axhline(FAP_width[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax3.axhline(FAP_width[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax4.axhline(FAP_ew[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax4.axhline(FAP_ew[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax4.axhline(FAP_ew[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    if max(LS_depth[1])>FAP_depth[0]:
        max_power = np.where(LS_depth[1]==max(LS_depth[1]))
        ax1.axvline(LS_depth[0][max_power], color='darkblue', label='Maximum')
    if max(LS_position[1])>FAP_position[0]:
        max_power = np.where(LS_position[1]==max(LS_position[1]))
        ax2.axvline(LS_position[0][max_power], color='darkblue')
    if max(LS_width[1])>FAP_width[0]:
        max_power = np.where(LS_width[1]==max(LS_width[1]))
        ax3.axvline(LS_width[0][max_power], color='darkblue')
    if max(LS_ew[1])>FAP_ew[0]:
        max_power = np.where(LS_ew[1]==max(LS_ew[1]))
        ax4.axvline(LS_ew[0][max_power], color='darkblue')

    #Rassine
    theta_in_question = theta_Si_rassine_HE
    err_in_question = err_theta_Si_rassine_HE

    LS_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).autopower(nyquist_factor=1)
    LS_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).autopower(nyquist_factor=1)
    LS_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).autopower(nyquist_factor=1)
    LS_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).autopower(nyquist_factor=1)

    probas = [0.1, 0.01, 0.001]
    FAP_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  

    ax5.loglog(LS_depth[0], LS_depth[1])
    ax6.loglog(LS_position[0], LS_position[1])
    ax7.loglog(LS_width[0], LS_width[1])
    ax8.loglog(LS_ew[0], LS_ew[1])
    ax5.set_xlabel('Frequency (1/d)', fontsize=14)
    ax6.set_xlabel('Frequency (1/d)', fontsize=14)
    ax7.set_xlabel('Frequency (1/d)', fontsize=14)
    ax8.set_xlabel('Frequency (1/d)', fontsize=14)
    ax5.set_ylabel('Power', fontsize=14)
    ax6.set_ylabel('Power', fontsize=14)
    ax7.set_ylabel('Power', fontsize=14)
    ax8.set_ylabel('Power', fontsize=14)
    ax5.set_title('Depth', fontsize=14)
    ax6.set_title('Position', fontsize=14)
    ax7.set_title('Line Width', fontsize=14)
    ax8.set_title('Equivalent Width', fontsize=14)
    ax5.tick_params(axis='both', labelsize=14)
    ax6.tick_params(axis='both', labelsize=14)
    ax7.tick_params(axis='both', labelsize=14)
    ax8.tick_params(axis='both', labelsize=14)
    ax5.axvline(oscillation_freq, color='r', label='Oscillation')
    ax6.axvline(oscillation_freq, color='r')
    ax7.axvline(oscillation_freq, color='r')
    ax8.axvline(oscillation_freq, color='r')
    ax5.axhline(FAP_depth[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax5.axhline(FAP_depth[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax5.axhline(FAP_depth[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax6.axhline(FAP_position[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax6.axhline(FAP_position[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax6.axhline(FAP_position[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax7.axhline(FAP_width[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax7.axhline(FAP_width[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax7.axhline(FAP_width[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax8.axhline(FAP_ew[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax8.axhline(FAP_ew[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax8.axhline(FAP_ew[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    if max(LS_depth[1])>FAP_depth[0]:
        max_power = np.where(LS_depth[1]==max(LS_depth[1]))
        ax5.axvline(LS_depth[0][max_power], color='darkblue', label='Maximum')
    if max(LS_position[1])>FAP_position[0]:
        max_power = np.where(LS_position[1]==max(LS_position[1]))
        ax6.axvline(LS_position[0][max_power], color='darkblue')
    if max(LS_width[1])>FAP_width[0]:
        max_power = np.where(LS_width[1]==max(LS_width[1]))
        ax7.axvline(LS_width[0][max_power], color='darkblue')
    if max(LS_ew[1])>FAP_ew[0]:
        max_power = np.where(LS_ew[1]==max(LS_ew[1]))
        ax8.axvline(LS_ew[0][max_power], color='darkblue')

    ax1.annotate('ANTARESS', (-0.2, 0.5), xytext=(-45, 0), ha='right', va='center',
                size=20, rotation=90, xycoords='axes fraction',
                textcoords='offset points')
    ax5.annotate('RASSINE', (-0.2, 0.5), xytext=(-45, 0), ha='right', va='center',
                size=20, rotation=90, xycoords='axes fraction',
                textcoords='offset points')
    
    ax1.legend(fontsize=14)
    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Periodogram_Si_Voigt_fit_HE.pdf')
    plt.show()
    
#If there is one mode of observation.    
else:    
    #Antaress
    theta_in_question = theta_Si_Antaress
    err_in_question = err_theta_Si_Antaress

    fig, ((ax1, ax2, ax3, ax4), (ax5, ax6, ax7, ax8)) = plt.subplots (2, 4, figsize=[20, 10])

    LS_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).autopower(nyquist_factor=1)
    LS_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).autopower(nyquist_factor=1)
    LS_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).autopower(nyquist_factor=1)
    LS_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).autopower(nyquist_factor=1)

    probas = [0.1, 0.01, 0.001]
    FAP_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  

    ax1.loglog(LS_depth[0], LS_depth[1])
    ax2.loglog(LS_position[0], LS_position[1])
    ax3.loglog(LS_width[0], LS_width[1])
    ax4.loglog(LS_ew[0], LS_ew[1])
    ax1.set_xlabel('Frequency (1/d)', fontsize=14)
    ax2.set_xlabel('Frequency (1/d)', fontsize=14)
    ax3.set_xlabel('Frequency (1/d)', fontsize=14)
    ax4.set_xlabel('Frequency (1/d)', fontsize=14)
    ax1.set_ylabel('Power', fontsize=14)
    ax2.set_ylabel('Power', fontsize=14)
    ax3.set_ylabel('Power', fontsize=14)
    ax4.set_ylabel('Power', fontsize=14)
    ax1.set_title('Depth', fontsize=14)
    ax2.set_title('Position', fontsize=14)
    ax3.set_title('Line Width', fontsize=14)
    ax4.set_title('Equivalent Width', fontsize=14)
    ax1.tick_params(axis='both', labelsize=14)
    ax2.tick_params(axis='both', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    ax1.axvline(oscillation_freq, color='r', label='Oscillation')
    ax2.axvline(oscillation_freq, color='r')
    ax3.axvline(oscillation_freq, color='r')
    ax4.axvline(oscillation_freq, color='r')
    ax1.axhline(FAP_depth[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax1.axhline(FAP_depth[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax1.axhline(FAP_depth[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax2.axhline(FAP_position[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax2.axhline(FAP_position[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax2.axhline(FAP_position[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax3.axhline(FAP_width[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax3.axhline(FAP_width[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax3.axhline(FAP_width[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax4.axhline(FAP_ew[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax4.axhline(FAP_ew[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax4.axhline(FAP_ew[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    if max(LS_depth[1])>FAP_depth[0]:
        max_power = np.where(LS_depth[1]==max(LS_depth[1]))
        ax1.axvline(LS_depth[0][max_power], color='darkblue', label='Maximum')
    if max(LS_position[1])>FAP_position[0]:
        max_power = np.where(LS_position[1]==max(LS_position[1]))
        ax2.axvline(LS_position[0][max_power], color='darkblue')
    if max(LS_width[1])>FAP_width[0]:
        max_power = np.where(LS_width[1]==max(LS_width[1]))
        ax3.axvline(LS_width[0][max_power], color='darkblue')
    if max(LS_ew[1])>FAP_ew[0]:
        max_power = np.where(LS_ew[1]==max(LS_ew[1]))
        ax4.axvline(LS_ew[0][max_power], color='darkblue')

    #Rassine
    theta_in_question = theta_Si_rassine
    err_in_question = err_theta_Si_rassine

    LS_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).autopower(nyquist_factor=1)
    LS_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).autopower(nyquist_factor=1)
    LS_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).autopower(nyquist_factor=1)
    LS_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).autopower(nyquist_factor=1)

    probas = [0.1, 0.01, 0.001]
    FAP_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  

    ax5.loglog(LS_depth[0], LS_depth[1])
    ax6.loglog(LS_position[0], LS_position[1])
    ax7.loglog(LS_width[0], LS_width[1])
    ax8.loglog(LS_ew[0], LS_ew[1])
    ax5.set_xlabel('Frequency (1/d)', fontsize=14)
    ax6.set_xlabel('Frequency (1/d)', fontsize=14)
    ax7.set_xlabel('Frequency (1/d)', fontsize=14)
    ax8.set_xlabel('Frequency (1/d)', fontsize=14)
    ax5.set_ylabel('Power', fontsize=14)
    ax6.set_ylabel('Power', fontsize=14)
    ax7.set_ylabel('Power', fontsize=14)
    ax8.set_ylabel('Power', fontsize=14)
    ax5.set_title('Depth', fontsize=14)
    ax6.set_title('Position', fontsize=14)
    ax7.set_title('Line Width', fontsize=14)
    ax8.set_title('Equivalent Width', fontsize=14)
    ax5.tick_params(axis='both', labelsize=14)
    ax6.tick_params(axis='both', labelsize=14)
    ax7.tick_params(axis='both', labelsize=14)
    ax8.tick_params(axis='both', labelsize=14)
    ax5.axvline(oscillation_freq, color='r', label='Oscillation')
    ax6.axvline(oscillation_freq, color='r')
    ax7.axvline(oscillation_freq, color='r')
    ax8.axvline(oscillation_freq, color='r')
    ax5.axhline(FAP_depth[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax5.axhline(FAP_depth[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax5.axhline(FAP_depth[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax6.axhline(FAP_position[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax6.axhline(FAP_position[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax6.axhline(FAP_position[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax7.axhline(FAP_width[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax7.axhline(FAP_width[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax7.axhline(FAP_width[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax8.axhline(FAP_ew[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax8.axhline(FAP_ew[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax8.axhline(FAP_ew[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    if max(LS_depth[1])>FAP_depth[0]:
        max_power = np.where(LS_depth[1]==max(LS_depth[1]))
        ax5.axvline(LS_depth[0][max_power], color='darkblue', label='Maximum')
    if max(LS_position[1])>FAP_position[0]:
        max_power = np.where(LS_position[1]==max(LS_position[1]))
        ax6.axvline(LS_position[0][max_power], color='darkblue')
    if max(LS_width[1])>FAP_width[0]:
        max_power = np.where(LS_width[1]==max(LS_width[1]))
        ax7.axvline(LS_width[0][max_power], color='darkblue')
    if max(LS_ew[1])>FAP_ew[0]:
        max_power = np.where(LS_ew[1]==max(LS_ew[1]))
        ax8.axvline(LS_ew[0][max_power], color='darkblue')

    ax1.annotate('ANTARESS', (-0.2, 0.5), xytext=(-45, 0), ha='right', va='center',
                size=20, rotation=90, xycoords='axes fraction',
                textcoords='offset points')
    ax5.annotate('RASSINE', (-0.2, 0.5), xytext=(-45, 0), ha='right', va='center',
                size=20, rotation=90, xycoords='axes fraction',
                textcoords='offset points')
    
    ax1.legend(fontsize=14)
    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Periodogram_Si_Voigt_fit.pdf')
    plt.show()


## Periodogram for the position, depth and width of the Si line (Gaussian)

In [ ]:
save = False
oscillation_freq = 24*60/5.4 #in 1/days!!

depth_index = 0
position_index = 1
width_index = 2
ew_index = -1

#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

    #HA
    print('HA')
    #Antaress
    theta_in_question = thetaG_Si_Antaress_HA
    err_in_question = err_thetaG_Si_Antaress_HA

    fig, ((ax1, ax2, ax3, ax4), (ax5, ax6, ax7, ax8)) = plt.subplots (2, 4, figsize=[20, 10])

    LS_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).autopower(nyquist_factor=1)
    LS_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).autopower(nyquist_factor=1)
    LS_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).autopower(nyquist_factor=1)
    LS_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).autopower(nyquist_factor=1)

    probas = [0.1, 0.01, 0.001]
    FAP_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  

    ax1.loglog(LS_depth[0], LS_depth[1])
    ax2.loglog(LS_position[0], LS_position[1])
    ax3.loglog(LS_width[0], LS_width[1])
    ax4.loglog(LS_ew[0], LS_ew[1])
    ax1.set_xlabel('Frequency (1/d)', fontsize=14)
    ax2.set_xlabel('Frequency (1/d)', fontsize=14)
    ax3.set_xlabel('Frequency (1/d)', fontsize=14)
    ax4.set_xlabel('Frequency (1/d)', fontsize=14)
    ax1.set_ylabel('Power', fontsize=14)
    ax2.set_ylabel('Power', fontsize=14)
    ax3.set_ylabel('Power', fontsize=14)
    ax4.set_ylabel('Power', fontsize=14)
    ax1.set_title('Depth', fontsize=14)
    ax2.set_title('Position', fontsize=14)
    ax3.set_title('Line Width', fontsize=14)
    ax4.set_title('Equivalent Width', fontsize=14)
    ax1.tick_params(axis='both', labelsize=14)
    ax2.tick_params(axis='both', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    ax1.axvline(oscillation_freq, color='r', label='Oscillation')
    ax2.axvline(oscillation_freq, color='r')
    ax3.axvline(oscillation_freq, color='r')
    ax4.axvline(oscillation_freq, color='r')
    ax1.axhline(FAP_depth[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax1.axhline(FAP_depth[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax1.axhline(FAP_depth[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax2.axhline(FAP_position[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax2.axhline(FAP_position[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax2.axhline(FAP_position[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax3.axhline(FAP_width[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax3.axhline(FAP_width[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax3.axhline(FAP_width[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax4.axhline(FAP_ew[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax4.axhline(FAP_ew[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax4.axhline(FAP_ew[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    if max(LS_depth[1])>FAP_depth[0]:
        max_power = np.where(LS_depth[1]==max(LS_depth[1]))
        ax1.axvline(LS_depth[0][max_power], color='darkblue', label='Maximum')
    if max(LS_position[1])>FAP_position[0]:
        max_power = np.where(LS_position[1]==max(LS_position[1]))
        ax2.axvline(LS_position[0][max_power], color='darkblue')
    if max(LS_width[1])>FAP_width[0]:
        max_power = np.where(LS_width[1]==max(LS_width[1]))
        ax3.axvline(LS_width[0][max_power], color='darkblue')
    if max(LS_ew[1])>FAP_ew[0]:
        max_power = np.where(LS_ew[1]==max(LS_ew[1]))
        ax4.axvline(LS_ew[0][max_power], color='darkblue')

    #Rassine
    theta_in_question = thetaG_Si_rassine_HA
    err_in_question = err_thetaG_Si_rassine_HA

    LS_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).autopower(nyquist_factor=1)
    LS_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).autopower(nyquist_factor=1)
    LS_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).autopower(nyquist_factor=1)
    LS_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).autopower(nyquist_factor=1)

    probas = [0.1, 0.01, 0.001]
    FAP_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  

    ax5.loglog(LS_depth[0], LS_depth[1])
    ax6.loglog(LS_position[0], LS_position[1])
    ax7.loglog(LS_width[0], LS_width[1])
    ax8.loglog(LS_ew[0], LS_ew[1])
    ax5.set_xlabel('Frequency (1/d)', fontsize=14)
    ax6.set_xlabel('Frequency (1/d)', fontsize=14)
    ax7.set_xlabel('Frequency (1/d)', fontsize=14)
    ax8.set_xlabel('Frequency (1/d)', fontsize=14)
    ax5.set_ylabel('Power', fontsize=14)
    ax6.set_ylabel('Power', fontsize=14)
    ax7.set_ylabel('Power', fontsize=14)
    ax8.set_ylabel('Power', fontsize=14)
    ax5.set_title('Depth', fontsize=14)
    ax6.set_title('Position', fontsize=14)
    ax7.set_title('Line Width', fontsize=14)
    ax8.set_title('Equivalent Width', fontsize=14)
    ax5.tick_params(axis='both', labelsize=14)
    ax6.tick_params(axis='both', labelsize=14)
    ax7.tick_params(axis='both', labelsize=14)
    ax8.tick_params(axis='both', labelsize=14)
    ax5.axvline(oscillation_freq, color='r', label='Oscillation')
    ax6.axvline(oscillation_freq, color='r')
    ax7.axvline(oscillation_freq, color='r')
    ax8.axvline(oscillation_freq, color='r')
    ax5.axhline(FAP_depth[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax5.axhline(FAP_depth[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax5.axhline(FAP_depth[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax6.axhline(FAP_position[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax6.axhline(FAP_position[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax6.axhline(FAP_position[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax7.axhline(FAP_width[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax7.axhline(FAP_width[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax7.axhline(FAP_width[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax8.axhline(FAP_ew[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax8.axhline(FAP_ew[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax8.axhline(FAP_ew[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    if max(LS_depth[1])>FAP_depth[0]:
        max_power = np.where(LS_depth[1]==max(LS_depth[1]))
        ax5.axvline(LS_depth[0][max_power], color='darkblue', label='Maximum')
    if max(LS_position[1])>FAP_position[0]:
        max_power = np.where(LS_position[1]==max(LS_position[1]))
        ax6.axvline(LS_position[0][max_power], color='darkblue')
    if max(LS_width[1])>FAP_width[0]:
        max_power = np.where(LS_width[1]==max(LS_width[1]))
        ax7.axvline(LS_width[0][max_power], color='darkblue')
    if max(LS_ew[1])>FAP_ew[0]:
        max_power = np.where(LS_ew[1]==max(LS_ew[1]))
        ax8.axvline(LS_ew[0][max_power], color='darkblue')

    ax1.annotate('ANTARESS', (-0.2, 0.5), xytext=(-45, 0), ha='right', va='center',
                size=20, rotation=90, xycoords='axes fraction',
                textcoords='offset points')
    ax5.annotate('RASSINE', (-0.2, 0.5), xytext=(-45, 0), ha='right', va='center',
                size=20, rotation=90, xycoords='axes fraction',
                textcoords='offset points')
    
    ax1.legend(fontsize=14)
    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Periodogram_Si_Gauss_fit_HA.pdf')
    plt.show()

    #HE
    print('HE')
    #Antaress
    theta_in_question = thetaG_Si_Antaress_HE
    err_in_question = err_thetaG_Si_Antaress_HE

    fig, ((ax1, ax2, ax3, ax4), (ax5, ax6, ax7, ax8)) = plt.subplots (2, 4, figsize=[20, 10])

    LS_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).autopower(nyquist_factor=1)
    LS_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).autopower(nyquist_factor=1)
    LS_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).autopower(nyquist_factor=1)
    LS_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).autopower(nyquist_factor=1)

    probas = [0.1, 0.01, 0.001]
    FAP_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  

    ax1.loglog(LS_depth[0], LS_depth[1])
    ax2.loglog(LS_position[0], LS_position[1])
    ax3.loglog(LS_width[0], LS_width[1])
    ax4.loglog(LS_ew[0], LS_ew[1])
    ax1.set_xlabel('Frequency (1/d)', fontsize=14)
    ax2.set_xlabel('Frequency (1/d)', fontsize=14)
    ax3.set_xlabel('Frequency (1/d)', fontsize=14)
    ax4.set_xlabel('Frequency (1/d)', fontsize=14)
    ax1.set_ylabel('Power', fontsize=14)
    ax2.set_ylabel('Power', fontsize=14)
    ax3.set_ylabel('Power', fontsize=14)
    ax4.set_ylabel('Power', fontsize=14)
    ax1.set_title('Depth', fontsize=14)
    ax2.set_title('Position', fontsize=14)
    ax3.set_title('Line Width', fontsize=14)
    ax4.set_title('Equivalent Width', fontsize=14)
    ax1.tick_params(axis='both', labelsize=14)
    ax2.tick_params(axis='both', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    ax1.axvline(oscillation_freq, color='r', label='Oscillation')
    ax2.axvline(oscillation_freq, color='r')
    ax3.axvline(oscillation_freq, color='r')
    ax4.axvline(oscillation_freq, color='r')
    ax1.axhline(FAP_depth[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax1.axhline(FAP_depth[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax1.axhline(FAP_depth[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax2.axhline(FAP_position[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax2.axhline(FAP_position[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax2.axhline(FAP_position[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax3.axhline(FAP_width[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax3.axhline(FAP_width[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax3.axhline(FAP_width[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax4.axhline(FAP_ew[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax4.axhline(FAP_ew[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax4.axhline(FAP_ew[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    if max(LS_depth[1])>FAP_depth[0]:
        max_power = np.where(LS_depth[1]==max(LS_depth[1]))
        ax1.axvline(LS_depth[0][max_power], color='darkblue', label='Maximum')
    if max(LS_position[1])>FAP_position[0]:
        max_power = np.where(LS_position[1]==max(LS_position[1]))
        ax2.axvline(LS_position[0][max_power], color='darkblue')
    if max(LS_width[1])>FAP_width[0]:
        max_power = np.where(LS_width[1]==max(LS_width[1]))
        ax3.axvline(LS_width[0][max_power], color='darkblue')
    if max(LS_ew[1])>FAP_ew[0]:
        max_power = np.where(LS_ew[1]==max(LS_ew[1]))
        ax4.axvline(LS_ew[0][max_power], color='darkblue')

    #Rassine
    theta_in_question = thetaG_Si_rassine_HE
    err_in_question = err_thetaG_Si_rassine_HE

    LS_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).autopower(nyquist_factor=1)
    LS_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).autopower(nyquist_factor=1)
    LS_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).autopower(nyquist_factor=1)
    LS_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).autopower(nyquist_factor=1)

    probas = [0.1, 0.01, 0.001]
    FAP_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  

    ax5.loglog(LS_depth[0], LS_depth[1])
    ax6.loglog(LS_position[0], LS_position[1])
    ax7.loglog(LS_width[0], LS_width[1])
    ax8.loglog(LS_ew[0], LS_ew[1])
    ax5.set_xlabel('Frequency (1/d)', fontsize=14)
    ax6.set_xlabel('Frequency (1/d)', fontsize=14)
    ax7.set_xlabel('Frequency (1/d)', fontsize=14)
    ax8.set_xlabel('Frequency (1/d)', fontsize=14)
    ax5.set_ylabel('Power', fontsize=14)
    ax6.set_ylabel('Power', fontsize=14)
    ax7.set_ylabel('Power', fontsize=14)
    ax8.set_ylabel('Power', fontsize=14)
    ax5.set_title('Depth', fontsize=14)
    ax6.set_title('Position', fontsize=14)
    ax7.set_title('Line Width', fontsize=14)
    ax8.set_title('Equivalent Width', fontsize=14)
    ax5.tick_params(axis='both', labelsize=14)
    ax6.tick_params(axis='both', labelsize=14)
    ax7.tick_params(axis='both', labelsize=14)
    ax8.tick_params(axis='both', labelsize=14)
    ax5.axvline(oscillation_freq, color='r', label='Oscillation')
    ax6.axvline(oscillation_freq, color='r')
    ax7.axvline(oscillation_freq, color='r')
    ax8.axvline(oscillation_freq, color='r')
    ax5.axhline(FAP_depth[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax5.axhline(FAP_depth[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax5.axhline(FAP_depth[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax6.axhline(FAP_position[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax6.axhline(FAP_position[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax6.axhline(FAP_position[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax7.axhline(FAP_width[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax7.axhline(FAP_width[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax7.axhline(FAP_width[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax8.axhline(FAP_ew[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax8.axhline(FAP_ew[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax8.axhline(FAP_ew[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    if max(LS_depth[1])>FAP_depth[0]:
        max_power = np.where(LS_depth[1]==max(LS_depth[1]))
        ax5.axvline(LS_depth[0][max_power], color='darkblue', label='Maximum')
    if max(LS_position[1])>FAP_position[0]:
        max_power = np.where(LS_position[1]==max(LS_position[1]))
        ax6.axvline(LS_position[0][max_power], color='darkblue')
    if max(LS_width[1])>FAP_width[0]:
        max_power = np.where(LS_width[1]==max(LS_width[1]))
        ax7.axvline(LS_width[0][max_power], color='darkblue')
    if max(LS_ew[1])>FAP_ew[0]:
        max_power = np.where(LS_ew[1]==max(LS_ew[1]))
        ax8.axvline(LS_ew[0][max_power], color='darkblue')

    ax1.annotate('ANTARESS', (-0.2, 0.5), xytext=(-45, 0), ha='right', va='center',
                size=20, rotation=90, xycoords='axes fraction',
                textcoords='offset points')
    ax5.annotate('RASSINE', (-0.2, 0.5), xytext=(-45, 0), ha='right', va='center',
                size=20, rotation=90, xycoords='axes fraction',
                textcoords='offset points')
    
    ax1.legend(fontsize=14)
    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Periodogram_Si_Gauss_fit_HE.pdf')
    plt.show()
    
#If there is one mode of observation.    
else:    
    #Antaress
    theta_in_question = thetaG_Si_Antaress
    err_in_question = err_thetaG_Si_Antaress

    fig, ((ax1, ax2, ax3, ax4), (ax5, ax6, ax7, ax8)) = plt.subplots (2, 4, figsize=[20, 10])

    LS_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).autopower(nyquist_factor=1)
    LS_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).autopower(nyquist_factor=1)
    LS_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).autopower(nyquist_factor=1)
    LS_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).autopower(nyquist_factor=1)

    probas = [0.1, 0.01, 0.001]
    FAP_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  

    ax1.loglog(LS_depth[0], LS_depth[1])
    ax2.loglog(LS_position[0], LS_position[1])
    ax3.loglog(LS_width[0], LS_width[1])
    ax4.loglog(LS_ew[0], LS_ew[1])
    ax1.set_xlabel('Frequency (1/d)', fontsize=14)
    ax2.set_xlabel('Frequency (1/d)', fontsize=14)
    ax3.set_xlabel('Frequency (1/d)', fontsize=14)
    ax4.set_xlabel('Frequency (1/d)', fontsize=14)
    ax1.set_ylabel('Power', fontsize=14)
    ax2.set_ylabel('Power', fontsize=14)
    ax3.set_ylabel('Power', fontsize=14)
    ax4.set_ylabel('Power', fontsize=14)
    ax1.set_title('Depth', fontsize=14)
    ax2.set_title('Position', fontsize=14)
    ax3.set_title('Line Width', fontsize=14)
    ax4.set_title('Equivalent Width', fontsize=14)
    ax1.tick_params(axis='both', labelsize=14)
    ax2.tick_params(axis='both', labelsize=14)
    ax3.tick_params(axis='both', labelsize=14)
    ax4.tick_params(axis='both', labelsize=14)
    ax1.axvline(oscillation_freq, color='r', label='Oscillation')
    ax2.axvline(oscillation_freq, color='r')
    ax3.axvline(oscillation_freq, color='r')
    ax4.axvline(oscillation_freq, color='r')
    ax1.axhline(FAP_depth[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax1.axhline(FAP_depth[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax1.axhline(FAP_depth[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax2.axhline(FAP_position[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax2.axhline(FAP_position[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax2.axhline(FAP_position[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax3.axhline(FAP_width[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax3.axhline(FAP_width[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax3.axhline(FAP_width[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax4.axhline(FAP_ew[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax4.axhline(FAP_ew[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax4.axhline(FAP_ew[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    if max(LS_depth[1])>FAP_depth[0]:
        max_power = np.where(LS_depth[1]==max(LS_depth[1]))
        ax1.axvline(LS_depth[0][max_power], color='darkblue', label='Maximum')
    if max(LS_position[1])>FAP_position[0]:
        max_power = np.where(LS_position[1]==max(LS_position[1]))
        ax2.axvline(LS_position[0][max_power], color='darkblue')
    if max(LS_width[1])>FAP_width[0]:
        max_power = np.where(LS_width[1]==max(LS_width[1]))
        ax3.axvline(LS_width[0][max_power], color='darkblue')
    if max(LS_ew[1])>FAP_ew[0]:
        max_power = np.where(LS_ew[1]==max(LS_ew[1]))
        ax4.axvline(LS_ew[0][max_power], color='darkblue')

    #Rassine
    theta_in_question = thetaG_Si_rassine
    err_in_question = err_thetaG_Si_rassine

    LS_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).autopower(nyquist_factor=1)
    LS_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).autopower(nyquist_factor=1)
    LS_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).autopower(nyquist_factor=1)
    LS_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).autopower(nyquist_factor=1)

    probas = [0.1, 0.01, 0.001]
    FAP_depth = LombScargle(date, theta_in_question[:, depth_index], err_in_question[:, depth_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_position = LombScargle(date, theta_in_question[:, position_index], err_in_question[:, position_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_width = LombScargle(date, theta_in_question[:, width_index], err_in_question[:, width_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  
    FAP_ew = LombScargle(date, theta_in_question[:, ew_index], err_in_question[:, ew_index]).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  

    ax5.loglog(LS_depth[0], LS_depth[1])
    ax6.loglog(LS_position[0], LS_position[1])
    ax7.loglog(LS_width[0], LS_width[1])
    ax8.loglog(LS_ew[0], LS_ew[1])
    ax5.set_xlabel('Frequency (1/d)', fontsize=14)
    ax6.set_xlabel('Frequency (1/d)', fontsize=14)
    ax7.set_xlabel('Frequency (1/d)', fontsize=14)
    ax8.set_xlabel('Frequency (1/d)', fontsize=14)
    ax5.set_ylabel('Power', fontsize=14)
    ax6.set_ylabel('Power', fontsize=14)
    ax7.set_ylabel('Power', fontsize=14)
    ax8.set_ylabel('Power', fontsize=14)
    ax5.set_title('Depth', fontsize=14)
    ax6.set_title('Position', fontsize=14)
    ax7.set_title('Line Width', fontsize=14)
    ax8.set_title('Equivalent Width', fontsize=14)
    ax5.tick_params(axis='both', labelsize=14)
    ax6.tick_params(axis='both', labelsize=14)
    ax7.tick_params(axis='both', labelsize=14)
    ax8.tick_params(axis='both', labelsize=14)
    ax5.axvline(oscillation_freq, color='r', label='Oscillation')
    ax6.axvline(oscillation_freq, color='r')
    ax7.axvline(oscillation_freq, color='r')
    ax8.axvline(oscillation_freq, color='r')
    ax5.axhline(FAP_depth[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax5.axhline(FAP_depth[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax5.axhline(FAP_depth[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax6.axhline(FAP_position[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax6.axhline(FAP_position[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax6.axhline(FAP_position[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax7.axhline(FAP_width[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax7.axhline(FAP_width[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax7.axhline(FAP_width[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    ax8.axhline(FAP_ew[0], color='k', label='F.A.P. = '+str(int(100*probas[0]))+'%')
    ax8.axhline(FAP_ew[1], color='k', linestyle='-.', label='F.A.P. = '+str(int(100*probas[1]))+'%')
    ax8.axhline(FAP_ew[2], color='k', linestyle='--', label='F.A.P. = '+str(100*probas[2])+'%')
    if max(LS_depth[1])>FAP_depth[0]:
        max_power = np.where(LS_depth[1]==max(LS_depth[1]))
        ax5.axvline(LS_depth[0][max_power], color='darkblue', label='Maximum')
    if max(LS_position[1])>FAP_position[0]:
        max_power = np.where(LS_position[1]==max(LS_position[1]))
        ax6.axvline(LS_position[0][max_power], color='darkblue')
    if max(LS_width[1])>FAP_width[0]:
        max_power = np.where(LS_width[1]==max(LS_width[1]))
        ax7.axvline(LS_width[0][max_power], color='darkblue')
    if max(LS_ew[1])>FAP_ew[0]:
        max_power = np.where(LS_ew[1]==max(LS_ew[1]))
        ax8.axvline(LS_ew[0][max_power], color='darkblue')

    ax1.annotate('ANTARESS', (-0.2, 0.5), xytext=(-45, 0), ha='right', va='center',
                size=20, rotation=90, xycoords='axes fraction',
                textcoords='offset points')
    ax5.annotate('RASSINE', (-0.2, 0.5), xytext=(-45, 0), ha='right', va='center',
                size=20, rotation=90, xycoords='axes fraction',
                textcoords='offset points')
    
    ax1.legend(fontsize=14)
    fig.tight_layout()
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Periodogram_Si_Gauss_fit.pdf')
    plt.show()


## Plotting the telluric spectra and the diff between the TC and not TC spectra

In [ ]:
reload(AF)
save = False

He_fit_region = (10831.5, 10834.1)
Si_fit_region = (10828, 10832)
#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    #HA
    print('HA')
    fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=[15, 8])
    for i in range(int(len(total_lamda_HA)/5)):
        if i==0:
            ax1.plot(tota_lamda_HA[i], tota_telluric_HA[i], 'r', label='Telluric')
        else:
            ax1.plot(tota_lamda_HA[i], tota_telluric_HA[i], 'r')
        #Antaress
        ax1.errorbar(total_lamda_HA[i], total_norm_spctr_Antaress_HA[i], yerr=total_norm_err_Antaress_HA[i])
        ax2.plot(tota_lamda_HA[i], tota_telluric_HA[i], 'r')
        #Rassine
        ax2.errorbar(rassine_total_lamda_HA[i], rassine_total_norm_spctr_HA[i], yerr=rassine_total_norm_err_HA[i])
            
    ax2.set_xlabel('Wavelength ($\AA$)', fontsize=14)
    ax1.set_ylabel('Normalized Flux - Antaress', fontsize=14)
    ax2.set_ylabel('Normalized Flux - Rassine', fontsize=14)
    ax1.set_title('Telluric Spectrum')
    ax1.set_xlim([10825, 10840])
    ax1.set_ylim(0.6, 1.05)
    ax2.set_ylim(0.6, 1.05)
    ax1.axvline(10833.2, color='g', label='He triplet location')
    ax1.axvline(10833.3, color='g')
    ax1.axvline(10832.1, color='g')
    ax1.axvline(10833.3145, color='b', linestyle='--', label='$\mathrm{H}_2\mathrm{O}$ lines location')
    ax1.axvline(10834.59165, color='b', linestyle='--')
    ax1.axvline(10835.068, color='b', linestyle='--')
    ax1.axvline(10836.94, color='b', linestyle='--')
    ax2.axvline(10833.2, color='g', label='He triplet location')
    ax2.axvline(10833.3, color='g')
    ax2.axvline(10832.1, color='g')
    ax2.axvline(10833.3145, color='b', linestyle='--', label='$\mathrm{H}_2\mathrm{O}$ lines location')
    ax2.axvline(10834.59165, color='b', linestyle='--')
    ax2.axvline(10835.068, color='b', linestyle='--')
    ax2.axvline(10836.94, color='b', linestyle='--')
    ax1.tick_params(axis='both', labelsize=14)
    ax2.tick_params(axis='both', labelsize=14)
    ax1.axvspan(He_fit_region[0], He_fit_region[1], color='k', alpha=0.2, label='He fit range')
    ax2.axvspan(He_fit_region[0], He_fit_region[1], color='k', alpha=0.2, label='He fit range')
    ax1.axvspan(Si_fit_region[0], Si_fit_region[1], color='orange', alpha=0.2, label='Si fit range')
    ax2.axvspan(Si_fit_region[0], Si_fit_region[1], color='orange', alpha=0.2, label='Si fit range')
    
    ax1.legend(fontsize=14)
    
    fig.subplots_adjust(hspace=0)
    
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Telluric_Spectrum_HA.pdf')
    plt.show()

    
    #HE
    print('HE')
    fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=[15, 8])
    for i in range(int(len(total_lamda_HE)/5)):
        if i==0:
            ax1.plot(tota_lamda_HE[i], tota_telluric_HE[i], 'r', label='Telluric')
        else:
            ax1.plot(tota_lamda_HE[i], tota_telluric_HE[i], 'r')
        #Antaress
        ax1.errorbar(total_lamda_HE[i], total_norm_spctr_Antaress_HE[i], yerr=total_norm_err_Antaress_HE[i])
        ax2.plot(tota_lamda_HE[i], tota_telluric_HE[i], 'r')
        #Rassine
        ax2.errorbar(rassine_total_lamda_HE[i], rassine_total_norm_spctr_HE[i], yerr=rassine_total_norm_err_HE[i])
            
    ax2.set_xlabel('Wavelength ($\AA$)', fontsize=14)
    ax1.set_ylabel('Normalized Flux - Antaress', fontsize=14)
    ax2.set_ylabel('Normalized Flux - Rassine', fontsize=14)
    ax1.set_title('Telluric Spectrum')
    ax1.set_xlim([10825, 10840])
    ax1.set_ylim(0.6, 1.05)
    ax2.set_ylim(0.6, 1.05)
    ax1.axvline(10833.2, color='g', label='He triplet location')
    ax1.axvline(10833.3, color='g')
    ax1.axvline(10832.1, color='g')
    ax1.axvline(10833.3145, color='b', linestyle='--', label='$\mathrm{H}_2\mathrm{O}$ lines location')
    ax1.axvline(10834.59165, color='b', linestyle='--')
    ax1.axvline(10835.068, color='b', linestyle='--')
    ax1.axvline(10836.94, color='b', linestyle='--')
    ax2.axvline(10833.2, color='g', label='He triplet location')
    ax2.axvline(10833.3, color='g')
    ax2.axvline(10832.1, color='g')
    ax2.axvline(10833.3145, color='b', linestyle='--', label='$\mathrm{H}_2\mathrm{O}$ lines location')
    ax2.axvline(10834.59165, color='b', linestyle='--')
    ax2.axvline(10835.068, color='b', linestyle='--')
    ax2.axvline(10836.94, color='b', linestyle='--')
    ax1.tick_params(axis='both', labelsize=14)
    ax2.tick_params(axis='both', labelsize=14)
    ax1.axvspan(He_fit_region[0], He_fit_region[1], color='k', alpha=0.2, label='He fit range')
    ax2.axvspan(He_fit_region[0], He_fit_region[1], color='k', alpha=0.2, label='He fit range')
    ax1.axvspan(Si_fit_region[0], Si_fit_region[1], color='orange', alpha=0.2, label='Si fit range')
    ax2.axvspan(Si_fit_region[0], Si_fit_region[1], color='orange', alpha=0.2, label='Si fit range')
    
    ax1.legend(fontsize=14)
    
    fig.subplots_adjust(hspace=0)
    
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Telluric_Spectrum_HE.pdf')
    plt.show()

#If there is one mode of observation
else:
    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=[15, 8])
    for i in range(int(len(total_lamda)/5)):
        if i==0:
            ax1.plot(tota_lamda[i], tota_telluric[i], 'r', label='Telluric')
        else:
            ax1.plot(tota_lamda[i], tota_telluric[i], 'r')
        #Antaress
        ax1.errorbar(total_lamda[i], total_norm_spctr_Antaress[i], yerr=total_norm_err_Antaress[i])
        ax2.plot(tota_lamda[i], tota_telluric[i], 'r')
        #Rassine
        ax2.errorbar(rassine_total_lamda[i], rassine_total_norm_spctr[i], yerr=rassine_total_norm_err[i])
            
    ax2.set_xlabel('Wavelength ($\AA$)', fontsize=14)
    ax1.set_ylabel('Normalized Flux - Antaress', fontsize=14)
    ax2.set_ylabel('Normalized Flux - Rassine', fontsize=14)
    ax1.set_title('Telluric Spectrum')
    ax1.set_xlim([10825, 10840])
    ax1.set_ylim(0.6, 1.05)
    ax2.set_ylim(0.6, 1.05)
    ax1.axvline(10833.2, color='g', label='He triplet location')
    ax1.axvline(10833.3, color='g')
    ax1.axvline(10832.1, color='g')
    ax1.axvline(10833.3145, color='b', linestyle='--', label='$\mathrm{H}_2\mathrm{O}$ lines location')
    ax1.axvline(10834.59165, color='b', linestyle='--')
    ax1.axvline(10835.068, color='b', linestyle='--')
    ax1.axvline(10836.94, color='b', linestyle='--')
    ax2.axvline(10833.2, color='g', label='He triplet location')
    ax2.axvline(10833.3, color='g')
    ax2.axvline(10832.1, color='g')
    ax2.axvline(10833.3145, color='b', linestyle='--', label='$\mathrm{H}_2\mathrm{O}$ lines location')
    ax2.axvline(10834.59165, color='b', linestyle='--')
    ax2.axvline(10835.068, color='b', linestyle='--')
    ax2.axvline(10836.94, color='b', linestyle='--')
    ax1.tick_params(axis='both', labelsize=14)
    ax2.tick_params(axis='both', labelsize=14)
    ax1.axvspan(He_fit_region[0], He_fit_region[1], color='k', alpha=0.2, label='He fit range')
    ax2.axvspan(He_fit_region[0], He_fit_region[1], color='k', alpha=0.2, label='He fit range')
    ax1.axvspan(Si_fit_region[0], Si_fit_region[1], color='orange', alpha=0.2, label='Si fit range')
    ax2.axvspan(Si_fit_region[0], Si_fit_region[1], color='orange', alpha=0.2, label='Si fit range')
    
    ax1.legend(fontsize=14)
    
    fig.subplots_adjust(hspace=0)
    if save:
        plt.savefig('/Users/samsonmercier/Downloads/Telluric_Spectrum.pdf')
    plt.show()


## Plotting the time series of the air mass and fitting a poynomial 

In [ ]:
reload(AF)
save = True
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    AF.plot_TS_Periodo(date_HA, total_AIRM_HA, [], 'Air Mass', 'Air Mass', '(HA)', save, fit=True, order=4)
    
    AF.plot_TS_Periodo(date_HE, total_AIRM_HE, [], 'Air Mass', 'Air Mass', '(HE)', save, fit=True, order=4)
#If there is one mode of observation.
else:
    AF.plot_TS_Periodo(date, total_AIRM, [], 'Air Mass', 'Air Mass', '', save, fit=True, order=4)

## Detrending the air mass from the CO2 IWV

In [ ]:
reload(AF)
save = True
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    AF.plot_TS_Periodo(date_HA, total_CO2_HA/total_AIRM_HA, total_CO2_err_HA/total_AIRM_HA, 'CO2 IWV', 'Detrended CO2', '(HA)', save)
    
    AF.plot_TS_Periodo(date_HE, total_CO2_HE/total_AIRM_HE, total_CO2_err_HE/total_AIRM_HE, 'Air Mass', 'Detrended CO2', '(HE)', save)
#If there is one mode of observation.
else:
    AF.plot_TS_Periodo(date, total_CO2/total_AIRM, total_CO2_err/total_AIRM, 'CO2 IWV', 'Detrended CO2', '', save)
    

## Detrending the air mass from the H2O IWV

In [ ]:
reload(AF)
save = True
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    AF.plot_TS_Periodo(date_HA, total_H2O_HA/total_AIRM_HA, total_H2O_err_HA/total_AIRM_HA, 'H2O IWV', 'Detrended H2O', '(HA)', save)
    
    AF.plot_TS_Periodo(date_HE, total_H2O_HE/total_AIRM_HE, total_H2O_err_HE/total_AIRM_HE, 'Air Mass', 'Detrended H2O', '(HE)', save)
#If there is one mode of observation.
else:
    AF.plot_TS_Periodo(date, total_H2O/total_AIRM, total_H2O_err/total_AIRM, 'H2O IWV', 'Detrended H2O', '', save)

# Random analyses of bad data

In [ ]:
'''reload(AF)

low = 10767
up = 10770
cap = 5
cap2 = 2

new_RV = tot_RV[tot_AIRM<cap2]
new_FWHM = tot_FWHM[tot_AIRM<cap2]
new_BIS_SPAN = tot_BIS_SPAN[tot_AIRM<cap2]
new_CONTRAST = tot_CONTRAST[tot_AIRM<cap2]
new_mode = tot_mode[tot_AIRM<cap2]

new_RV_err = tot_RV_err[tot_AIRM<cap2]
new_FWHM_err = tot_FWHM_err[tot_AIRM<cap2]
new_BIS_SPAN_err = tot_BIS_SPAN_err[tot_AIRM<cap2]
new_CONTRAST_err = tot_CONTRAST_err[tot_AIRM<cap2]


new_SNR = AF.SNR_calculator(low, up, tot_lamda[tot_AIRM<cap2], tot_spctr[tot_AIRM<cap2], tot_err[tot_AIRM<cap2])

#Correlation between SNR and RV
AF.Correlation_Plot(new_mode[new_SNR>cap], new_SNR[new_SNR>cap], new_RV[new_SNR>cap], [], new_RV_err[new_SNR>cap], 'SNR', 'RV (km/s)', 'SNR-RV', day)

#Correlation between SNR and FWHM
AF.Correlation_Plot(new_mode[new_SNR>cap], new_SNR[new_SNR>cap], new_FWHM[new_SNR>cap], [], new_FWHM_err[new_SNR>cap], 'SNR', 'FWHM (km/s)', 'SNR-FWHM', day)

#Correlation between SNR and BIS SPAN
AF.Correlation_Plot(new_mode[new_SNR>cap], new_SNR[new_SNR>cap], new_BIS_SPAN[new_SNR>cap], [], new_BIS_SPAN_err[new_SNR>cap], 'SNR', 'Bisector Span (km/s)', 'SNR-Bisector Span', day)

#Correlation between SNR and CONTRAST
AF.Correlation_Plot(new_mode[new_SNR>cap], new_SNR[new_SNR>cap], new_CONTRAST[new_SNR>cap], [], new_CONTRAST_err[new_SNR>cap], 'SNR', 'Contrast (%)', 'SNR-Contrast', day)

'''

# Making color maps

In [ ]:
extra = mt.floor(min(date))

master_lamda, master_norm_spctr, master_norm_err

bound_up_spctr = 10839
bound_down_spctr = 10825

# Create the 2D plot with colormap
fig, ((ax1, ax2, ax5), (ax3, ax4, ax6)) = plt.subplots(2, 3, sharey=True, figsize=[20, 10], gridspec_kw={'width_ratios': [6, 6, 1]})

lim_up = 1.04
lim_down = 0.4

im1 = ax1.imshow(rassine_total_norm_spctr, cmap='jet', aspect='auto', vmin = lim_down, vmax = lim_up, extent=[rassine_total_lamda[0][0], rassine_total_lamda[0][-1], date[0]-extra, date[-1]-extra])
im1 = ax2.imshow(total_norm_spctr, cmap='jet', aspect='auto', vmin = lim_down, vmax = lim_up, extent=[total_lamda[0][0], total_lamda[0][-1], date[0]-extra, date[-1]-extra])

# Set labels and title
ax1.set_ylabel('Time (MJD -'+str(extra)+')', fontsize=14)
ax1.set_title('Rassine Spectra Colormap', fontsize=14)
ax1.set_xlim([bound_down_spctr, bound_up_spctr])

ax2.set_title('Spectra Colormap', fontsize=14)
ax2.set_xlim([bound_down_spctr, bound_up_spctr])

#Set a colorbar 
ax5.set_visible(False)
fig.colorbar(im1, label='Normalized Flux', ax=ax5)



lim_up = 0.01
lim_down = -0.01
im2=ax3.imshow(rassine_total_norm_spctr - rassine_master_spctr, aspect='auto', vmin = lim_down , vmax = lim_up, extent=[rassine_total_lamda[0][0], rassine_total_lamda[0][-1], date[0]-extra, date[-1]-extra])
im2=ax4.imshow(total_norm_spctr - master_norm_spctr, aspect='auto', vmin=lim_down, vmax=lim_up, extent=[total_lamda[0][0], total_lamda[0][-1], date[0]-extra, date[-1]-extra])
# Set labels and title
ax3.set_ylabel('Time (MJD -'+str(extra)+')', fontsize=14)
ax3.set_xlabel('Wavelength ($\AA$)', fontsize=14)
ax3.set_title('Rassine Residual Colormap', fontsize=14)
ax3.set_xlim([bound_down_spctr, bound_up_spctr])

ax4.set_xlabel('Wavelength ($\AA$)', fontsize=14)
ax4.set_title('Residual Colormap', fontsize=14)
ax4.set_xlim([bound_down_spctr, bound_up_spctr])

# Add a colorbar
ax6.set_visible(False)
fig.colorbar(im2, label='Normalized Flux', ax=ax6)

ax1.tick_params(axis='both', labelsize=14)
ax2.tick_params(axis='both', labelsize=14)
ax3.tick_params(axis='both', labelsize=14)
ax4.tick_params(axis='both', labelsize=14)

ax1.sharex(ax3)
ax2.sharex(ax4)
# Show the plot
fig.tight_layout()
plt.show()

# Re-normalizing the spectra with Rassine

## Storing the input in .csv files

In [ ]:
header = ['wave', 'flux']

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    #Create folder
    if not os.path.exists('Rassine-data'):
        os.mkdir('Rassine-data')

    #Fill up the folders
    #HE
    if len(os.listdir('Rassine-data'))==0:
        for i in range(28):
            with open('Rassine-data/rassine_'+str(i)+'.csv', 'w') as f:
                writer = csv.writer(f)

                writer.writerow(header)

                for j in range(len(total_spctr_HE[i])):

                    writer.writerow([total_lamda_HE[i][j], total_spctr_HE[i][j]])

    #HA
        for i in range(119):
            with open('Rassine-data/rassine_'+str(28+i)+'.csv', 'w') as f:
                writer = csv.writer(f)

                writer.writerow(header)

                for j in range(len(total_spctr_HA[i])):

                    writer.writerow([total_lamda_HA[i][j], total_spctr_HA[i][j]])
    
    #HE    
        for i in range(28, 59):
            with open('Rassine-data/rassine_'+str(119+i)+'.csv', 'w') as f:
                writer = csv.writer(f)

                writer.writerow(header)

                for j in range(len(total_spctr_HE[i])):

                    writer.writerow([total_lamda_HE[i][j], total_spctr_HE[i][j]])

    #HA
        for i in range(119, len(total_spctr_HA)):
            with open('Rassine-data/rassine_'+str(len(total_spctr_HE)+i)+'.csv', 'w') as f:
                writer = csv.writer(f)

                writer.writerow(header)

                for j in range(len(total_spctr_HA[i])):

                    writer.writerow([total_lamda_HA[i][j], total_spctr_HA[i][j]])
    
else:
    
    #Create folders
    if not os.path.exists('Rassine-data'):
        os.mkdir('Rassine-data')
    
    #Fill up the folders
    if len(os.listdir('Rassine-data'))==0:
        for i in range(len(total_spctr)):
            with open('Rassine-data/rassine_'+str(i)+'.csv', 'w') as f:
                writer = csv.writer(f)

                writer.writerow(header)

                for j in range(len(total_spctr[i])):

                    writer.writerow([total_lamda[i][j], total_spctr[i][j]])

In [ ]:
header = ['wave', 'flux', 'flux_std']

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    
    #Create folders
    if not os.path.exists('Rassine-data-HA'):
        os.mkdir('Rassine-data-HA')
    
    if not os.path.exists('Rassine-data-HE'):
        os.mkdir('Rassine-data-HE')

    #Fill up the folders
    #HA
    if len(os.listdir('Rassine-data-HA'))==0:
        for i in range(len(total_spctr_HA)):
            with open('Rassine-data-HA/rassine_'+str(i)+'.csv', 'w') as f:
                writer = csv.writer(f)

                writer.writerow(header)

                for j in range(len(total_spctr_HA[i])):

                    writer.writerow([total_lamda_HA[i][j], total_spctr_HA[i][j]])

    #HE
    if len(os.listdir('Rassine-data-HE'))==0:
        for i in range(len(total_spctr_HE)):
            with open('Rassine-data-HE/rassine_'+str(i)+'.csv', 'w') as f:
                writer = csv.writer(f)

                writer.writerow(header)

                for j in range(len(total_spctr_HE[i])):

                    writer.writerow([total_lamda_HE[i][j], total_spctr_HE[i][j]])

else:
    
    #Create folders
    if not os.path.exists('Rassine-data-std'):
        os.mkdir('Rassine-data-std')
    
    #Fill up the folders
    if len(os.listdir('Rassine-data-std'))==0:
        for i in range(len(total_spctr)):
            with open('Rassine-data-std/rassine_'+str(i)+'.csv', 'w') as f:
                writer = csv.writer(f)

                writer.writerow(header)

                for j in range(len(total_spctr[i])):

                    writer.writerow([total_lamda[i][j], total_spctr[i][j], total_err[i][j]])

## Opening the output of Rassine

In [ ]:
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    #Extract the normalized master spectrum
    for name in os.listdir('Rassine-data/MASTER/'):
        if name[:7] == 'RASSINE':
            obj = pd.read_pickle(r'Rassine-data/MASTER/'+name)
            norm_master_spctr = obj['flux']/obj['matching_diff']['continuum_linear']
            master_wav = obj['wave']
    
    #Extract the Rassine normalized spectra - HA
    rassine_total_norm_spctr_HA = np.zeros((len(total_spctr_HA), len(master_wav)))
    rassine_total_lamda_HA = np.zeros((len(total_spctr_HA), len(master_wav)))
    for i in range(len(total_spctr_HA)):
        obj = pd.read_pickle(r'Rassine-data/STACKED/RASSINE_prepared_rassine_'+str(i)+'_HA.p')
        rassine_total_lamda_HA[i] = obj['wave']
        rassine_total_norm_spctr_HA[i] = obj['flux']/obj['matching_diff']['continuum_linear']

    #Extract the Rassine normalized spectra - HE
    rassine_total_norm_spctr_HE = np.zeros((len(total_spctr_HE), len(master_wav)))
    rassine_total_lamda_HE = np.zeros((len(total_spctr_HE), len(master_wav)))
    for i in range(len(total_spctr_HE)):
        obj = pd.read_pickle(r'Rassine-data/STACKED/RASSINE_prepared_rassine_'+str(i)+'_HE.p')
        rassine_total_lamda_HE[i] = obj['wave']
        rassine_total_norm_spctr_HE[i] = obj['flux']/obj['matching_diff']['continuum_linear']

else:
    print('Here')
    #Extract the normalized master spectrum
    for name in os.listdir('Rassine-data-std/MASTER/'):
        if name[:7] == 'RASSINE':
            obj = pd.read_pickle(r'Rassine-data-std/MASTER/'+name)
            print(obj.keys())
            norm_master_spctr = obj['flux']/obj['matching_diff']['continuum_linear']
            master_wav = obj['wave']

    #Extract the Rassine normalized spectra
    rassine_total_norm_spctr = np.zeros((len(total_spctr), len(master_wav)))
    rassine_total_lamda = np.zeros((len(total_spctr), len(master_wav)))
    for i in range(1):#len(total_spctr)):
        obj = pd.read_pickle(r'Rassine-data-std/STACKED/RASSINE_prepared_rassine_'+str(i)+'.p')
        print(master_wav.shape)
        print(total_lamda[i].shape)
        print(obj.keys())
        rassine_total_lamda[i] = obj['wave']
        rassine_total_norm_spctr[i] = obj['flux']/obj['matching_diff']['continuum_linear']

## Making a 2D map of the TC spectra and telluric spectra for that day

In [ ]:
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    #Telluric-Corrected Spectra
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6))
    fig.suptitle('Colormap Plot of Rassine Normalized TC Spectra')
    #For HA
    for i in range(len(total_norm_spctr_HA)):
        ax1.scatter(rassine_total_lamda_HA[i], [i]*len(rassine_total_lamda_HA[i]), c=rassine_total_norm_spctr_HA[i], cmap='jet', s=5)
       
    #For HE
    for i in range(len(total_norm_spctr_HE)):
        ax2.scatter(rassine_total_lamda_HE[i], [i]*len(rassine_total_lamda_HE[i]), c=rassine_total_norm_spctr_HE[i], cmap='jet', s=5)

    #Getting the yticks - HA
    tick_dates_HA = []
    tick_positions_HA = []
    for i in range(len(date_HA)):
        if i%40 == 0 and i!=0:
            tick_dates_HA.append("{:.3f}".format(date_HA[i]))
            tick_positions_HA.append(i)

    #Getting the yticks - HE
    tick_dates_HE = []
    tick_positions_HE = []
    for i in range(len(date_HE)):
        if i%40 == 0 and i!=0:
            tick_dates_HE.append("{:.3f}".format(date_HE[i]))
            tick_positions_HE.append(i)

    # Add labels and title
    ax1.set_yticks(tick_positions_HA, tick_dates_HA)
    ax1.set_xlabel('Wavelength ($\AA$)')
    ax1.set_ylabel('Time (MJD)')
    ax1.set_title('HA')
    ax1.set_xlim([10825, 10835])

    ax2.set_yticks(tick_positions_HE, tick_dates_HE)
    ax1.set_xlabel('Wavelength ($\AA$)')
    ax2.set_ylabel('Time (MJD)')
    ax2.set_title('HE')
    ax2.set_xlim([10825, 10835])

    # Add a colorbar
    fig.colorbar(label='Normalized Flux')

    # Show the plot
    plt.show()
    
    #Telluric Spectra
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6))
    fig.suptitle('Colormap Plot of Normalized Telluric Spectra')
    #For HA
    for i in range(len(total_telluric_HA)):
        ax1.scatter(total_lamda_HA[i], [i]*len(total_lamda_HA[i]), c=total_telluric_HA[i], cmap='jet', s=5)

    #For HE
    for i in range(len(total_telluric_HE)):
        ax2.scatter(total_lamda_HE[i], [i]*len(total_lamda_HE[i]), c=total_telluric_HE[i], cmap='jet', s=5)

    # Add labels and title
    ax1.set_yticks(tick_positions_HA, tick_dates_HA)
    ax1.set_xlabel('Wavelength ($\AA$)')
    ax1.set_ylabel('Time (MJD)')
    ax1.set_title('HA')
    ax1.set_xlim([10829, 10834])

    ax2.set_yticks(tick_positions_HE, tick_dates_HE)
    ax1.set_xlabel('Wavelength ($\AA$)')
    ax2.set_ylabel('Time (MJD)')
    ax2.set_title('HE')
    ax2.set_xlim([10829, 10834])

    # Add a colorbar
    fig.colorbar(label='Normalized Flux')

    # Show the plot
    plt.show()


else:
    #Telluric-Corrected Spectra
    plt.figure(figsize=(10, 6))
    for i in range(len(rassine_total_norm_spctr)):
        plt.scatter(rassine_total_lamda[1], [i]*len(rassine_total_lamda[i]), c=rassine_total_norm_spctr[i], cmap='jet', s=5)

    #Getting the yticks
    tick_dates = []
    tick_positions = []
    for i in range(len(date)):
        if i%40 == 0 and i!=0:
            tick_dates.append("{:.3f}".format(date[i]))
            tick_positions.append(i)

    # Add labels and title
    plt.yticks(tick_positions, tick_dates)
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Time (MJD)')
    plt.title('Colormap of Rassine Normalized TC Spectra')
    plt.xlim([10825, 10835])

    # Add a colorbar
    plt.colorbar(label='Normalized Flux')

    # Show the plot
    plt.show()

    #Residual Telluric-Corrected Spectra
    plt.figure(figsize=(10, 6))
    for i in range(len(total_norm_spctr)):
        plt.plot(rassine_total_lamda[i], [i]*len(rassine_total_lamda[i]), c=rassine_total_norm_spctr[i]-norm_master_spctr, cmap='jet', s=5)

    #Getting the yticks
    tick_dates = []
    tick_positions = []
    for i in range(len(date)):
        if i%40 == 0 and i!=0:
            tick_dates.append("{:.3f}".format(date[i]))
            tick_positions.append(i)

    # Add labels and title
    plt.yticks(tick_positions, tick_dates)
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Time (MJD)')
    plt.title('Residual Colormap of Rassine Normalized TC Spectra')
    plt.xlim([10825, 10835])

    # Add a colorbar
    plt.colorbar(label='Normalized Flux')

    # Show the plot
    plt.show()


In [ ]:
import math as mt 

extra = mt.floor(min(date))

bound_up_spctr = 10839
bound_down_spctr = 10825
# Create the 2D plot with colormap
plt.figure(figsize=[10, 8])

test_max = np.zeros(len(rassine_total_norm_spctr))
test_min = np.zeros(len(rassine_total_norm_spctr))
for i in range(len(rassine_total_norm_spctr)):
    test_max[i] = max(AF.bound(bound_down_spctr, bound_up_spctr, rassine_total_lamda[0], rassine_total_norm_spctr[i]))
    test_min[i] = min(AF.bound(bound_down_spctr, bound_up_spctr, rassine_total_lamda[0], rassine_total_norm_spctr[i]))

lim_up = max(test_max)
lim_down = min(test_min)

plt.imshow(rassine_total_norm_spctr, cmap='jet', aspect='auto', vmin = lim_down, vmax = lim_up, extent=[rassine_total_lamda[0][0], rassine_total_lamda[0][-1], date[0]-extra, date[-1]-extra])

# Set labels and title
plt.ylabel('Time (MJD -'+str(extra)+')')
plt.xlabel('Wavelength ($\AA$)')
plt.title('Spectra Colormap')
plt.xlim([bound_down_spctr, bound_up_spctr])
#plt.axvline(10829, color='b')
#plt.axvline(10828.3, color='b')
# Add a colorbar
plt.colorbar(label='Normalized Flux')

# Show the plot
plt.show()


# Create the 2D plot with colormap
plt.figure(figsize=[10, 8])

test_max = np.zeros(len(rassine_total_norm_spctr))
test_min = np.zeros(len(rassine_total_norm_spctr))
for i in range(len(rassine_total_norm_spctr)):
    test_max[i] = max(AF.bound(bound_down_spctr, bound_up_spctr, rassine_total_lamda[0], rassine_total_norm_spctr[i]-norm_master_spctr))
    test_min[i] = min(AF.bound(bound_down_spctr, bound_up_spctr, rassine_total_lamda[0], rassine_total_norm_spctr[i]-norm_master_spctr))

lim_up = 0.01
lim_down = -0.01
plt.imshow(rassine_total_norm_spctr - norm_master_spctr, cmap='jet', aspect='auto', vmin = lim_down , vmax = lim_up, extent=[rassine_total_lamda[0][0], rassine_total_lamda[0][-1], date[0]-extra, date[-1]-extra])

# Set labels and title
plt.ylabel('Time (MJD -'+str(extra)+')')
plt.xlabel('Wavelength ($\AA$)')
plt.title('Residual Colormap')
plt.xlim([bound_down_spctr, bound_up_spctr])
# Add a colorbar
plt.colorbar(label='Normalized Flux')

# Show the plot
plt.show()


In [ ]:
wav_10826 = np.zeros(len(rassine_total_lamda))
for i in range(len(rassine_total_lamda)):
    wav_10826[i] = rassine_total_norm_spctr[i][1737]

plt.figure(figsize=[8, 6])
plt.plot(date-extra, wav_10826, '.')
plt.xlabel('Time (MJD -'+str(extra)+')', fontsize=12)
plt.ylabel('Flux', fontsize=12)
plt.tick_params(axis='both', labelsize=12)
plt.title('Distribution of flux values at 10826 $\AA$', fontsize=12)
plt.show()

plt.figure(figsize=[8, 6])
plt.plot(total_SNR, wav_10826, '.')
plt.xlabel('SNR', fontsize=12)
plt.ylabel('Flux', fontsize=12)
plt.tick_params(axis='both', labelsize=12)
plt.title('Correlation between SNR and flux values at 10826 $\AA$', fontsize=12)
plt.show()


# Re-make correlation plot between He triplet and Si with the newly normalized spectra

## He triplet

In [ ]:
reload(AF)
lower_lim = 10831.5
upper_lim = 10834.1
guess_params = [-0.06, -100, 10833.3, 0.1, 0.008]
bounds_param = ((-np.inf, -np.inf, 10833.2, -np.inf, -np.inf),(0, np.inf, 10833.4, np.inf, np.inf)) 

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

    new_theta_HA, new_err_theta_HA = AF.fit_spctr_line(AF.gaussian_L, lower_lim, upper_lim, guess_params, 
                                               bounds_param, rassine_total_lamda_HA, rassine_total_norm_spctr_HA, [], 'r')

    new_theta_HE, new_err_theta_HE = AF.fit_spctr_line(AF.gaussian_L, lower_lim, upper_lim, guess_params, 
                                               bounds_param, rassine_total_lamda_HE, rassine_total_norm_spctr_HE, [], 'k')
#If there is one mode of observation
else:
    new_theta, new_err_theta = AF.fit_spctr_line(AF.gaussian_L, lower_lim, upper_lim, guess_params, 
                                               bounds_param, rassine_total_lamda, rassine_total_norm_spctr, [], 'r')

    new_theta[:, 0] = np.abs(new_theta[:, 0])

## Si line

In [ ]:
reload(AF)

lower_l = 10828
upper_l = 10832
guess_para = [0.3, 0.6, 10830, 0.1, 0.2]
bound = ((-np.inf, -np.inf, -np.inf, -np.inf, -np.inf), (np.inf, np.inf, np.inf, np.inf, np.inf))

#Distinguish two cases depending on the number of modes of observation
#If there are two modes of observation
if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):

    new_theta_Si_HA, new_err_theta_Si_HA = AF.fit_spctr_line(AF.Voigt, lower_l, upper_l, guess_para, bound, 
                  rassine_total_lamda_HA, rassine_total_norm_spctr_HA, [], 'r')
    
    new_theta_Si_HE, new_err_theta_Si_HE = AF.fit_spctr_line(AF.Voigt, lower_l, upper_l, guess_para, bound, 
                  rassine_total_lamda_HE, rassine_total_norm_spctr_HE, [], 'k')

#If there is one mode of observation
else:
    
    new_theta_Si, new_err_theta_Si = AF.fit_spctr_line(AF.Voigt, lower_l, upper_l, guess_para, bound, 
                  rassine_total_lamda, rassine_total_norm_spctr, [], 'r')

## Correlation

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=[17, 8])
                                                                        
ref_He = AF.air2vac((10830.33977+10830.25010)/2)
ref_Si = AF.air2vac(10827.091) 

axes[0, 0].errorbar(theta[:, 0], theta_Si[:, 0], xerr = err_theta[:, 0],  yerr=err_theta_Si[:, 0], fmt='.', alpha=0.15)
axes[0, 0].plot(theta[:, 0], theta_Si[:, 0], 'b.')
axes[0, 1].errorbar((theta[:, 2]-ref_He)*3e8/ref_He, (theta_Si[:, 2]-ref_Si)*3e8/ref_Si, xerr = err_theta[:, 2]*3e8/ref_He,  yerr=err_theta_Si[:, 2]*3e8/ref_Si, fmt='.', alpha=0.15)
axes[0, 1].plot((theta[:, 2]-ref_He)*3e8/ref_He, (theta_Si[:, 2]-ref_Si)*3e8/ref_Si, 'b.')
axes[0, 2].errorbar(theta[:, 3], theta_Si[:, 4], xerr = err_theta[:, 3],  yerr=err_theta_Si[:, 4], fmt='.', alpha=0.15)
axes[0, 2].plot(theta[:, 3], theta_Si[:, 4], 'b.')
axes[0, 3].plot(theta[:, -1], theta_Si[:, -1], '.')

axes[1, 0].errorbar(new_theta[:, 0], new_theta_Si[:, 0], xerr = new_err_theta[:, 0],  yerr=new_err_theta_Si[:, 0], fmt='g.', linestyle='-', alpha=0.15)
axes[1, 0].plot(new_theta[:, 0], new_theta_Si[:, 0], 'g.')
axes[1, 1].errorbar((new_theta[:, 2]-ref_He)*3e8/ref_He, (new_theta_Si[:, 2]-ref_Si)*3e8/ref_Si, xerr = new_err_theta[:, 2]*3e8/ref_He,  yerr=err_theta_Si[:, 2]*3e8/ref_Si, fmt='g.', alpha=0.15)
axes[1, 1].plot((new_theta[:, 2]-ref_He)*3e8/ref_He, (new_theta_Si[:, 2]-ref_Si)*3e8/ref_Si, 'g.')
axes[1, 2].errorbar(new_theta[:, 3], new_theta_Si[:, 4], xerr = new_err_theta[:, 3],  yerr=new_err_theta_Si[:, 4], fmt='g.', alpha=0.15)
axes[1, 2].plot(new_theta[:, 3], new_theta_Si[:, 4], 'g.')
axes[1, 3].plot(new_theta[:, -1], new_theta_Si[:, -1], 'g.')

axes[0, 0].set_xlabel('He depth (flux)', fontsize=14)
axes[0, 1].set_xlabel('He position (m/s)', fontsize=14)
axes[0, 2].set_xlabel('He line width ($\AA$)', fontsize=14)
axes[0, 3].set_xlabel('He equivalent width ($\AA$)', fontsize=14)

axes[1, 0].set_xlabel('He depth (flux)', fontsize=14)
axes[1, 1].set_xlabel('He position (m/s)', fontsize=14)
axes[1, 2].set_xlabel('He line width ($\AA$)', fontsize=14)
axes[1, 3].set_xlabel('He equivalent width ($\AA$)', fontsize=14)

axes[0, 0].set_ylabel('Si depth (flux)', fontsize=14)
axes[0, 1].set_ylabel('Si position (m/s)', fontsize=14)
axes[0, 2].set_ylabel('Si line width ($\AA$)', fontsize=14)
axes[0, 3].set_ylabel('Si equivalent width ($\AA$)', fontsize=14)

axes[1, 0].set_ylabel('Si depth (flux)', fontsize=14)
axes[1, 1].set_ylabel('Si position (m/s)', fontsize=14)
axes[1, 2].set_ylabel('Si line width ($\AA$)', fontsize=14)
axes[1, 3].set_ylabel('Si equivalent width ($\AA$)', fontsize=14)

axes[0, 0].tick_params(axis='both', labelsize=14)
axes[0, 1].tick_params(axis='both', labelsize=14)
axes[0, 2].tick_params(axis='both', labelsize=14)
axes[0, 3].tick_params(axis='both', labelsize=14)

axes[1, 0].tick_params(axis='both', labelsize=14)
axes[1, 1].tick_params(axis='both', labelsize=14)
axes[1, 2].tick_params(axis='both', labelsize=14)
axes[1, 3].tick_params(axis='both', labelsize=14)



A=theta[:, 0]
B=theta_Si[:, 0]
textstr1 = '\n'.join((r"$r_P = %.3f$" % (np.corrcoef(A, B)[0][1], ), 
                     r"$r_S = %.3f$" % (ss.spearmanr(A, B).correlation, )))
axes[0, 0].tick_params(axis='both', labelsize=14)
axes[0, 0].text(0.5, 0.78, textstr1, transform=axes[0, 0].transAxes, fontsize=12, bbox = dict(facecolor='white', alpha=0.7))

A=(theta[:, 2]-ref_He)*3e8/ref_He
B=(theta_Si[:, 2]-ref_Si)*3e8/ref_Si
textstr2 = '\n'.join((r"$r_P = %.3f$" % (np.corrcoef(A, B)[0][1], ), 
                     r"$r_S = %.3f$" % (ss.spearmanr(A, B).correlation, )))
axes[0, 1].tick_params(axis='both', labelsize=14)
axes[0, 1].text(0.5, 0.78, textstr2, transform=axes[0, 1].transAxes, fontsize=12, bbox = dict(facecolor='white', alpha=0.7))

A=theta[:, 3]
B=theta_Si[:, 4]
textstr3 = '\n'.join((r"$r_P = %.3f$" % (np.corrcoef(A, B)[0][1], ), 
                     r"$r_S = %.3f$" % (ss.spearmanr(A, B).correlation, )))
axes[0, 2].tick_params(axis='both', labelsize=14)
axes[0, 2].text(0.5, 0.78, textstr3, transform=axes[0, 2].transAxes, fontsize=12, bbox = dict(facecolor='white', alpha=0.7))

A=theta[:, -1]
B=theta_Si[:, -1]
textstr4 = '\n'.join((r"$r_P = %.3f$" % (np.corrcoef(A, B)[0][1], ), 
                     r"$r_S = %.3f$" % (ss.spearmanr(A, B).correlation, )))
axes[0, 3].tick_params(axis='both', labelsize=14)
axes[0, 3].text(0.1, 0.78, textstr4, transform=axes[0, 3].transAxes, fontsize=12, bbox = dict(facecolor='white', alpha=0.7))

A=new_theta[:, 0]
B=new_theta_Si[:, 0]
textstr1 = '\n'.join((r"$r_P = %.3f$" % (np.corrcoef(A, B)[0][1], ), 
                     r"$r_S = %.3f$" % (ss.spearmanr(A, B).correlation, )))
axes[1, 0].tick_params(axis='both', labelsize=14)
axes[1, 0].text(0.5, 0.78, textstr1, transform=axes[1, 0].transAxes, fontsize=12, bbox = dict(facecolor='white', alpha=0.7))

A=(new_theta[:, 2]-ref_He)*3e8/ref_He
B=(new_theta_Si[:, 2]-ref_Si)*3e8/ref_Si
textstr2 = '\n'.join((r"$r_P = %.3f$" % (np.corrcoef(A, B)[0][1], ), 
                     r"$r_S = %.3f$" % (ss.spearmanr(A, B).correlation, )))
axes[1, 1].tick_params(axis='both', labelsize=14)
axes[1, 1].text(0.5, 0.78, textstr2, transform=axes[1, 1].transAxes, fontsize=12, bbox = dict(facecolor='white', alpha=0.7))

A=new_theta[:, 3]
B=new_theta_Si[:, 4]
textstr3 = '\n'.join((r"$r_P = %.3f$" % (np.corrcoef(A, B)[0][1], ), 
                     r"$r_S = %.3f$" % (ss.spearmanr(A, B).correlation, )))
axes[1, 2].tick_params(axis='both', labelsize=14)
axes[1, 2].text(0.5, 0.78, textstr3, transform=axes[1, 2].transAxes, fontsize=12, bbox = dict(facecolor='white', alpha=0.7))

A=new_theta[:, -1]
B=new_theta_Si[:, -1]
textstr4 = '\n'.join((r"$r_P = %.3f$" % (np.corrcoef(A, B)[0][1], ), 
                     r"$r_S = %.3f$" % (ss.spearmanr(A, B).correlation, )))
axes[1, 3].tick_params(axis='both', labelsize=14)
axes[1, 3].text(0.1, 0.78, textstr4, transform=axes[1, 3].transAxes, fontsize=12, bbox = dict(facecolor='white', alpha=0.7))

plt.subplots_adjust(hspace = 0.4, wspace=0.5)
#fig.tight_layout()
plt.savefig('/Users/samsonmercier/Downloads/test_corr.pdf')


# Correlation between He triplet and water line with new normalization


In [ ]:
#Distinguish two cases depending on the number of modes of observation.
#If there are two modes of observation.
AF.Correlation_Plot([], theta_T[:, 0], theta[:, 0], err_theta_T[:, 0], err_theta[:, 0], 'Telluric line depth', 'He line depth', 'Depth', day)

AF.Correlation_Plot([], theta_T[:, 2], theta[:, 2], err_theta_T[:, 2], err_theta[:, 2], 'Telluric line position', 'He line position', 'Position', day)

AF.Correlation_Plot([], theta_T[:, 3], theta[:, 3], err_theta_T[:, 3], err_theta[:, 3], 'Telluric line width', 'He line width', 'Width', day)

AF.Correlation_Plot([], theta_T[:, -1], theta[:, -1], [], [], 'Telluric eq. width', 'He eq. width', 'eq. width', day)

# Correlation between He triplet and other CCF properties 

## CCF RV

In [ ]:
AF.Correlation_Plot([], theta[:, 0], total_RV, err_theta[:, 0], total_RV_err, 'He line depth', 'CCF RV', 'Depth', day)

AF.Correlation_Plot([], theta[:, 2], total_RV, err_theta[:, 2], total_RV_err, 'He line position', 'CCF RV', 'Position', day)

AF.Correlation_Plot([], theta[:, 3], total_RV, err_theta[:, 3], total_RV_err, 'He line width', 'CCF RV', 'Width', day)

AF.Correlation_Plot([], theta[:, -1], total_RV, [], total_RV_err, 'He eq. width', 'CCF RV', 'eq. width', day)


## CCF Bisector Span

In [ ]:
AF.Correlation_Plot([], theta[:, 0], total_BIS_SPAN, err_theta[:, 0], total_BIS_SPAN_err, 'He line depth', 'CCF BIS SPAN', 'Depth', day)

AF.Correlation_Plot([], theta[:, 2], total_BIS_SPAN, err_theta[:, 2], total_BIS_SPAN_err, 'He line position', 'CCF BIS SPAN', 'Position', day)

AF.Correlation_Plot([], theta[:, 3], total_BIS_SPAN, err_theta[:, 3], total_BIS_SPAN_err, 'He line width', 'CCF BIS SPAN', 'Width', day)

AF.Correlation_Plot([], theta[:, -1], total_BIS_SPAN, [], total_BIS_SPAN_err, 'He eq. width', 'CCF BIS SPAN', 'eq. width', day)


## CCF FWHM

In [ ]:
AF.Correlation_Plot([], theta[:, 0], total_FWHM, err_theta[:, 0], total_FWHM_err, 'He line depth', 'CCF FWHM', 'Depth', day)

AF.Correlation_Plot([], theta[:, 2], total_FWHM, err_theta[:, 2], total_FWHM_err, 'He line position', 'CCF FWHM', 'Position', day)

AF.Correlation_Plot([], theta[:, 3], total_FWHM, err_theta[:, 3], total_FWHM_err, 'He line width', 'CCF FWHM', 'Width', day)

AF.Correlation_Plot([], theta[:, -1], total_FWHM, [], total_FWHM_err, 'He eq. width', 'CCF FWHM', 'eq. width', day)


In [ ]:
NTC_spectra = np.zeros((len(os.listdir(NTC_dir_name)), 4084))
NTC_spectra_err = np.zeros((len(os.listdir(NTC_dir_name)), 4084))
NTC_lamda = np.zeros((len(os.listdir(NTC_dir_name)), 4084))

for i in range(len(os.listdir(NTC_dir_name))):
    file = pf.open(NTC_dir_name+sorted(os.listdir(NTC_dir_name))[i])
    
    file_DLL = file[6].data[14]
    
    file_spctr = file[1].data[14]
    
    file_err = file[2].data[14]
    
    NTC_lamda[i] = file[4].data[14]
    
    #Getting the blaze file.
    blaze = pf.open(blaze_dir+'/'+os.listdir(blaze_dir)[0])

    #Getting the blaze spectra.
    blaze_spctr = blaze[1].data[14]

    #Performing the Blaze correction.
    BC_spctr = file_spctr/blaze_spctr
    BC_err = file_err/blaze_spctr

    #Removing the trend of the spectrum's continuum using DLL and the Sun's Black Body spectrum.
    BC_DLL_spctr = BC_spctr/file_DLL
    BC_DLL_err = BC_err/file_DLL

    NTC_spectra[i] = BC_DLL_spctr/np.average(AF.bound(10831.4, 10832, NTC_lamda[i], BC_DLL_spctr), weights = 1/AF.bound(10831.4, 10832, NTC_lamda[i],BC_DLL_err)**2)
    NTC_spectra_err[i] = BC_DLL_err/np.average(AF.bound(10831.4, 10832, NTC_lamda[i], BC_DLL_spctr), weights = 1/AF.bound(10831.4, 10832, NTC_lamda[i],BC_DLL_err)**2)


In [ ]:
total_NTC_lamda = AF.segment_and_reduce(tot_mode, tot_SNR, NTC_lamda, tot_RV, tot_AIRM, sig)
total_NTC_spectra = AF.segment_and_reduce(tot_mode, tot_SNR, NTC_spectra, tot_RV, tot_AIRM, sig)
total_NTC_spectra_err = AF.segment_and_reduce(tot_mode, tot_SNR, NTC_spectra_err, tot_RV, tot_AIRM, sig)

## CCF Contrast

In [ ]:
AF.Correlation_Plot([], theta[:, 0], total_CONTRAST, err_theta[:, 0], total_CONTRAST_err, 'He line depth', 'CCF CONTRAST', 'Depth', day)

AF.Correlation_Plot([], theta[:, 2], total_CONTRAST, err_theta[:, 2], total_CONTRAST_err, 'He line position', 'CCF CONTRAST', 'Position', day)

AF.Correlation_Plot([], theta[:, 3], total_CONTRAST, err_theta[:, 3], total_CONTRAST_err, 'He line width', 'CCF CONTRAST', 'Width', day)

AF.Correlation_Plot([], theta[:, -1], total_CONTRAST, [], total_CONTRAST_err, 'He eq. width', 'CCF CONTRAST', 'eq. width', day)


In [ ]:
NTC_dir_name = 'NTC-data/'
test = pf.open(NTC_dir_name+os.listdir(NTC_dir_name)[12])
test.info()

In [ ]:
total_master_NTC_spectra = np.average(total_NTC_spectra, axis=0, weights = 1/total_NTC_spectra_err**2)

In [ ]:
import math as mt 

extra = mt.floor(min(date))

bound_up_spctr = 10839
bound_down_spctr = 10825
# Create the 2D plot with colormap
plt.figure(figsize=[10, 8])

test_max = np.zeros(len(total_NTC_spectra))
test_min = np.zeros(len(total_NTC_spectra))
for i in range(len(total_NTC_spectra)):
    test_max[i] = max(AF.bound(bound_down_spctr, bound_up_spctr, total_NTC_lamda[0], total_NTC_spectra[i]))
    test_min[i] = min(AF.bound(bound_down_spctr, bound_up_spctr, total_NTC_lamda[0], total_NTC_spectra[i]))

lim_up = 1
lim_down = min(test_min)

plt.imshow(total_NTC_spectra, cmap='jet', aspect='auto', vmin = lim_down, vmax = lim_up, extent=[total_NTC_lamda[0][0], total_NTC_lamda[0][-1], date[0]-extra, date[-1]-extra])

# Set labels and title
plt.ylabel('Time (MJD -'+str(extra)+')')
plt.xlabel('Wavelength ($\AA$)')
plt.title('NTC spectra Colormap')
plt.xlim([bound_down_spctr, bound_up_spctr])
#plt.axvline(10829, color='b')
#plt.axvline(10828.3, color='b')
# Add a colorbar
plt.colorbar(label='Normalized Flux')

# Show the plot
plt.show()


# Create the 2D plot with colormap
plt.figure(figsize=[10, 8])

test_max = np.zeros(len(total_NTC_spectra))
test_min = np.zeros(len(total_NTC_spectra))
for i in range(len(total_NTC_spectra)):
    test_max[i] = max(AF.bound(bound_down_spctr, bound_up_spctr, total_NTC_lamda[0], total_NTC_spectra[i]-total_master_NTC_spectra))
    test_min[i] = min(AF.bound(bound_down_spctr, bound_up_spctr, total_NTC_lamda[0], total_NTC_spectra[i]-total_master_NTC_spectra))

lim_up = 0.01
lim_down = -0.01
plt.imshow(total_NTC_spectra - total_master_NTC_spectra, cmap='jet', aspect='auto', vmin = lim_down , vmax = lim_up, extent=[total_NTC_lamda[0][0], total_NTC_lamda[0][-1], date[0]-extra, date[-1]-extra])

# Set labels and title
plt.ylabel('Time (MJD -'+str(extra)+')')
plt.xlabel('Wavelength ($\AA$)')
plt.title('Residual Colormap')
plt.xlim([bound_down_spctr, bound_up_spctr])
# Add a colorbar
plt.colorbar(label='Normalized Flux')

# Show the plot
plt.show()

In [ ]:
plt.figure(figsize=[10, 8])
plt.plot(total_NTC_lamda[4], total_NTC_spectra[4], '.')
plt.xlim([10820, 10840])

In [ ]:
plt.figure(figsize=[10, 8])
for i in range(len(total_NTC_spectra)):
    plt.scatter(total_NTC_lamda[i], [i]*len(total_NTC_lamda[i]), c=total_NTC_spectra[i] - total_master_NTC_spectra, cmap='jet', s=5, vmin=-0.01, vmax=0.01)

#Getting the yticks
tick_dates = []
tick_positions = []
for i in range(len(date)):
    if i%40 == 0 and i!=0:
        tick_dates.append("{:.3f}".format(date[i]))
        tick_positions.append(i)

# Add labels and title
plt.yticks(tick_positions, tick_dates)
plt.xlabel('Wavelength ($\AA$)')
plt.ylabel('Time (MJD)')
plt.xlim([10825, 10835])

# Add a colorbar
plt.colorbar(label='Normalized Flux')

# Show the plot
plt.show()

In [ ]:
total_NTC_lamda[0]

In [ ]:
total_NTC_lamda[-1]

In [ ]:
%matplotlib notebook
plt.plot(total_NTC_lamda[0], total_NTC_spectra[0], 'b')
plt.plot(total_NTC_lamda[204], total_NTC_spectra[204], 'r')
plt.show()

In [ ]:
%matplotlib notebook
plt.plot(total_NTC_lamda[0], total_NTC_spectra[0], 'b')
plt.plot(total_NTC_lamda[-1], total_NTC_spectra[-1], 'r')
plt.show()

# Continuum chunks - testing section

In [ ]:
#Finding the wavelength ranges for the telluric spectra
#for h in range(0, len(total_lamda), 20):
h=10

wav_low = 10750
wav_up = 10860

#Continuum Ranges
## Defining ranges for the continuum
wav_ranges = [(10758.2, 10762.4), (10778.2, 10781.2), (10804.8, 10805.7), (10807.4, 10808.4), (10838.7, 10839.7), (10856.7, 10858.7)]

##Retrieving the spectrum, error and wavelengths in these ranges
continuu_spctr = []
continuu_wav = []
continuu_err = []
for x in wav_ranges:
    contin_spctr = AF.bound(x[0], x[1], total_lamda[h], total_norm_spctr_Antaress[h])
    contin_err = AF.bound(x[0], x[1], total_lamda[h], total_norm_err_Antaress[h])
    contin_wav = AF.bound(x[0], x[1], total_lamda[h], total_lamda[h])
    continuu_spctr.append(contin_spctr)
    continuu_wav.append(contin_wav)
    continuu_err.append(contin_err)
continuum_spctr = np.array(list(itertools.chain.from_iterable(continuu_spctr)))
continuum_err = np.array(list(itertools.chain.from_iterable(continuu_err)))
continuum_wav = np.array(list(itertools.chain.from_iterable(continuu_wav)))

##Fitting the continuum range with  a 1st order polynomial 
p = np.poly1d(np.polyfit(continuum_wav, continuum_spctr, 1, w=1/continuum_err**2))

#Plotting
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=[10, 6], sharex=True)
ax1.errorbar(total_lamda[h], total_norm_spctr_Antaress[h], yerr=total_norm_err_Antaress[h], label='Old')
#ax1.plot(total_lamda[h], p(total_lamda[h]), label='Polyn. model')
ax2.plot(tota_lamda[h], tota_telluric[h], 'k', label='New')
for i, x in enumerate(wav_ranges):
    ax1.axvline(x[0], color='r', linestyle='--')
    ax2.axvline(x[0], color='r', linestyle='--')
    ax1.axvline(x[1], color='r', linestyle='--')
    ax2.axvline(x[1], color='r', linestyle='--')
    if i==0:
        ax1.axvspan(x[0], x[1], color='r', alpha=0.3, label='Continuum Range')
    else:
        ax1.axvspan(x[0], x[1], color='r', alpha=0.3)
    ax2.axvspan(x[0], x[1], color='r', alpha=0.3)
ax1.set_ylabel('Stellar Spectrum')
ax2.set_ylabel('Telluric Spectrum')
ax2.set_xlabel('Wavelength ($\AA$)')
ax1.set_xlim(wav_low, wav_up)
ax1.legend()
ax2.legend()
fig.subplots_adjust(hspace=0)
plt.show()

# Antaress interpolation function

In [ ]:
import bindensity as bind
import numpy as np

def interpolate_with_Antaress(old_wav, new_wav, spctr, err):
    '''
    Routine to interpolate a spectrum on a new wavelength solution with bindensity/Antaress.
    Parameters
    ----------
    :param old_wav: array, 'old' wavelength solution of the spectrum in question.
    :param new_wav: array, new wavelength solution on which we want to interpolate the spectrum.
    :param spctr: array, flux values of the spectrum.
    :param err: array, error on the flux values of the spectrum.
    Returns
    ----------
    :param new_spctr: array, containing the flux values of the spectrum interpolated on the new wavelength solution.
    :param new_err: array, containing the error on the flux values of the spectrum interpolated on the 
    new wavelength solution.
    '''

    #bindensity requires the edges of the old wavelength solution as input. This means that if the size 
    #of spctr is n then the wavelength solution fed to bindensity needs to have a size of n+1.
    #Similarly, bindensity takes the edges of the new wavelength solution as input. So if the new wavelength 
    #solution you want has a size k, then the new wavelength solution fed to Antaress for the interpolation 
    #must have a size of k+1. This is why I input two new wavelength solutions, preliminary_lamda and 
    #ref_wave_Antaress, into the resampling function instead of old_wav and new_wav.
    preliminary_lamda = np.append(old_wav, old_wav[-1]+np.diff(old_wav)[-1])
    
    ref_wave_Antaress = np.append(new_wav, new_wav[-1]+np.diff(new_wav)[-1])
        
    test = bind.resampling(ref_wave_Antaress, preliminary_lamda, spctr, cov = np.array([err**2]) , kind='cubic')

    new_spctr = test[0]

    new_err = np.sqrt(test[1][0])

    return new_spctr, new_err
    

# Telluric line analysis

In [ ]:
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/HITRAN')
os.getcwd()

In [ ]:
file_CO = pf.open('Static_hitran_qt_CO.fits')
file_CO2 = pf.open('Static_hitran_qt_CO2.fits')
file_CH4 = pf.open('Static_hitran_qt_CH4.fits')
file_H2O = pf.open('Static_hitran_qt_H2O.fits')
file_O2 = pf.open('Static_hitran_qt_O2.fits')

def extract_mol_line(file):
    data = file[1].data
    wavelengths = np.zeros(len(data))
    intensity = np.zeros(len(data))
    for i in range(len(data)):
        wavelengths[i] = 1/data[i][0]
        intensity[i] = data[i][1]
    return wavelengths*1e8, intensity

wav_CO = extract_mol_line(file_CO)[0]
wav_CO2 = extract_mol_line(file_CO2)[0]
wav_CH4 = extract_mol_line(file_CH4)[0]
wav_H2O, intensity_H2O = extract_mol_line(file_H2O) 
wav_O2 = extract_mol_line(file_O2)[0]

In [ ]:
#Initially in cm!!
#We multiply by 1e8 to put it in angstrom
test = wav_H2O[wav_H2O<10900]
test2 = intensity_H2O[wav_H2O<10900]

test3 = test[test>10800]
test4 = test2[test>10800]

for i in range(len(test3)):
    print(test3[i], test4[i])